In [ ]:

# coding: utf-8

In [ ]:


from __future__ import print_function

In [ ]:


import argparse
import pickle
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import shutil

import time
import random
import torch.nn.functional as F

In [ ]:


import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from utils import Bar, Logger, AverageMeter, accuracy, mkdir_p, savefig
from pruned_layers import *
import numpy as np
from matplotlib import pyplot
# from matplotlib import pyplot
from matplotlib.pylab import plt

from privacy_risk_score_utils import *

In [ ]:


from sklearn.cluster import KMeans
from sklearn import datasets

In [ ]:


import urllib
import tarfile

In [ ]:


use_cuda = torch.cuda.is_available()
use_cuda

In [ ]:


DATASET_PATH = './datasets/texas/'

In [ ]:


if not os.path.isdir(DATASET_PATH):
    mkdir_p(DATASET_PATH)

In [ ]:


DATASET_FEATURES = os.path.join(DATASET_PATH,'texas/100/feats')
DATASET_LABELS = os.path.join(DATASET_PATH,'texas/100/labels')

In [ ]:


if not os.path.isfile(DATASET_FEATURES):
    print("Dowloading the dataset...")
    urllib.urlretrieve("https://www.comp.nus.edu.sg/~reza/files/dataset_texas.tgz",os.path.join(DATASET_PATH,'tmp.tgz'))
    print('Dataset Downloaded')
    tar = tarfile.open(os.path.join(DATASET_PATH,'tmp.tgz'))
    tar.extractall(path=DATASET_PATH)

In [ ]:


X=[]
Y=[]

In [ ]:


# data_set_features =np.genfromtxt(DATASET_FEATURES,delimiter=',')
# data_set_label =np.genfromtxt(DATASET_LABELS,delimiter=',')

# X = data_set_features.astype(np.float64)
# Y = data_set_label.astype(np.int32)-1

In [ ]:


# np.savez('texas100_data', x = X, y = Y)
# np.savez('texas100_index', x = init_r)

In [ ]:


npzdata=np.load('./texas100_data.npz')

In [ ]:



X=npzdata['x'][:,:]
Y=npzdata['y'][:]

In [ ]:


# np.array_equal(X, X_x)

In [ ]:


X.shape

In [ ]:


# # larger model
# class Texas(nn.Module):
#     def __init__(self,num_classes=100):
#         super(Texas, self).__init__()
#         self.features = nn.Sequential(
# #             nn.Linear(6169,1024),
#             PruneLinear(6169,2048),
#             nn.Tanh(),
#             PruneLinear(2048,1024),
#             nn.Tanh(),
#             PruneLinear(1024,512),
#             nn.Tanh(),
#             PruneLinear(512,256),
#             nn.Tanh(),
#             PruneLinear(256,128),
#             nn.Tanh(),
#         )
#         self.classifier = nn.Linear(128,num_classes)
# #         for key in self.state_dict():
# #             if key.split('.')[-1] == 'weight':    
# #                 nn.init.normal(self.state_dict()[key], std=0.01)
# #                 print (key)
                
# #             elif key.split('.')[-1] == 'bias':
# #                 self.state_dict()[key][...] = 0
        
#     def forward(self,x):
#         hidden_out = self.features(x)
        
#         return self.classifier(hidden_out),hidden_out

In [ ]:


# larger model
class Texas_layer_out(nn.Module):
    def __init__(self,num_classes=100):
        super(Texas_layer_out, self).__init__()
#         self.features = nn.Sequential(
#             nn.Linear(6169,1024),
        self.fc1 = PruneLinear(6169,1024)
        self.tanh1 = nn.Tanh()
#         self.fc2 = PruneLinear(2048,1024)
#         self.tanh2 = nn.Tanh()
        self.fc3 = PruneLinear(1024,512)
        self.tanh3 = nn.Tanh()
        self.fc4 = PruneLinear(512,256)
        self.tanh4 = nn.Tanh()
        self.fc5 = PruneLinear(256,128)
        self.tanh5 = nn.Tanh()
#         )
        self.classifier = nn.Linear(128,num_classes)
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        y = self.fc1(x)
        y1 = self.tanh1(y)
#         y = self.fc2(y1)
#         y2 = self.tanh2(y)
        y = self.fc3(y1)
        y3 = self.tanh3(y)
        y = self.fc4(y3)
        y4 = self.tanh4(y)
        y = self.fc5(y4)
        y5 = self.tanh5(y)
        output = self.classifier(y5)
        
        return output, y1, y3, y4, y5

In [ ]:


# larger model
class Texas_layer_out_scale(nn.Module):
    def __init__(self,num_classes=100, q = 100, alpha = 1):
        super(Texas_layer_out_scale, self).__init__()
#         self.features = nn.Sequential(
#             nn.Linear(6169,1024),
        self.fc1 = PruneLinear(6169,1024)
        self.tanh1 = nn.Tanh()
#         self.fc2 = PruneLinear(2048,1024)
#         self.tanh2 = nn.Tanh()
        self.fc3 = PruneLinear(1024,512)
        self.tanh3 = nn.Tanh()
        self.fc4 = PruneLinear(512,256)
        self.tanh4 = nn.Tanh()
        self.fc5 = PruneLinear(256,128)
        self.tanh5 = nn.Tanh()
#         )
        self.classifier = nn.Linear(128,num_classes)
    
        self.q = q
        self.alpha = alpha
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        y = self.fc1(x)
        y1 = self.tanh1(y)
        y1 = scale_by_percentage(y1, q=self.q, alpha = self.alpha)
#         y = self.fc2(y1)
#         y2 = self.tanh2(y)
        y = self.fc3(y1)
        y3 = self.tanh3(y)
        y3 = scale_by_percentage(y3, q=self.q, alpha = self.alpha)
        y = self.fc4(y3)
        y4 = self.tanh4(y)
        y4 = scale_by_percentage(y4, q=self.q, alpha = self.alpha)
        y = self.fc5(y4)
        y5 = self.tanh5(y)
        y5 = scale_by_percentage(y5, q=self.q, alpha = self.alpha)
        output = self.classifier(y5)
        
        return output, y1, y3, y4, y5

    
# def scale_by_percentage(x, q=5.0, alpha = 1):
#     """
#     scale paramters by threshold.

#     """
#     temp_shape = x.shape
#     weight = x.data.cpu().numpy()
#     flattened_weights = np.abs(weight.flatten())
#     nonzero = flattened_weights[np.nonzero(flattened_weights)]
#     percentile_value = np.percentile(nonzero, q)
#     mask = np.ones(flattened_weights.shape)
#     new_mask = np.where(flattened_weights >= percentile_value, alpha, mask)
#     new_mask = new_mask.reshape(temp_shape)
#     new_feature_map = np.asarray(weight * new_mask, dtype=np.float32)
#     x.data = torch.from_numpy(new_feature_map).cuda()
#     return x

def scale_by_percentage(x, q=5.0, alpha = 1):
    """
    scale paramters by threshold.

    """
    temp_shape = x.shape
    weight = x.data.cpu().numpy()
    flattened_weights = np.abs(weight.flatten())
#     flattened_weights = np.abs(weight)
#     percentile_value = np.percentile(flattened_weights, q)
    nonzero = flattened_weights[np.nonzero(flattened_weights)]
    percentile_value  = np.percentile(nonzero, q)
    amp = alpha - 1.
    
    tweight = x.data
    new_mask = (tweight >= percentile_value)*amp  + 1.
    
#     mask = np.ones(flattened_weights.shape)
#     new_mask = np.where(flattened_weights >= percentile_value, alpha, mask)

##     new_mask = (flattened_weights >= percentile_value)*amp + 1.

#     new_mask = new_mask.reshape(temp_shape)
#     new_feature_map = np.asarray(weight * new_mask, dtype=np.float32)
#     x.data = torch.from_numpy(new_feature_map).cuda()
    x.data  = new_mask * tweight
    return x

In [ ]:


# larger model
class Texas_layer_out_scale2(nn.Module):
    def __init__(self,num_classes=100, q = 100, alpha = 1):
        super(Texas_layer_out_scale2, self).__init__()
#         self.features = nn.Sequential(
#             nn.Linear(6169,1024),
        self.fc1 = PruneLinear(6169,1024)
        self.tanh1 = nn.Tanh()
#         self.fc2 = PruneLinear(2048,1024)
#         self.tanh2 = nn.Tanh()
        self.fc3 = PruneLinear(1024,512)
        self.tanh3 = nn.Tanh()
        self.fc4 = PruneLinear(512,256)
        self.tanh4 = nn.Tanh()
        self.fc5 = PruneLinear(256,128)
        self.tanh5 = nn.Tanh()
#         )
        self.classifier = nn.Linear(128,num_classes)
    
        self.q = q
        self.alpha = alpha
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        y = self.fc1(x)
        y1 = self.tanh1(y)
#         y1 = scale_by_percentage(y1, q=self.q, alpha = self.alpha)
#         y = self.fc2(y1)
#         y2 = self.tanh2(y)
        y = self.fc3(y1)
        y3 = self.tanh3(y)
#         y3 = scale_by_percentage(y3, q=self.q, alpha = self.alpha)
        y = self.fc4(y3)
        y4 = self.tanh4(y)
        y4 = scale_by_percentage(y4, q=self.q, alpha = self.alpha)
        y = self.fc5(y4)
        y5 = self.tanh5(y)
        y5 = scale_by_percentage(y5, q=self.q, alpha = self.alpha)
        output = self.classifier(y5)
        
        return output, y1, y3, y4, y5

    
# def scale_by_percentage(x, q=5.0, alpha = 1):
#     """
#     scale paramters by threshold.

#     """
#     temp_shape = x.shape
#     weight = x.data.cpu().numpy()
#     flattened_weights = np.abs(weight.flatten())
#     nonzero = flattened_weights[np.nonzero(flattened_weights)]
#     percentile_value = np.percentile(nonzero, q)
#     mask = np.ones(flattened_weights.shape)
#     new_mask = np.where(flattened_weights >= percentile_value, alpha, mask)
#     new_mask = new_mask.reshape(temp_shape)
#     new_feature_map = np.asarray(weight * new_mask, dtype=np.float32)
#     x.data = torch.from_numpy(new_feature_map).cuda()
#     return x

def scale_by_percentage(x, q=5.0, alpha = 1):
    """
    scale paramters by threshold.

    """
    temp_shape = x.shape
    weight = x.data.cpu().numpy()
    flattened_weights = np.abs(weight.flatten())
#     flattened_weights = np.abs(weight)
#     percentile_value = np.percentile(flattened_weights, q)
    nonzero = flattened_weights[np.nonzero(flattened_weights)]
    percentile_value  = np.percentile(nonzero, q)
    amp = alpha - 1.
    
    tweight = x.data
    new_mask = (tweight >= percentile_value)*amp  + 1.
    
#     mask = np.ones(flattened_weights.shape)
#     new_mask = np.where(flattened_weights >= percentile_value, alpha, mask)

##     new_mask = (flattened_weights >= percentile_value)*amp + 1.

#     new_mask = new_mask.reshape(temp_shape)
#     new_feature_map = np.asarray(weight * new_mask, dtype=np.float32)
#     x.data = torch.from_numpy(new_feature_map).cuda()
    x.data  = new_mask * tweight
    return x

In [ ]:


# larger model
class Texas_layer_out_scale2_mid(nn.Module):
    def __init__(self,num_classes=100, qfc_l = 100,qfc_h = 100, afc_l = 1, afc_h = 1):
        super(Texas_layer_out_scale2_mid, self).__init__()
#         self.features = nn.Sequential(
#             nn.Linear(6169,1024),
        self.fc1 = PruneLinear(6169,1024)
        self.tanh1 = nn.Tanh()
#         self.fc2 = PruneLinear(2048,1024)
#         self.tanh2 = nn.Tanh()
        self.fc3 = PruneLinear(1024,512)
        self.tanh3 = nn.Tanh()
        self.fc4 = PruneLinear(512,256)
        self.tanh4 = nn.Tanh()
        self.fc5 = PruneLinear(256,128)
        self.tanh5 = nn.Tanh()
#         )
        self.classifier = nn.Linear(128,num_classes)
    

        self.qfc_l = qfc_l
        self.qfc_h = qfc_h
        
        self.afc_l = afc_l
        self.afc_h= afc_h
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        y = self.fc1(x)
        y1 = self.tanh1(y)
#         y1 = scale_by_percentage(y1, q=self.q, alpha = self.alpha)
#         y = self.fc2(y1)
#         y2 = self.tanh2(y)
        y = self.fc3(y1)
        y3 = self.tanh3(y)
#         y3 = scale_by_percentage(y3, q=self.q, alpha = self.alpha)
        y = self.fc4(y3)
        y4 = self.tanh4(y)
        y4 = scale_by_percentage_mid(y4, q_l=self.qfc_l, q_h=self.qfc_h, alpha_l=self.afc_l, alpha_h=self.afc_h)
        y = self.fc5(y4)
        y5 = self.tanh5(y)
        y5 = scale_by_percentage_mid(y5, q_l=self.qfc_l, q_h=self.qfc_h, alpha_l=self.afc_l, alpha_h=self.afc_h)
        output = self.classifier(y5)
        
        return output, y1, y3, y4, y5

    
def scale_by_percentage(x, q=5.0, alpha = 1):
    """
    scale paramters by threshold.

    """
    temp_shape = x.shape
    weight = x.data.cpu().numpy()
    flattened_weights = np.abs(weight.flatten())
    nonzero = flattened_weights[np.nonzero(flattened_weights)]
    percentile_value = np.percentile(nonzero, q)
    mask = np.ones(flattened_weights.shape)
    new_mask = np.where(flattened_weights >= percentile_value, alpha, mask)
    new_mask = new_mask.reshape(temp_shape)
    new_feature_map = np.asarray(weight * new_mask, dtype=np.float32)
    x.data = torch.from_numpy(new_feature_map).cuda()
    return x

def scale_by_percentage_mid(x, q_l = 50, q_h = 90, alpha_l = 1, alpha_h = 1):
    """
    scale paramters by threshold.

    """
#     print('q_l: ', q_l, 'q_h: ', q_h)
    temp_shape = x.shape
    weight = x.data.cpu().numpy()
    
    flattened_weights = np.abs(weight.flatten())
    nonzero = flattened_weights[np.nonzero(flattened_weights)]
    p_value_low = np.percentile(nonzero, q_l)
    p_value_high = np.percentile(nonzero, q_h)
#     p_value_low = np.percentile(flattened_weights, q_l)
#     p_value_high = np.percentile(flattened_weights, q_h)
#     print('p_value_low: ', p_value_low, 'p_value_high: ',p_value_high)
    mask = np.ones(flattened_weights.shape)
    new_mask = np.where(flattened_weights >= p_value_low, alpha_l, mask)
#     print(new_mask)
    new_mask = np.where(flattened_weights >= p_value_high, alpha_h, new_mask)
#     print(new_mask)
    new_mask = new_mask.reshape(temp_shape)
    
    new_feature_map = np.asarray(weight * new_mask, dtype=np.float32)
    x.data = torch.from_numpy(new_feature_map).cuda()
    return x

In [ ]:


# small model
class Texas_1layer(nn.Module):
    def __init__(self,num_classes=100):
        super(Texas_1layer, self).__init__()
        self.features = nn.Sequential(
#             nn.Linear(6169,1024),
            PruneLinear(6169,256),
            nn.Tanh()
#             PruneLinear(1024,512),
#             nn.Tanh(),
#             PruneLinear(512,256),
#             nn.Tanh(),
#             PruneLinear(512,128),
#             nn.Tanh()
        )
#         self.classifier = nn.Linear(128,num_classes)
        self.classifier = PruneLinear(256,num_classes)
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        hidden_out = self.features(x)
        
        return self.classifier(hidden_out),hidden_out

In [ ]:


# small model
class Texas_2layer(nn.Module):
    def __init__(self,num_classes=100):
        super(Texas_2layer, self).__init__()
        self.features = nn.Sequential(
#             nn.Linear(6169,1024),
            PruneLinear(6169,512),
            nn.Tanh(),
#             PruneLinear(1024,512),
#             nn.Tanh(),
#             PruneLinear(512,256),
#             nn.Tanh(),
            PruneLinear(512,128),
            nn.Tanh()
        )
#         self.classifier = nn.Linear(128,num_classes)
        self.classifier = PruneLinear(128,num_classes)
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        hidden_out = self.features(x)
        
        return self.classifier(hidden_out),hidden_out

In [ ]:


# small model
class Texas(nn.Module):
    def __init__(self,num_classes=100):
        super(Texas, self).__init__()
        self.features = nn.Sequential(
#             nn.Linear(6169,1024),
            PruneLinear(6169,512),
            nn.Tanh(),
            PruneLinear(1024,512),
            nn.Tanh(),
            PruneLinear(512,256),
            nn.Tanh(),
            PruneLinear(256,128),
            nn.Tanh()
        )
#         self.classifier = nn.Linear(128,num_classes)
        self.classifier = PruneLinear(128,num_classes)
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        hidden_out = self.features(x)
        
        return self.classifier(hidden_out),hidden_out

In [ ]:


# small model
class Texas_drop(nn.Module):
    def __init__(self,p=0.5,num_classes = 100):
        self.num_classes = num_classes
        self.p = p
        super(Texas_drop, self).__init__()
        self.features = nn.Sequential(
#             nn.Linear(6169,1024),
            PruneLinear(6169,1024),
            nn.Tanh(),
            nn.Dropout2d(p),
            PruneLinear(1024,512),
            nn.Tanh(),
            nn.Dropout2d(p),
            PruneLinear(512,256),
            nn.Tanh(),
            nn.Dropout2d(p),
            PruneLinear(256,128),
            nn.Tanh(),
            nn.Dropout2d(p)
        )
#         self.classifier = nn.Linear(128,num_classes)
        self.classifier = PruneLinear(128,num_classes)
#         for key in self.state_dict():
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
        
    def forward(self,x):
        hidden_out = self.features(x)
        
        return self.classifier(hidden_out),hidden_out

In [ ]:


# class InferenceAttack_HZ(nn.Module):
#     def __init__(self,num_classes):
#         self.num_classes=num_classes
#         super(InferenceAttack_HZ, self).__init__()
#         self.features=nn.Sequential(
#             nn.Linear(num_classes,512),
#             nn.ReLU(),
#             nn.Linear(512,64),
#             nn.ReLU(),
#             )
        
  
        
#         self.labels=nn.Sequential(
#            nn.Linear(num_classes,64),
#             )
#         self.combine=nn.Sequential(
#             nn.Linear(64*2,64),
#             nn.ReLU(),
#             nn.Linear(64,1),
#             )
#         for key in self.state_dict():
#             print (key)
#             if key.split('.')[-1] == 'weight':    
#                 nn.init.normal(self.state_dict()[key], std=0.01)
#                 print (key)
                
#             elif key.split('.')[-1] == 'bias':
#                 self.state_dict()[key][...] = 0
#         self.output= nn.Sigmoid()
#     def forward(self,x1,x2,l):
#         #print (l.size(),x.size())
#         out_x1 = self.features(x1)
        
#         out_l = self.labels(l)
        
        
        
            
#         is_member =self.combine( torch.cat((out_x1,out_l),1))
        
        
#         return self.output(is_member)

In [ ]:


# defense model
class Defense_Model(nn.Module):
    def __init__(self,num_classes=1):
        super(Defense_Model, self).__init__()
        self.features = nn.Sequential(
            nn.Linear(num_classes,256),
            
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,1),
            )
        
        self.output= nn.Sigmoid()
    def forward(self,x):
        is_member = self.features(x)
        
        return self.output(is_member), is_member

In [ ]:


class InferenceAttack_HZ(nn.Module):
    def __init__(self,num_classes):
        self.num_classes=num_classes
        super(InferenceAttack_HZ, self).__init__()
        self.features=nn.Sequential(
            nn.Linear(num_classes,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,64),
            nn.ReLU(),
            )
        
  
        
        self.labels=nn.Sequential(
           nn.Linear(num_classes,512),
            nn.ReLU(),
            nn.Linear(512,64),
            nn.ReLU(),
            )
        self.combine=nn.Sequential(
            nn.Linear(64*2,512),
            
            nn.ReLU(),
            nn.Linear(512,256),
            
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,1),
            )
        for key in self.state_dict():
            print (key)
            if key.split('.')[-1] == 'weight':    
                nn.init.normal(self.state_dict()[key], std=0.01)
                print (key)
                
            elif key.split('.')[-1] == 'bias':
                self.state_dict()[key][...] = 0
        self.output= nn.Sigmoid()
    def forward(self,x1,x2,l):
        #print (l.size(),x.size())
        out_x1 = self.features(x1)
        
        out_l = self.labels(l)

            
        is_member =self.combine( torch.cat((out_x1,out_l),1))
        
        return self.output(is_member), is_member
#         return self.output(is_member)
    


In [ ]:


float(X.shape[0])
len_train =len(X)
# r = np.arange(len_train)
# np.random.shuffle(r)
# pickle.dump(r,open('./random_r_texas100','w'))
# init_r=pickle.load(open('./random_r_texas100_prune')) #for py2
init_r=pickle.load(open('./random_r_texas100_prune', 'rb'), encoding='latin1')# for py3
X=X[init_r]
Y=Y[init_r]

In [ ]:


train_classifier_ratio, train_attack_ratio = float(10000)/float(X.shape[0]),0.3
train_classifier_data = X[:int(train_classifier_ratio*len_train)]
train_attack_data = X[int(train_classifier_ratio*len_train):int((train_classifier_ratio+train_attack_ratio)*len_train)]
test_data = X[int((train_classifier_ratio+train_attack_ratio)*len_train):]

In [ ]:


train_classifier_data.shape
train_attack_data.shape
test_data.shape

In [ ]:



# train_classifier_ratio, train_attack_ratio = float(10000)/float(X.shape[0]),0.3
# train_classifier_ratio = 0.4
# train_attack_ratio = 0.3
train_classifier_data_length  = 10000

train_classifier_data = X[:train_classifier_data_length]
train_attack_data = X[10000:20000]
test_data = X[10000:]
# attacker_train_member = X[30000:35000]
# attacker_train_nonmember = X[35000:40000]
# original_train_data = X[:int((train_classifier_ratio+train_attack_ratio)*len_train)]

In [ ]:


# train_classifier_ratio, train_attack_ratio = float(10000)/float(X.shape[0]),0.3
# print('train_classifier_ratio: ', train_classifier_ratio)
# print('train_attack_ratio: ', train_attack_ratio)
# train_classifier_ratio*len_train
print('train_classifier_data: ', len(train_classifier_data))
print('train_attack_data: ', len(train_attack_data))
print('test_data: ', len(test_data))

In [ ]:


train_classifier_label = Y[:train_classifier_data_length]
train_attack_label = Y[10000:20000]
test_label = Y[10000:]
# attacker_train_member_label = Y[30000:35000]
# attacker_train_nonmember_label = Y[35000:40000]
# original_train_label = Y[:int((train_classifier_ratio+train_attack_ratio)*len_train)]

In [ ]:


print(init_r[0:200])

In [ ]:


def train(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999):
    # switch to train mode
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_,_,_,_ = model(inputs)
        
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%10==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg)
        


In [ ]:


def test(test_data,labels, model, criterion, epoch, use_cuda):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    end = time.time()
    len_t =  (len(test_data)//batch_size)
    
    for ind in range(len_t):
        # measure data loading time
        inputs = test_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs,_,_,_,_ = model(inputs)
        
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind % 100==0:
            
            print ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                        batch=ind + 1,
                        size=len(test_data),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        loss=losses.avg,
                        top1=top1.avg,
                        top5=top5.avg,
                        ))
    return (losses.avg, top1.avg)

In [ ]:


def train_mod(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        loss = criterion(outputs, targets) + alpha*(torch.mean(max_outputs - 0.85))
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg)
        


In [ ]:


def test_mod(test_data,labels, model, criterion, epoch, use_cuda, alpha = 1.0):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    # switch to evaluate mode
    model.eval()
    end = time.time()
    len_t =  (len(test_data)//batch_size)
    softmax = nn.Softmax()
    
    for ind in range(len_t):
        # measure data loading time
        inputs = test_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        loss = criterion(outputs, targets) + alpha*(torch.mean(max_outputs - 0.85))
#         loss = criterion(outputs, targets) + alpha*(torch.mean(torch.abs(0.85 - max_outputs)))

#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
#         if ind % 100==0:
            
#             print ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
#                         batch=ind + 1,
#                         size=len(test_data),
#                         data=data_time.avg,
#                         bt=batch_time.avg,
#                         loss=losses.avg,
#                         top1=top1.avg,
#                         top5=top5.avg,
#                         ))
    return (losses.avg, top1.avg)

In [ ]:


def train_mod1(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        loss = criterion(outputs, targets) + alpha*(torch.mean(torch.abs(0.85 - max_outputs)))
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg)
        


In [ ]:


def train_mod2(train_data,labels,untrain_data, untrain_labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_d = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        r= np.arange(untrain_data.shape[0])
        np.random.shuffle(r)
        untrain_inputs = untrain_data[r[:batch_size]]
        untrain_targets = untrain_labels[r[:batch_size]]

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            untrain_inputs, untrain_targets = untrain_inputs.cuda(), untrain_targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        untrain_inputs, untrain_targets = torch.autograd.Variable(untrain_inputs), torch.autograd.Variable(untrain_targets)

        # compute output
        outputs,_ = model(inputs)
        untrain_outputs,_ = model(untrain_inputs)

        soft_outputs = softmax(outputs)
        soft_untrain_outputs = softmax(untrain_outputs)

    #         max_outputs = torch.max(soft_outputs,1).values
        sort_soft_outputs,_ = torch.sort(soft_outputs)
        sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)
        loss_d = alpha*(torch.mean(torch.sum(torch.abs(sort_soft_outputs - sort_soft_untrain_outputs),1)))
        loss = criterion(outputs, targets) + loss_d
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_d.update(loss_d.data, untrain_inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |  Loss_d: {loss_d:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_d = losses_d.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, losses_d.avg, top1.avg)
        


In [ ]:


def train_mod3(train_data,labels,untrain_data, untrain_labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0, beta = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_d = AverageMeter()
    losses_a = AverageMeter()
    max_train_outputs = AverageMeter()
    max_untrain_outputs = AverageMeter()
    min_untrain_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha, ' beta: ', beta)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        r= np.arange(untrain_data.shape[0])
        np.random.shuffle(r)
        untrain_inputs = untrain_data[r[:batch_size]]
        untrain_targets = untrain_labels[r[:batch_size]]

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            untrain_inputs, untrain_targets = untrain_inputs.cuda(), untrain_targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        untrain_inputs, untrain_targets = torch.autograd.Variable(untrain_inputs), torch.autograd.Variable(untrain_targets)

        # compute output
        outputs,_ = model(inputs)
        untrain_outputs,_ = model(untrain_inputs)

        soft_outputs = softmax(outputs)
        soft_untrain_outputs = softmax(untrain_outputs)

        sort_soft_outputs,_ = torch.sort(soft_outputs)
        sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)
        loss_d = alpha*(torch.mean(torch.sum(torch.abs(sort_soft_outputs - sort_soft_untrain_outputs),1)))
#         print('loss_d: ', loss_d)
        max_soft_outputs = torch.max(soft_outputs,1).values
        max_soft_untrain_outputs = torch.mean(torch.max(sort_soft_untrain_outputs,1).values)
        min_soft_untrain_outputs = torch.min(torch.max(sort_soft_untrain_outputs,1).values)
#         print('max_soft_outputs: ', max_soft_outputs)
#         print('max_soft_untrain_outputs: ', max_soft_untrain_outputs)
#         print('min_soft_untrain_outputs: ', min_soft_untrain_outputs)
#         max_untrain_outputs = max_untrain_outputs + max_soft_untrain_outputs
        loss_a = beta*torch.mean(torch.abs(max_soft_outputs - 0.01))
#         print('loss_a: ', loss_a)
        loss = criterion(outputs, targets) + loss_d + loss_a
#         print('loss: ', loss)
#         print('inputs.size()[0]: ', inputs.size()[0])
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_d.update(loss_d.data, untrain_inputs.size()[0])
        losses_a.update(loss_a.data, untrain_inputs.size()[0])
        max_train_outputs.update(torch.mean(max_soft_outputs), inputs.size()[0])
#         max_untrain_outputs.update(max_soft_untrain_outputs, 1)
        max_untrain_outputs.update(max_soft_untrain_outputs, inputs.size()[0])
        min_untrain_outputs.update(min_soft_untrain_outputs, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        # plot progress
        if ind%100==0:
            print  ('({batch}/{size})| Loss: {loss:.4f}| Loss_d: {loss_d:.4f}| Loss_a:{loss_a:.4f}|max train:{max_train:.4f}|max untrain:{max_untrain:.4f}|min: {min_untrain:.4f}|top1:{top1: .4f}|top5:{top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
                    loss=losses.avg,
                    loss_d = losses_d.avg,
                    loss_a = losses_a.avg,
                    max_train = max_train_outputs.avg,
                    max_untrain = max_untrain_outputs.avg,    #torch.mean(max_untrain_outputs),
                    min_untrain = min_untrain_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
#             print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |  Loss_d: {loss_d:.4f} |  Loss_a: {loss_a:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
#                     batch=ind + 1,
#                     size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
#                     loss=losses.avg,
#                     loss_d = losses_d.avg,
#                     loss_a = losses_a.avg,

#                     top1=top1.avg,
#                     top5=top5.avg,
#                     ))
    return (losses.avg, losses_d.avg, losses_a.avg, top1.avg)
        


In [ ]:


def train_mod4(train_data,labels,untrain_data, untrain_labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0, beta = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_d = AverageMeter()
    losses_a = AverageMeter()
    max_train_outputs = AverageMeter()
    max_untrain_outputs = AverageMeter()
    min_untrain_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha, ' beta: ', beta)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        r= np.arange(untrain_data.shape[0])
        np.random.shuffle(r)
        untrain_inputs = untrain_data[r[:batch_size]]
        untrain_targets = untrain_labels[r[:batch_size]]

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            untrain_inputs, untrain_targets = untrain_inputs.cuda(), untrain_targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        untrain_inputs, untrain_targets = torch.autograd.Variable(untrain_inputs), torch.autograd.Variable(untrain_targets)

        # compute output
        outputs,_ = model(inputs)
        untrain_outputs,_ = model(untrain_inputs)

        soft_outputs = softmax(outputs)
        soft_untrain_outputs = softmax(untrain_outputs)

        sort_soft_outputs,_ = torch.sort(soft_outputs)
        sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)
        loss_d = alpha*(torch.mean(torch.sum(torch.abs(sort_soft_outputs - sort_soft_untrain_outputs),1)))
#         print('loss_d: ', loss_d)
        max_soft_outputs = torch.max(soft_outputs,1).values
        max_soft_untrain_outputs = torch.mean(torch.max(sort_soft_untrain_outputs,1).values)
        min_soft_untrain_outputs = torch.min(torch.max(sort_soft_untrain_outputs,1).values)
#         print('max_soft_outputs: ', max_soft_outputs)
#         print('max_soft_untrain_outputs: ', max_soft_untrain_outputs)
#         print('min_soft_untrain_outputs: ', min_soft_untrain_outputs)
#         max_untrain_outputs = max_untrain_outputs + max_soft_untrain_outputs
        loss_a = beta*torch.mean(torch.abs(max_soft_outputs - min_soft_untrain_outputs))
#         print('loss_a: ', loss_a)
        loss = criterion(outputs, targets) + loss_d + loss_a
#         print('loss: ', loss)
#         print('inputs.size()[0]: ', inputs.size()[0])
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_d.update(loss_d.data, untrain_inputs.size()[0])
        losses_a.update(loss_a.data, untrain_inputs.size()[0])
        max_train_outputs.update(torch.mean(max_soft_outputs), inputs.size()[0])
#         max_untrain_outputs.update(max_soft_untrain_outputs, 1)
        max_untrain_outputs.update(max_soft_untrain_outputs, inputs.size()[0])
        min_untrain_outputs.update(min_soft_untrain_outputs, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        # plot progress
        if ind%100==0:
            print  ('({batch}/{size})| Loss: {loss:.4f}| Loss_d: {loss_d:.4f}| Loss_a:{loss_a:.4f}|max train:{max_train:.4f}|max untrain:{max_untrain:.4f}|min: {min_untrain:.4f}|top1:{top1: .4f}|top5:{top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
                    loss=losses.avg,
                    loss_d = losses_d.avg,
                    loss_a = losses_a.avg,
                    max_train = max_train_outputs.avg,
                    max_untrain = max_untrain_outputs.avg,    #torch.mean(max_untrain_outputs),
                    min_untrain = min_untrain_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
#             print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |  Loss_d: {loss_d:.4f} |  Loss_a: {loss_a:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
#                     batch=ind + 1,
#                     size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
#                     loss=losses.avg,
#                     loss_d = losses_d.avg,
#                     loss_a = losses_a.avg,

#                     top1=top1.avg,
#                     top5=top5.avg,
#                     ))
    return (losses.avg, losses_d.avg, losses_a.avg, top1.avg)
        


In [ ]:


def train_mod5(train_data,labels,untrain_data, untrain_labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0, beta1 = 1.0, beta2 = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_d = AverageMeter()
    losses_min = AverageMeter()
    losses_max = AverageMeter()
    max_train_outputs = AverageMeter()
    max_untrain_outputs = AverageMeter()
    min_untrain_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha, ' beta: ', beta)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        r= np.arange(untrain_data.shape[0])
        np.random.shuffle(r)
        untrain_inputs = untrain_data[r[:batch_size]]
        untrain_targets = untrain_labels[r[:batch_size]]

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            untrain_inputs, untrain_targets = untrain_inputs.cuda(), untrain_targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        untrain_inputs, untrain_targets = torch.autograd.Variable(untrain_inputs), torch.autograd.Variable(untrain_targets)

        # compute output
        outputs,_ = model(inputs)
        untrain_outputs,_ = model(untrain_inputs)

        soft_outputs = softmax(outputs)
        soft_untrain_outputs = softmax(untrain_outputs)

        sort_soft_outputs,_ = torch.sort(soft_outputs)
        sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)
        loss_d = alpha*(torch.mean(torch.sum(torch.abs(sort_soft_outputs - sort_soft_untrain_outputs),1)))
#         print('loss_d: ', loss_d)
        max_soft_outputs = torch.max(soft_outputs,1).values
        max_soft_untrain_outputs = torch.mean(torch.max(sort_soft_untrain_outputs,1).values)
        min_soft_untrain_outputs = torch.min(torch.max(sort_soft_untrain_outputs,1).values)
#         print('max_soft_outputs: ', max_soft_outputs)
#         print('max_soft_untrain_outputs: ', max_soft_untrain_outputs)
#         print('min_soft_untrain_outputs: ', min_soft_untrain_outputs)
#         max_untrain_outputs = max_untrain_outputs + max_soft_untrain_outputs
        loss_min = beta1*torch.mean((max_soft_outputs - min_soft_untrain_outputs))
        loss_max = beta2*torch.mean((max_soft_outputs - max_soft_untrain_outputs))
#         print('loss_a: ', loss_a)
        loss = criterion(outputs, targets) + loss_d + loss_min + loss_max
#         print('loss: ', loss)
#         print('inputs.size()[0]: ', inputs.size()[0])
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_d.update(loss_d.data, untrain_inputs.size()[0])
        losses_min.update(loss_min.data, untrain_inputs.size()[0])
        losses_max.update(loss_max.data, untrain_inputs.size()[0])
        max_train_outputs.update(torch.mean(max_soft_outputs), inputs.size()[0])
#         max_untrain_outputs.update(max_soft_untrain_outputs, 1)
        max_untrain_outputs.update(max_soft_untrain_outputs, inputs.size()[0])
        min_untrain_outputs.update(min_soft_untrain_outputs, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        # plot progress
        if ind%100==0:
            print  ('({batch}/{size})| Loss: {loss:.4f}|Loss_d: {loss_d:.4f}|L_max:{loss_max:.4f}| L_min:{loss_min:.4f}|train:{max_train:.4f}|max:{max_untrain:.4f}|min: {min_untrain:.4f}|top1:{top1: .4f}|top5:{top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
                    loss=losses.avg,
                    loss_d = losses_d.avg,
                    loss_max = losses_max.avg,
                    loss_min = losses_min.avg,
                    max_train = max_train_outputs.avg,
                    max_untrain = max_untrain_outputs.avg,    #torch.mean(max_untrain_outputs),
                    min_untrain = min_untrain_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
#             print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |  Loss_d: {loss_d:.4f} |  Loss_a: {loss_a:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
#                     batch=ind + 1,
#                     size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
#                     loss=losses.avg,
#                     loss_d = losses_d.avg,
#                     loss_a = losses_a.avg,

#                     top1=top1.avg,
#                     top5=top5.avg,
#                     ))
    return (losses.avg, losses_d.avg, losses_max.avg,losses_min.avg, top1.avg)
        


In [ ]:


def train_mod7(train_data,labels,untrain_data, untrain_labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0, beta = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_d = AverageMeter()
    losses_a = AverageMeter()
    losses_ce = AverageMeter()
    max_train_outputs = AverageMeter()
    max_untrain_outputs = AverageMeter()
    min_untrain_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha, ' beta: ', beta)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        r= np.arange(untrain_data.shape[0])
        np.random.shuffle(r)
        untrain_inputs = untrain_data[r[:batch_size]]
        untrain_targets = untrain_labels[r[:batch_size]]

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            untrain_inputs, untrain_targets = untrain_inputs.cuda(), untrain_targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        untrain_inputs, untrain_targets = torch.autograd.Variable(untrain_inputs), torch.autograd.Variable(untrain_targets)

        # compute output
        outputs,_ = model(inputs)
        untrain_outputs,_ = model(untrain_inputs)

        soft_outputs = softmax(outputs)
        soft_untrain_outputs = softmax(untrain_outputs)

        sort_outputs,_ = torch.sort(outputs)
        sort_untrain_outputs,_ = torch.sort(untrain_outputs)
        
        sort_soft_outputs,_ = torch.sort(soft_outputs)
        sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)
        
        mean_sort_soft_untrain_outputs = torch.mean(sort_soft_untrain_outputs, 0)
        loss_ce = -torch.mean(torch.sum(mean_sort_soft_untrain_outputs*torch.log(sort_soft_outputs),1))
        
        loss_d = alpha*(torch.mean(torch.sum(torch.abs(sort_outputs - sort_untrain_outputs),1)))

#         print('loss_d: ', loss_d)
        max_soft_outputs = torch.max(sort_outputs,1).values
        max_soft_untrain_outputs = torch.mean(torch.max(sort_untrain_outputs,1).values)
        min_soft_untrain_outputs = torch.min(torch.max(sort_untrain_outputs,1).values)
#         print('max_soft_outputs: ', max_soft_outputs)
#         print('max_soft_untrain_outputs: ', max_soft_untrain_outputs)
#         print('min_soft_untrain_outputs: ', min_soft_untrain_outputs)
#         max_untrain_outputs = max_untrain_outputs + max_soft_untrain_outputs
        loss_a = beta*torch.mean(torch.abs(max_soft_outputs - min_soft_untrain_outputs))
#         print('loss_a: ', loss_a)
#         loss = criterion(outputs, targets) + loss_d + loss_a
        loss = criterion(outputs, targets) + alpha*loss_ce
#         print('loss: ', loss)
#         print('inputs.size()[0]: ', inputs.size()[0])
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_d.update(loss_d.data, untrain_inputs.size()[0])
        losses_a.update(loss_a.data, untrain_inputs.size()[0])
        losses_ce.update(loss_ce.data, untrain_inputs.size()[0])
        max_train_outputs.update(torch.mean(max_soft_outputs), inputs.size()[0])
#         max_untrain_outputs.update(max_soft_untrain_outputs, 1)
        max_untrain_outputs.update(max_soft_untrain_outputs, inputs.size()[0])
        min_untrain_outputs.update(min_soft_untrain_outputs, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        # plot progress
        if ind%100==0:
            print  ('({batch}/{size})| Loss: {loss:.4f} Loss_ce: {loss_ce:.4f}|| Loss_d: {loss_d:.4f}|max train:{max_train:.4f}|max untrain:{max_untrain:.4f}|min: {min_untrain:.4f}|top1:{top1: .4f}|top5:{top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
                    loss=losses.avg,
                    loss_ce = losses_ce.avg,
                    loss_d = losses_d.avg,
#                     loss_a = losses_a.avg,
                    max_train = max_train_outputs.avg,
                    max_untrain = max_untrain_outputs.avg,    #torch.mean(max_untrain_outputs),
                    min_untrain = min_untrain_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
#             print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |  Loss_d: {loss_d:.4f} |  Loss_a: {loss_a:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
#                     batch=ind + 1,
#                     size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
#                     loss=losses.avg,
#                     loss_d = losses_d.avg,
#                     loss_a = losses_a.avg,

#                     top1=top1.avg,
#                     top5=top5.avg,
#                     ))
    return (losses.avg, losses_d.avg, losses_ce.avg, top1.avg)
        


In [ ]:


def train_privatly(train_data,labels, model,inference_model, criterion, optimizer, epoch, use_cuda,num_batchs=10000,skip_batch=0,alpha=0.5):
    # switch to train mode
    model.train()
    inference_model.eval()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()
    
    
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(skip_batch,len_t):
        
        if ind >= skip_batch+num_batchs:
            break
        
        # measure data loading time
        
        #print (ind)
        
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        
        
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,h_layer = model(inputs)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((outputs.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
        
        
        
        inference_output = inference_model ( outputs,h_layer,infer_input_one_hot)
        #print (inference_output.mean())
        
        relu = nn.ReLU()
        loss = criterion(outputs, targets) + ((alpha)*(torch.mean((inference_output)) - 0.5))

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  (alpha, '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg)

In [ ]:


def outputres(test_data,labels, model, criterion, epoch, use_cuda):
    

    # switch to evaluate mode
    model.eval()
    end = time.time()
    len_t =  (len(test_data)//batch_size)-1
    alloutputs = np.zeros((batch_size*(len_t),100))
    for ind in range(len_t):
        # measure data loading time
        inputs = test_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        alloutputs[ind*batch_size:(ind+1)*batch_size,:]=F.softmax(outputs,dim=1).data.cpu().numpy()
    return alloutputs

In [ ]:


def save_checkpoint(state, is_best, checkpoint='./checkpoints_texas_10class_little_model_defense/', filename='checkpoint.pth.tar'):
    if not os.path.isdir(checkpoint):
        mkdir_p(checkpoint)
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [ ]:


best_acc = 0.0

In [ ]:


epochs=100

In [ ]:


def train_attack(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda,num_batchs=100000,skip_batch=0):
    # switch to train mode
    model.eval()
    attack_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    #print (skip_batch, len_t)
    
    for ind in range(skip_batch, len_t):
        
        if ind >= skip_batch+num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
            
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)
        # compute output
        outputs, h_layer = model(inputs)
        outputs_non, h_layer_non = model(inputs_attack)
        
        classifier_input = torch.cat((inputs,inputs_attack))
        comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
        if use_cuda:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
        else:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        attack_input = comb_inputs #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
#         sf= nn.Softmax(dim=0)
        
#         att_inp=torch.stack([attack_input, infer_input_one_hot],1)
        
        
#         att_inp = att_inp.view([attack_input.size()[0],1,2,attack_input.size(1)])
        
        #attack_output = attack_model(att_inp).view([-1])
        attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        
#         classifier_targets = comb_targets.clone().view([-1]).type(torch.cuda.LongTensor)
        
        loss_attack = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss_attack.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        #print ( attack_output.data.cpu().numpy(),v_is_member_labels.data.cpu().numpy() ,attack_input.data.cpu().numpy())
        #raise
        
        
        
        # compute gradient and do SGD step
        attack_optimizer.zero_grad()
        loss_attack.backward()
        attack_optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
    return (losses.avg, top1.avg)

In [ ]:


def test_attack(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)

        # compute output
        outputs,h_layer = model(inputs)
        outputs_non,h_layer_non = model(inputs_attack)
        
        comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
        if use_cuda:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
        else:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        attack_input = comb_inputs #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
        #attack_output = attack_model(att_inp).view([-1])
        attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        
        
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        sum_correct += correct
        #raise
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
            
#         break
#     return (losses.avg, top1.avg)  #,prec1,attack_output,  v_is_member_labels)
        
    return (losses.avg, top1.avg, sum_correct)

    return 

In [ ]:


def train_attack_softmax(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda,num_batchs=100000,skip_batch=0):
    # switch to train mode
    model.eval()
    attack_model.train()
    
    softmax = nn.Softmax()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    #print (skip_batch, len_t)
    
    for ind in range(skip_batch, len_t):
        
        if ind >= skip_batch+num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
            
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)
        # compute output
        outputs,_,_,_, h_layer = model(inputs)
        outputs_non,_,_,_, h_layer_non = model(inputs_attack)
        
        classifier_input = torch.cat((inputs,inputs_attack))
        comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
        if use_cuda:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
        else:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        attack_input = softmax(comb_inputs) #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
#         sf= nn.Softmax(dim=0)
        
#         att_inp=torch.stack([attack_input, infer_input_one_hot],1)
        
        
#         att_inp = att_inp.view([attack_input.size()[0],1,2,attack_input.size(1)])
        
        #attack_output = attack_model(att_inp).view([-1])
#         attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
#         print(attack_input.shape, comb_inputs_h.shape, infer_input_one_hot.shape)
        attack_output, _ = attack_model(attack_input,comb_inputs_h,infer_input_one_hot)
        attack_output = attack_output.view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        
#         classifier_targets = comb_targets.clone().view([-1]).type(torch.cuda.LongTensor)
        
        loss_attack = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss_attack.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        #print ( attack_output.data.cpu().numpy(),v_is_member_labels.data.cpu().numpy() ,attack_input.data.cpu().numpy())
        #raise
        
        
        
        # compute gradient and do SGD step
        attack_optimizer.zero_grad()
        loss_attack.backward()
        attack_optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
    return (losses.avg, top1.avg)

In [ ]:


def test_attack_softmax(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)

        # compute output
        outputs,_,_,_,h_layer = model(inputs)
        outputs_non,_,_,_,h_layer_non = model(inputs_attack)
        
        comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
        if use_cuda:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
        else:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        attack_input = softmax(comb_inputs) #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
        #attack_output = attack_model(att_inp).view([-1])
        attack_output, _ = attack_model(attack_input,comb_inputs_h,infer_input_one_hot)
        attack_output = attack_output.view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        
        
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        sum_correct += correct
        #raise
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
            
#         break
#     return (losses.avg, top1.avg)  #,prec1,attack_output,  v_is_member_labels)
        
    return (losses.avg, top1.avg, sum_correct)

    return 

In [ ]:


def test_attack_softmax_modify(train_data,labels, train_logits,attack_data,attack_label,test_logits, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        train_attack_logits = train_logits[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        test_attack_logits = test_logits[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
            train_attack_logits, test_attack_logits = train_attack_logits.cuda(), test_attack_logits.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)
        train_attack_logits , test_attack_logits = torch.autograd.Variable(train_attack_logits), torch.autograd.Variable(test_attack_logits)
        
        # compute output
        outputs,h_layer = model(inputs)
        outputs_non,h_layer_non = model(inputs_attack)
        
        comb_inputs_h = torch.cat((h_layer,h_layer_non))
        
        comb_inputs = torch.cat((train_attack_logits,test_attack_logits))
        
        if use_cuda:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
        else:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        attack_input = softmax(comb_inputs) #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        #attack_output = attack_model(att_inp).view([-1])
#         attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
        attack_output, _ = attack_model(attack_input,comb_inputs_h,infer_input_one_hot)
        attack_output = attack_output.view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((train_attack_logits.size()[0]+test_attack_logits.size()[0]))
        att_labels [:train_attack_logits.size()[0]] =1.0
        att_labels [train_attack_logits.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
    
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        sum_correct += correct
        #raise
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
            
#         break
#     return (losses.avg, top1.avg)  #,prec1,attack_output,  v_is_member_labels)
        
    return (losses.avg, top1.avg, sum_correct)

    return 

In [ ]:


def attack_test_trainset(train_data,labels, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    
    end = time.time()
#     len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))-1
    len_t = len(train_data)//batch_size + 1
    
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
#         inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
#         targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
#         #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
#             inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
#         inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)

        # compute output
        outputs,h_layer = model(inputs)
#         outputs_non,h_layer_non = model(inputs_attack)
        
        comb_inputs_h = h_layer  #torch.cat((h_layer,h_layer_non))
        comb_inputs = outputs  #torch.cat((outputs,outputs_non))
        
        if use_cuda:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
            comb_targets= targets.view([-1,1]).type(torch.cuda.FloatTensor)
        else:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            comb_targets= targets.view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        attack_input = comb_inputs #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
#         target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
        #attack_output = attack_model(att_inp).view([-1])
        attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0])) #+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
#         att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        sum_correct += correct
#         print('prec1: ', prec1)
        #raise
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
#         break
            
    return (losses.avg, top1.avg, sum_correct)
        
    return

In [ ]:


def attack_test_testset(attack_data,labels, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    end = time.time()
#     len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))-1
    len_t = len(attack_data)//batch_size + 1
    
    for ind in range(len_t):
        # measure data loading time
        inputs = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
#         inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
#         targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
#         #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
#             inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
#         inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)

        # compute output
        outputs,h_layer = model(inputs)
#         outputs_non,h_layer_non = model(inputs_attack)
        
        comb_inputs_h = h_layer  #torch.cat((h_layer,h_layer_non))
        comb_inputs = outputs  #torch.cat((outputs,outputs_non))
        
        if use_cuda:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
            comb_targets= targets.view([-1,1]).type(torch.cuda.FloatTensor)
        else:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            comb_targets= targets.view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        attack_input = comb_inputs #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
#         target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
        #attack_output = attack_model(att_inp).view([-1])
        attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0])) #+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =0.0
#         att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() > 0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
#         print('prec1: ', prec1)
        #raise
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() > 0.5),(v_is_member_labels.data.cpu().numpy() > 0.5)))
        sum_correct += correct
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
#         break
    return (losses.avg, top1.avg, sum_correct) #, loss, attack_input)
        
    return

In [ ]:


def train_defense_softmax(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda,num_batchs=100000,skip_batch=0):
    # switch to train mode
    model.eval()
    attack_model.train()
    
    softmax = nn.Softmax()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    #print (skip_batch, len_t)
    
    for ind in range(skip_batch, len_t):
        
        if ind >= skip_batch+num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
            
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)
        # compute output
        outputs, _,_,_, h_layer = model(inputs)
        outputs_non, _,_,_, h_layer_non = model(inputs_attack)
        
        classifier_input = torch.cat((inputs,inputs_attack))
#         comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
#         if use_cuda:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
#         else:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
#         print (comb_inputs.size(),comb_targets.size())
#         print('comb_inputs.shape:',comb_inputs.shape)
#         print('comb_inputs:', comb_inputs[0,:])
        sort_inputs, indices= torch.sort(comb_inputs)
#         print('sort_inputs.shape:',sort_inputs.shape)
#         print('sort_inputs:', sort_inputs[0,:])
#         print(torch.sort(comb_inputs[0,:]))
#         break
        attack_input = softmax(sort_inputs) #torch.cat((comb_inputs,comb_targets),1)
#         print('attack_input:', attack_input[0,:])
#         print('attack_input.shape:',attack_input.shape)
#         break
#         one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
#         target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
#         infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
#         sf= nn.Softmax(dim=0)
        
#         att_inp=torch.stack([attack_input, infer_input_one_hot],1)
        
        
#         att_inp = att_inp.view([attack_input.size()[0],1,2,attack_input.size(1)])
        
        #attack_output = attack_model(att_inp).view([-1])
#         attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
#         attack_output = attack_model(attack_input).view([-1])
        attack_output,_ = attack_model(attack_input)
        attack_output = attack_output.view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        
#         classifier_targets = comb_targets.clone().view([-1]).type(torch.cuda.LongTensor)
        
        loss_attack = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss_attack.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        #print ( attack_output.data.cpu().numpy(),v_is_member_labels.data.cpu().numpy() ,attack_input.data.cpu().numpy())
        #raise
        
        
        # compute gradient and do SGD step
        attack_optimizer.zero_grad()
        loss_attack.backward()
        attack_optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
    return (losses.avg, top1.avg)

In [ ]:


def test_defense_softmax(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
#     print(len_t)
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)

        # compute output
        outputs, _,_,_,h_layer = model(inputs)
        outputs_non, _,_,_,h_layer_non = model(inputs_attack)
        
#         comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
#         if use_cuda:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
#         else:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        sort_inputs, indices= torch.sort(comb_inputs)
        attack_input = softmax(sort_inputs) #torch.cat((comb_inputs,comb_targets),1)
        
        
#         one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
#         target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
#         infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
        #attack_output = attack_model(att_inp).view([-1])
#         attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
#         attack_output = attack_model(attack_input).view([-1])
        attack_output,_ = attack_model(attack_input)
        attack_output = attack_output.view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)

        
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        sum_correct += correct
        #raise
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
            
#         break
#     return (losses.avg, top1.avg)  #,prec1,attack_output,  v_is_member_labels)
        
    return (losses.avg, top1.avg, sum_correct)

    return 

In [ ]:


def test_defense_softmax_unsort(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
#     print(len_t)
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)

        # compute output
        outputs,h_layer = model(inputs)
        outputs_non,h_layer_non = model(inputs_attack)
        
#         comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
#         if use_cuda:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
#         else:
#             comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
#         ---------------sort-------------------------------
#         sort_inputs, indices= torch.sort(comb_inputs)
#         attack_input = softmax(sort_inputs) #torch.cat((comb_inputs,comb_targets),1)
#         ---------------sort-------------------------------
        attack_input = softmax(comb_inputs)
        
#         one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
#         target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
#         infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
         
        
        
        #attack_output = attack_model(att_inp).view([-1])
#         attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
#         attack_output = attack_model(attack_input).view([-1])
        attack_output,_ = attack_model(attack_input)
        attack_output = attack_output.view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)

        
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        sum_correct += correct
        #raise
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
            
#         break
#     return (losses.avg, top1.avg)  #,prec1,attack_output,  v_is_member_labels)
        
    return (losses.avg, top1.avg, sum_correct)

    return 

In [ ]:


classes = 100

# Complement Entropy (CE)


class ComplementEntropy(nn.Module):

    def __init__(self):
        super(ComplementEntropy, self).__init__()

    # here we implemented step by step for corresponding to our formula
    # described in the paper
    def forward(self, yHat, y):
        self.batch_size = len(y)
        self.classes = classes
        yHat = F.softmax(yHat, dim=1)
        Yg = torch.gather(yHat, 1, torch.unsqueeze(y, 1))
        Yg_ = (1 - Yg) + 1e-7  # avoiding numerical issues (first)
        Px = yHat / Yg_.view(len(yHat), 1)
        Px_log = torch.log(Px + 1e-10)  # avoiding numerical issues (second)
        y_zerohot = torch.ones(self.batch_size, self.classes).scatter_(
            1, y.view(self.batch_size, 1).data.cpu(), 0)
        output = Px * Px_log * y_zerohot.cuda()
        loss = torch.sum(output)
        loss /= float(self.batch_size)
        loss /= float(self.classes)
        return loss

In [ ]:


classes = 100

class GuidedComplementEntropy(nn.Module):

    def __init__(self, alpha):
        super(GuidedComplementEntropy, self).__init__()
        self.alpha = alpha

    # here we implemented step by step for corresponding to our formula
    # described in the paper
    def forward(self, yHat, y):
        self.batch_size = len(y)
        self.classes = classes
        yHat = F.softmax(yHat, dim=1)
        Yg = torch.gather(yHat, 1, torch.unsqueeze(y, 1))
        Yg_ = (1 - Yg) + 1e-7  # avoiding numerical issues (first)
        # avoiding numerical issues (second)
        guided_factor = (Yg + 1e-7) ** self.alpha
        Px = yHat / Yg_.view(len(yHat), 1)
        Px_log = torch.log(Px + 1e-10)  # avoiding numerical issues (third)
        y_zerohot = torch.ones(self.batch_size, self.classes).scatter_(
            1, y.view(self.batch_size, 1).data.cpu(), 0)
        output = Px * Px_log * y_zerohot.cuda()
        guided_output = guided_factor.squeeze() * torch.sum(output, dim=1)
        loss = torch.sum(guided_output)
        loss /= float(self.batch_size)
        loss /= math.log(float(self.classes))
        return loss

In [ ]:


def train_mod8(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
        mean_outputs = torch.mean(soft_outputs)
        var = torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))
        
        loss = criterion(outputs, targets) + alpha*var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s| Loss: {loss:.4f} | Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def train_mod9(train_data,labels,untrain_data, untrain_labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0, beta = 1.0,lamd = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_d = AverageMeter()
    losses_a = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    max_untrain_outputs = AverageMeter()
    min_untrain_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha, ' beta: ', beta, ', lamd: ',lamd)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        r= np.arange(untrain_data.shape[0])
        np.random.shuffle(r)
        untrain_inputs = untrain_data[r[:batch_size]]
        untrain_targets = untrain_labels[r[:batch_size]]

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            untrain_inputs, untrain_targets = untrain_inputs.cuda(), untrain_targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        untrain_inputs, untrain_targets = torch.autograd.Variable(untrain_inputs), torch.autograd.Variable(untrain_targets)

        # compute output
        outputs,_ = model(inputs)
        untrain_outputs,_ = model(untrain_inputs)

        soft_outputs = softmax(outputs)
        soft_untrain_outputs = softmax(untrain_outputs)

        sort_soft_outputs,_ = torch.sort(soft_outputs)
        sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)
        
        mean_outputs = torch.mean(soft_outputs)
        var = torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))
        
        loss_d = alpha*(torch.mean(torch.sum(torch.abs(sort_soft_outputs - sort_soft_untrain_outputs),1)))
#         print('loss_d: ', loss_d)
        max_soft_outputs = torch.max(soft_outputs,1).values
        max_soft_untrain_outputs = torch.mean(torch.max(sort_soft_untrain_outputs,1).values)
        min_soft_untrain_outputs = torch.min(torch.max(sort_soft_untrain_outputs,1).values)
#         print('max_soft_outputs: ', max_soft_outputs)
#         print('max_soft_untrain_outputs: ', max_soft_untrain_outputs)
#         print('min_soft_untrain_outputs: ', min_soft_untrain_outputs)
#         max_untrain_outputs = max_untrain_outputs + max_soft_untrain_outputs
        loss_a = beta*torch.mean(torch.abs(max_soft_outputs - min_soft_untrain_outputs))
#         print('loss_a: ', loss_a)
        loss = criterion(outputs, targets) + loss_d + loss_a + lamd*var
#         print('loss: ', loss)
#         print('inputs.size()[0]: ', inputs.size()[0])
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_d.update(loss_d.data, untrain_inputs.size()[0])
        losses_a.update(loss_a.data, untrain_inputs.size()[0])
        max_train_outputs.update(torch.mean(max_soft_outputs), inputs.size()[0])
#         max_untrain_outputs.update(max_soft_untrain_outputs, 1)
        max_untrain_outputs.update(max_soft_untrain_outputs, inputs.size()[0])
        min_untrain_outputs.update(min_soft_untrain_outputs, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        # plot progress
        if ind%100==0:
            print  ('({batch}/{size})| Loss: {loss:.4f}| Loss_d: {loss_d:.4f}| Loss_a:{loss_a:.4f}|max train:{max_train:.4f}|max untrain:{max_untrain:.4f}|min: {min_untrain:.4f}|top1:{top1: .4f}|top5:{top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
                    loss=losses.avg,
                    loss_d = losses_d.avg,
                    loss_a = losses_a.avg,
                    max_train = max_train_outputs.avg,
                    max_untrain = max_untrain_outputs.avg,    #torch.mean(max_untrain_outputs),
                    min_untrain = min_untrain_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
#             print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |  Loss_d: {loss_d:.4f} |  Loss_a: {loss_a:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
#                     batch=ind + 1,
#                     size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
#                     loss=losses.avg,
#                     loss_d = losses_d.avg,
#                     loss_a = losses_a.avg,

#                     top1=top1.avg,
#                     top5=top5.avg,
#                     ))
    return (losses.avg, losses_d.avg, losses_a.avg, losses_var.avg, top1.avg)
        


In [ ]:


def test_mod8(test_data,labels, model, criterion, epoch, use_cuda):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    softmax = nn.Softmax()
    end = time.time()
    len_t =  (len(test_data)//batch_size)
    
    for ind in range(len_t):
        # measure data loading time
        inputs = test_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        loss = criterion(outputs, targets)
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
        mean_outputs = torch.mean(soft_outputs)
        var = torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind % 100==0:
            
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s| Loss: {loss:.4f} | Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, losses_var.avg, top1.avg)

In [ ]:


def train_attack_softmax_sort_mod(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda,num_batchs=100000,skip_batch=0):
    # switch to train mode
    model.eval()
    attack_model.train()
    
    softmax = nn.Softmax()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    #print (skip_batch, len_t)
    
    for ind in range(skip_batch, len_t):
        
        if ind >= skip_batch+num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
            
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)
        # compute output
        outputs, h_layer = model(inputs)
        outputs_non, h_layer_non = model(inputs_attack)
        
        classifier_input = torch.cat((inputs,inputs_attack))
        comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
        if use_cuda:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
        else:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        
        sort_inputs, indices= torch.sort(comb_inputs)
        
        attack_input = softmax(sort_inputs) #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        for i in range(len(indices)):
            target_one_hot_tr[i] = target_one_hot_tr[i][indices[i]]
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
        
        
#         sf= nn.Softmax(dim=0)
        
#         att_inp=torch.stack([attack_input, infer_input_one_hot],1)
        
        
#         att_inp = att_inp.view([attack_input.size()[0],1,2,attack_input.size(1)])
        
        #attack_output = attack_model(att_inp).view([-1])
#         attack_output = attack_model(attack_input,comb_inputs_h,infer_input_one_hot).view([-1])
        attack_output, _ = attack_model(attack_input,comb_inputs_h,infer_input_one_hot)
        attack_output = attack_output.view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        
#         classifier_targets = comb_targets.clone().view([-1]).type(torch.cuda.LongTensor)
        
        loss_attack = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss_attack.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        #print ( attack_output.data.cpu().numpy(),v_is_member_labels.data.cpu().numpy() ,attack_input.data.cpu().numpy())
        #raise
        
        
        
        # compute gradient and do SGD step
        attack_optimizer.zero_grad()
        loss_attack.backward()
        attack_optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
    return (losses.avg, top1.avg)

In [ ]:


def test_attack_softmax_sort_mod(train_data,labels,attack_data,attack_label, model,attack_model, criterion,attack_criterion, optimizer,attack_optimizer, epoch, use_cuda):
    model.eval()
    attack_model.eval()
    
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    sum_correct = 0.0
    
    end = time.time()
    len_t =  min((len(attack_data)//batch_size) ,(len(train_data)//batch_size))+1
    
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        
        inputs_attack = attack_data[ind*batch_size:(ind+1)*batch_size]
        targets_attack = attack_label[ind*batch_size:(ind+1)*batch_size]
        #print ( len(targets_attack), len(targets))
        
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs_attack , targets_attack = inputs_attack.cuda(), targets_attack.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        inputs_attack , targets_attack = torch.autograd.Variable(inputs_attack), torch.autograd.Variable(targets_attack)

        # compute output
        outputs,h_layer = model(inputs)
        outputs_non,h_layer_non = model(inputs_attack)
        
        comb_inputs_h = torch.cat((h_layer,h_layer_non))
        comb_inputs = torch.cat((outputs,outputs_non))
        
        if use_cuda:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.cuda.FloatTensor)
        else:
            comb_targets= torch.cat((targets,targets_attack)).view([-1,1]).type(torch.FloatTensor)
            
        #print (comb_inputs.size(),comb_targets.size())
        sort_inputs, indices= torch.sort(comb_inputs)
        
        attack_input = softmax(sort_inputs) #torch.cat((comb_inputs,comb_targets),1)
        
        
        one_hot_tr = torch.from_numpy((np.zeros((attack_input.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
        target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((targets,targets_attack)).type(torch.cuda.LongTensor).view([-1,1]).data,1)
        
        for i in range(len(indices)):
            target_one_hot_tr[i] = target_one_hot_tr[i][indices[i]]
        
        infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
        
        
        #attack_output = attack_model(att_inp).view([-1])
        attack_output, _ = attack_model(attack_input,comb_inputs_h,infer_input_one_hot)
        attack_output = attack_output.view([-1])
        #attack_output = attack_model(attack_input).view([-1])
        att_labels = np.zeros((inputs.size()[0]+inputs_attack.size()[0]))
        att_labels [:inputs.size()[0]] =1.0
        att_labels [inputs.size()[0]:] =0.0
        is_member_labels = torch.from_numpy(att_labels).type(torch.FloatTensor)
        if use_cuda:
            is_member_labels = is_member_labels.cuda()
        
        v_is_member_labels = torch.autograd.Variable(is_member_labels)
        
        
        
        loss = attack_criterion(attack_output, v_is_member_labels)
        

        # measure accuracy and record loss
        #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
        
        prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        losses.update(loss.data, attack_input.size()[0])
        top1.update(prec1, attack_input.size()[0])
        
        correct = np.sum(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
        sum_correct += correct
        #raise
        
        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    ))
            
#         break
#     return (losses.avg, top1.avg)  #,prec1,attack_output,  v_is_member_labels)
        
    return (losses.avg, top1.avg, sum_correct)

    return 

In [ ]:


def train_max_var(train_data,labels, model, criterion, optimizer, class_count, class_index, epoch, use_cuda,num_batchs=999999, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    # len_t =  (len(train_classifier_label_tensor)//batch_size)-1
    len_t = np.max(class_count)/10
    batch_index = np.copy(class_index)
    batch_data = torch.Tensor()
    batch_label = torch.LongTensor()
    for ind in range(len_t):
        if ind > num_batchs:
            break
    #     # measure data loading time
    #     inputs = train_classifier_data_tensor[ind*batch_size:(ind+1)*batch_size]
    #     targets = train_classifier_label_tensor[ind*batch_size:(ind+1)*batch_size]
        inputs = torch.Tensor()
        targets = torch.LongTensor()
        for i in range(class_num):
            if batch_index[i] + 10 > class_index[i+1]:
            #             batch_data = sort_train_classifier_data[batch_index[i]: class_index[i+1]]
                batch_index[i] = class_index[i]
                inputs = torch.cat((inputs, train_classifier_data_tensor[batch_index[i]: batch_index[i] + 10]), 0)
                targets = torch.cat((targets, train_classifier_label_tensor[batch_index[i]: batch_index[i] + 10]),0)
                batch_index[i] = batch_index[i] + 10

            else:
                inputs = torch.cat((inputs, train_classifier_data_tensor[batch_index[i]: batch_index[i] + 10]), 0)
                targets= torch.cat((targets, train_classifier_label_tensor[batch_index[i]: batch_index[i] + 10]),0)
                batch_index[i] = batch_index[i] + 10

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)

        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
        var = []
        for i in range(class_num):
            mean_outputs = torch.mean(soft_outputs[i*10:(i+1)*10],0)
            var.append(torch.mean(torch.sum((soft_outputs[i*10:(i+1)*10] - mean_outputs).pow(2),1)))
        batch_var = torch.stack(var,0)
        loss_var = alpha*(-torch.mean(batch_var))
        loss = criterion(outputs, targets) + loss_var
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(loss_var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%20==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s| Loss: {loss:.4f} | Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def train_max_var_min_a(train_data,labels, model, criterion, optimizer, class_count, class_index, epoch, use_cuda,num_batchs=999999, alpha = 1.0, beta = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    losses_a = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha,' beta: ', beta)
    end = time.time()
    # len_t =  (len(train_classifier_label_tensor)//batch_size)-1
    len_t = np.max(class_count)/10
    batch_index = np.copy(class_index)
    batch_data = torch.Tensor()
    batch_label = torch.LongTensor()
    for ind in range(len_t):
        if ind > num_batchs:
            break
    #     # measure data loading time
    #     inputs = train_classifier_data_tensor[ind*batch_size:(ind+1)*batch_size]
    #     targets = train_classifier_label_tensor[ind*batch_size:(ind+1)*batch_size]
        inputs = torch.Tensor()
        targets = torch.LongTensor()
        for i in range(class_num):
            if batch_index[i] + 10 > class_index[i+1]:
            #             batch_data = sort_train_classifier_data[batch_index[i]: class_index[i+1]]
                batch_index[i] = class_index[i]
                inputs = torch.cat((inputs, train_classifier_data_tensor[batch_index[i]: batch_index[i] + 10]), 0)
                targets = torch.cat((targets, train_classifier_label_tensor[batch_index[i]: batch_index[i] + 10]),0)
                batch_index[i] = batch_index[i] + 10

            else:
                inputs = torch.cat((inputs, train_classifier_data_tensor[batch_index[i]: batch_index[i] + 10]), 0)
                targets= torch.cat((targets, train_classifier_label_tensor[batch_index[i]: batch_index[i] + 10]),0)
                batch_index[i] = batch_index[i] + 10

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)

        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
#         max_soft_outputs = torch.max(soft_outputs,1).values
#         max_soft_untrain_outputs = torch.mean(torch.max(sort_soft_untrain_outputs,1).values)
#         min_soft_untrain_outputs = torch.min(torch.max(sort_soft_untrain_outputs,1).values)
#         print('max_soft_outputs: ', max_soft_outputs)
#         print('max_soft_untrain_outputs: ', max_soft_untrain_outputs)
#         print('min_soft_untrain_outputs: ', min_soft_untrain_outputs)
#         max_untrain_outputs = max_untrain_outputs + max_soft_untrain_outputs
        loss_a = beta*torch.mean(torch.abs(max_outputs - min_outputs))
    
        var = []
        for i in range(class_num):
            mean_outputs = torch.mean(soft_outputs[i*10:(i+1)*10],0)
            var.append(torch.mean(torch.sum((soft_outputs[i*10:(i+1)*10] - mean_outputs).pow(2),1)))
        batch_var = torch.stack(var,0)
        loss_var = alpha*(-torch.mean(batch_var))
#         print("batch_var: ", batch_var)
#         loss_var = alpha*(-torch.sum(batch_var))
        loss = criterion(outputs, targets) + loss_var + loss_a
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(loss_var.data, inputs.size()[0])
        losses_a.update(loss_a.data, inputs.size()[0])

        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%20==0:
            print  ('({batch}/{size}) Data:{data:.3f}s | Batch:{bt:.3f}s| Loss: {loss:.4f} | Var: {loss_var:.4f}|loss_a: {loss_a:.4f}| Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    loss_a = losses_a.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg,losses_a.avg, top1.avg)
        


In [ ]:


def train_max_var_mod2(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999,num_class=100, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)
    
#     mean_class = torch.from_numpy(np.zeros((num_class,num_class))).cuda().type(torch.cuda.FloatTensor)
    mean_class = np.zeros((num_class,num_class))
    var_n = np.zeros(num_class)
#     mean_var = torch.from_numpy(np.zeros((num_class))).cuda().type(torch.cuda.FloatTensor)
    mean_var = np.zeros((num_class))
 
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
#         mean_outputs = torch.mean(soft_outputs,0)
#         var = -torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))

        for i in range(batch_size):
            var_n[targets[i]] += 1
#             mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i]/var_n[targets[i]]
            mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i].data.cpu().numpy()/var_n[targets[i]]
#             mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - mean_class[targets[i]]).pow(2))/var_n[targets[i]]
            mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + np.sum((soft_outputs[i].data.cpu().numpy() - mean_class[targets[i]])**2)/var_n[targets[i]]

            
        var = -torch.mean(torch.from_numpy(mean_var)).cuda()*alpha
        var = torch.autograd.Variable(var)
        loss = criterion(outputs, targets) + var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()#retain_graph=True)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%20==0:
            print  ('({batch}/{size}) Data: {data:.3f}s |Batch: {bt:.3f}s| Loss: {loss:.4f} |Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def train_max_var_mod3(train_data,labels, model, criterion, optimizer, mean_class,var_n,mean_var, epoch, use_cuda,num_batchs=999999,num_class=100, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)
    
#     mean_class = torch.from_numpy(np.zeros((num_class,num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_class = np.zeros((num_class,num_class))
#     var_n = np.zeros(num_class)
# #     mean_var = torch.from_numpy(np.zeros((num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_var = np.zeros((num_class))
 
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
#         mean_outputs = torch.mean(soft_outputs,0)
#         var = -torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))

        for i in range(batch_size):
            var_n[targets[i]] += 1
#             mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i]/var_n[targets[i]]
            mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i].data.cpu().numpy()/var_n[targets[i]]
#             mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - mean_class[targets[i]]).pow(2))/var_n[targets[i]]
            mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + np.sum((soft_outputs[i].data.cpu().numpy() - mean_class[targets[i]])**2)/var_n[targets[i]]

            
        var = -torch.mean(torch.from_numpy(mean_var)).cuda()*alpha
        var = torch.autograd.Variable(var)
        loss = criterion(outputs, targets) + var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()#retain_graph=True)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%20==0:
            print  ('({batch}/{size}) Data: {data:.3f}s |Batch: {bt:.3f}s| Loss: {loss:.4f} |Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def train_max_var_mod4(train_data,labels, model, criterion, optimizer, mean_class,var_n, epoch, use_cuda,num_batchs=999999,num_class=100, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)
    
#     mean_class = torch.from_numpy(np.zeros((num_class,num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_class = np.zeros((num_class,num_class))
#     var_n = np.zeros(num_class)
# #     mean_var = torch.from_numpy(np.zeros((num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_var = np.zeros((num_class))
    mean_var = torch.from_numpy(np.zeros((num_class))).cuda().type(torch.cuda.FloatTensor)
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
#         mean_outputs = torch.mean(soft_outputs,0)
#         var = -torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))

        for i in range(batch_size):
            var_n[targets[i]] += 1
#             mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i]/var_n[targets[i]]
            mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i].data.cpu().numpy()/var_n[targets[i]]
            mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - torch.from_numpy(mean_class[targets[i]]).cuda()).pow(2))/var_n[targets[i]]
#             mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + np.sum((soft_outputs[i].data.cpu().numpy() - mean_class[targets[i]])**2)/var_n[targets[i]]

        var = torch.mean(mean_var)*alpha
#         var = -torch.mean(torch.from_numpy(mean_var)).cuda()*alpha
#         var = torch.autograd.Variable(var)
        loss = criterion(outputs, targets) + var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%10==0:
            print  ('({batch}/{size}) Data: {data:.3f}s |Batch: {bt:.3f}s| Loss: {loss:.4f} |Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def train_max_var_mod7(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values # max for each data
        min_outputs = torch.min(max_outputs)
        
        mean_outputs = torch.mean(soft_outputs,0)
        var = torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))*alpha
        
        loss = criterion(outputs, targets) + var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%10==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s| Loss: {loss:.4f} | Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def train_max_var_mod5(train_data,labels, model, criterion, optimizer, mean_class,var_n, epoch, use_cuda,num_batchs=999999,num_class=100, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)
    
#     mean_class = torch.from_numpy(np.zeros((num_class,num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_class = np.zeros((num_class,num_class))
#     var_n = np.zeros(num_class)
#     mean_var = torch.from_numpy(np.zeros((num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_var = np.zeros((num_class))
 
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
#         mean_outputs = torch.mean(soft_outputs,0)
#         var = -torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))

        for i in range(batch_size):
#         var_n[targets[i]] = var_n[targets[i]] + 1
            var_n[targets[i]] += 1
    #         mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i]/var_n[targets[i]]
            mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i].data.cpu().numpy()/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - mean_class[targets[i]]).pow(2))/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - torch.from_numpy(mean_class[targets[i]]).cuda()).pow(2))/var_n[targets[i]]

        temp_mean = torch.from_numpy(mean_class).cuda()
        mean_var = torch.sum((soft_outputs - temp_mean[targets]).pow(2),1)
    
        var = torch.mean(mean_var)*alpha
#         var = -torch.mean(torch.from_numpy(mean_var)).cuda()*alpha
#         var = torch.autograd.Variable(var)
        loss = criterion(outputs, targets) + var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward() #retain_graph=True)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%20==0:
            print  ('({batch}/{size}) Data: {data:.3f}s |Batch: {bt:.3f}s| Loss: {loss:.4f} |Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def train_half_diff(train_data,labels, model, criterion, optimizer,alpha, epoch, use_cuda,num_batchs=999999):
    # switch to train mode
    model.train()
    print('alpha = ', alpha)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    diffes = AverageMeter()
    
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs, y1,y2,y3,y4,y5 = model(inputs)
        
        out_list = [y1,y2,y3,y4,y5, outputs]
#         out_list[0].shape
        sum_diff = torch.zeros(out_list[0].shape[0]).cuda()
        for out_layer in out_list:

            # hidden_outputs.shape
            hidden_map = torch.ones(out_layer.shape[1]).cuda()
            hidden_map[out_layer.shape[1]//2:] = -1
        #     torch.sum(hidden_map)
            hd_diff_map = out_layer * hidden_map
            # hd_diff_map.shape
            hd_diff = torch.sum(hd_diff_map, 1)
            var_hd = hd_diff ** 2 / hd_diff_map.shape[1]
            sum_diff += var_hd
        
        diff_var = sum_diff.mean() * alpha
        loss = criterion(outputs, targets) + diff_var

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        diffes.update(diff_var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | | diff: {diff:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    diff=diffes.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg, diffes.avg)
        


In [ ]:


def train_half_diff_2(train_data,labels, model, criterion, optimizer,alpha, epoch, use_cuda,num_batchs=999999):
    # switch to train mode
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    diffes = AverageMeter()
    
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs, hidden_outputs = model(inputs)
        
        hidden_map = torch.ones(256).cuda()
        hidden_map[128:] = -1
        torch.sum(hidden_map)
        hd_diff_map = hidden_outputs * hidden_map
        # hd_diff_map.shape
        hd_diff = torch.mean(torch.sum(hd_diff_map, 1))** 2 / 256

        output_map = torch.ones(10).cuda()
        output_map[5:] = -1
        output_diff_map = outputs * output_map
        # hd_diff_map.shape
        output_diff = torch.mean(torch.sum(output_diff_map , 1)) ** 2 / 10
#         output_diff 

        diff_var = alpha * (hd_diff + output_diff) 
        
        loss = criterion(outputs, targets) + diff_var

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        diffes.update(diff_var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | | diff: {diff:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    diff=diffes.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg, diffes.avg)
        


In [ ]:


def train_half_diff_min_var(train_data,labels, model, criterion, optimizer,alpha, epoch, use_cuda, var_n,mean_class,beta):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()
    
    losses_var = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    diffes = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        # measure data loading time
        
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs, y1,y3,y4,y5 = model(inputs)
        
        out_list = [y1,y3,y4,y5, outputs]
#         out_list[0].shape
        sum_diff = torch.zeros(out_list[0].shape[0]).cuda()
        for out_layer in out_list:

            # hidden_outputs.shape
            hidden_map = torch.ones(out_layer.shape[1]).cuda()
            hidden_map[out_layer.shape[1]//2:] = -1
        #     torch.sum(hidden_map)
            hd_diff_map = out_layer * hidden_map
            # hd_diff_map.shape
            hd_diff = torch.sum(hd_diff_map, 1)
            var_hd = hd_diff ** 2 / hd_diff_map.shape[1]
            sum_diff += var_hd
        
        diff_var = sum_diff.mean() * alpha
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
        
#         mean_outputs = torch.mean(soft_outputs,0)
#         var = -torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))

        for i in range(batch_size):
#         var_n[targets[i]] = var_n[targets[i]] + 1
            var_n[targets[i]] += 1
    #         mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i]/var_n[targets[i]]
            mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i].data.cpu().numpy()/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - mean_class[targets[i]]).pow(2))/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - torch.from_numpy(mean_class[targets[i]]).cuda()).pow(2))/var_n[targets[i]]

        temp_mean = torch.from_numpy(mean_class).cuda()
        mean_var = torch.sum((soft_outputs - temp_mean[targets]).pow(2),1)
    
        var = torch.mean(mean_var)*beta
        
        loss = criterion(outputs, targets) + diff_var + var

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        diffes.update(diff_var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])     
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%10==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |var: {loss_var:.4f}| diff: {diff:.4f}|Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    diff=diffes.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg, diffes.avg, losses_var.avg)
        


In [ ]:


def _log_value(probs, small_value=1e-30):
    return -torch.log(torch.max(probs, torch.tensor(small_value).cuda()))

def _m_entr_comp(probs, true_labels):
    log_probs = _log_value(probs)
    reverse_probs = 1-probs
    log_reverse_probs = _log_value(reverse_probs)
#     modified_probs = np.copy(probs)
    modified_probs = probs.detach().clone()
    modified_probs[range(true_labels.size(0)), true_labels] = reverse_probs[range(true_labels.size(0)), true_labels]
    modified_log_probs = log_reverse_probs.detach().clone()
    modified_log_probs[range(true_labels.size(0)), true_labels] = log_probs[range(true_labels.size(0)), true_labels]
    return torch.sum(torch.mul(modified_probs, modified_log_probs),axis=1)

In [ ]:


def train_half_diff_min_var_me(train_data,labels, model, criterion, optimizer,alpha, epoch, use_cuda, var_n,mean_class,beta, gamma):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()
    
    losses_var = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    diffes = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    me_vars = AverageMeter()
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs, y1,y3,y4,y5 = model(inputs)
        
        out_list = [y1,y3,y4,y5, outputs]
#         out_list[0].shape
        sum_diff = torch.zeros(out_list[0].shape[0]).cuda()
        for out_layer in out_list:

            # hidden_outputs.shape
            hidden_map = torch.ones(out_layer.shape[1]).cuda()
            hidden_map[out_layer.shape[1]//2:] = -1
        #     torch.sum(hidden_map)
            hd_diff_map = out_layer * hidden_map
            # hd_diff_map.shape
            hd_diff = torch.sum(hd_diff_map, 1)
            var_hd = hd_diff ** 2 / hd_diff_map.shape[1]
            sum_diff += var_hd
        
        diff_var = sum_diff.mean() * alpha
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
#         s_tr_m_entr = _m_entr_comp(soft_outputs.data.cpu().numpy(), targets.data.cpu().numpy())
        s_tr_m_entr = _m_entr_comp(soft_outputs, targets)
        me_var = -torch.var(s_tr_m_entr) * gamma
#         me_var = torch.tensor(me_var).cuda()

#         mean_outputs = torch.mean(soft_outputs,0)
#         var = -torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))

        for i in range(batch_size):
#         var_n[targets[i]] = var_n[targets[i]] + 1
            var_n[targets[i]] += 1
    #         mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i]/var_n[targets[i]]
            mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i].data.cpu().numpy()/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - mean_class[targets[i]]).pow(2))/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - torch.from_numpy(mean_class[targets[i]]).cuda()).pow(2))/var_n[targets[i]]

        temp_mean = torch.from_numpy(mean_class).cuda()
        mean_var = torch.sum((soft_outputs - temp_mean[targets]).pow(2),1)
    
        var = torch.mean(mean_var)*beta
        
        loss = criterion(outputs, targets) + diff_var + var +  me_var

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        diffes.update(diff_var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        me_vars.update(me_var.data, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])     
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) | Batch: {bt:.3f}s | | Loss: {loss:.4f} |var: {loss_var:.4f}| diff: {diff:.4f}|me_var: {me_vars:.4f}|Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
#                     data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    diff=diffes.avg,
                    me_vars = me_vars.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg, diffes.avg, losses_var.avg)
        


In [ ]:


print('train_classifier_data: ',len(train_classifier_data))

print('train_attack_data: ',len(train_attack_data))

print('test_data: ',len(test_data))

In [ ]:


small_train_classifier_data_length = 10000
small_train_classifier_data = X[:small_train_classifier_data_length]
small_train_classifier_label = Y[:small_train_classifier_data_length]

len(small_train_classifier_data)

In [ ]:



a = np.zeros(100)
for i in range(len(Y)):
    a[Y[i]] += 1

sort_a = np.sort(a)
class_index = np.argsort(a)
select_class = class_index[90:]
select_class

In [ ]:


class10_index = []
for i in range(len(Y)):
    if Y[i] in select_class:
        class10_index.append(i)

class10_index = np.asarray(class10_index)
class10_index.shape

In [ ]:


ori_class10_label = Y[class10_index]
print(ori_class10_label.shape)
class10_data = X[class10_index]
sort_select_class = np.sort(select_class).tolist()
sort_select_class

In [ ]:


class10_label = np.zeros(len(ori_class10_label), dtype=int)
for i in range(len(class10_label)):
    class10_label[i] = sort_select_class.index(ori_class10_label[i])

In [ ]:


class10_train_classifier_data = class10_data[:7000]
class10_train_attack_data = class10_data[7000:14000]
class10_test_data = class10_data[14000:]

class10_train_classifier_label = class10_label[:7000]
class10_train_attack_label = class10_label[7000:14000]
class10_test_label = class10_label[14000:]

print('class10_train_classifier_data: ',len(class10_train_classifier_data))

print('class10_train_attack_data: ',len(class10_train_attack_data))

print('class10_test_data: ',len(class10_test_data))

In [ ]:


# npdata = np.load('texas10_sort_data.npz')
# sort_class10_data = npdata['x'][:,:]
# sort_class10_label = npdata['y'][:]

In [ ]:


class10_500_trainset_index = []
class10_500_attack_index = []
class10_500_testset_index = []
class10_500_count = np.zeros(10)
for i in range(len(class10_label)):
    if(class10_500_count[class10_label[i]]<500):
        class10_500_trainset_index.append(i)
        class10_500_count[class10_label[i]] += 1
    elif(500<= class10_500_count[class10_label[i]]<1000):
        class10_500_attack_index.append(i)
        class10_500_count[class10_label[i]] += 1
    elif(1000<= class10_500_count[class10_label[i]]<1500):
        class10_500_testset_index.append(i)
        class10_500_count[class10_label[i]] += 1

In [ ]:


class10_500_trainset_index = np.asarray(class10_500_trainset_index)
class10_500_attack_index = np.asarray(class10_500_attack_index)
class10_500_testset_index = np.asarray(class10_500_testset_index)

print(class10_500_trainset_index.shape)
print(class10_500_attack_index.shape)
print(class10_500_testset_index.shape)

In [ ]:


class10_500_trainset_data = class10_data[class10_500_trainset_index]
class10_500_attack_data = class10_data[class10_500_attack_index]
class10_500_testset_data = class10_data[class10_500_testset_index]

class10_500_trainset_label = class10_label[class10_500_trainset_index]
class10_500_attack_label = class10_label[class10_500_attack_index]
class10_500_testset_label = class10_label[class10_500_testset_index]

In [ ]:


class10_500_trainset_data.shape

In [ ]:


# start train
lr=0.001
epochs=20
# p = 0.7
state={}
state['lr']=lr
# net = Texas()
# net = Texas_drop(p=p)
# net = Texas_2layer()
# net = Texas_1layer(num_classes=10)
net = Texas_layer_out(num_classes=100)
net = torch.nn.DataParallel(net).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
batch_size=128

In [ ]:


def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in [800, 1500]:
#     if epoch in [100, 90]:
        state['lr'] *= 10 
#         state['lr'] *= 1
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']
    if epoch in [810,400,80,150, 1500]:
#     if epoch in [100, 90]:
        state['lr'] *= 0.1 
#         state['lr'] *= 1
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [ ]:


net

In [ ]:



batch_size=256
best_acc = 0.0
best_epoch = 0
epochs = 100
num_class = 100
mean_class = np.zeros((num_class,num_class))
var_n = np.zeros(num_class)
alpha = 20
beta = 0
train_time_list = []
print('alpha = ', alpha, ' beta = ', beta)

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)
#     if epoch == 30:
#         alpha = 3000
#         print('alpha = ', alpha, ' beta = ', beta)
    r= np.arange(len(train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = train_classifier_data[r]
    s_train_classifier_label = train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_time = time.time()
    train_loss, train_acc, diff, var = train_half_diff_min_var(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, alpha, epoch, use_cuda, var_n,mean_class,beta)
    train_time_list.append(time.time()-train_time)
    print ('train acc: %.4f, train loss: %.4f, train diff: %.4f, last var: %.4f'%(train_acc, train_loss, diff, var))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
        # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 50:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, is_best, filename='Texas_5layer_balance_output_partition_alpha20_min_var_beta0_epoch_%d'%(epoch+1))

In [ ]:



batch_size=256
best_acc = 0.0
best_epoch = 0
epochs = 200
num_class = 100
mean_class = np.zeros((num_class,num_class))
var_n = np.zeros(num_class)
alpha = 500
beta = 3000
train_time_list = []
print('alpha = ', alpha, ' beta = ', beta)

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)
#     if epoch == 30:
#         alpha = 3000
#         print('alpha = ', alpha, ' beta = ', beta)
    r= np.arange(len(train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = train_classifier_data[r]
    s_train_classifier_label = train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_time = time.time()
    train_loss, train_acc, diff, var = train_half_diff_min_var(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, alpha, epoch, use_cuda, var_n,mean_class,beta)
    train_time_list.append(time.time()-train_time)
    print ('train acc: %.4f, train loss: %.4f, train diff: %.4f, last var: %.4f'%(train_acc, train_loss, diff, var))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
        # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 50:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, is_best, filename='Texas_5layer_balance_output_partition_alpha500_min_var_beta3000_epoch_%d'%(epoch+1))

In [ ]:




batch_size=128
best_acc = 0.0
best_epoch = 0
epochs = 200
num_class = 100
mean_class = np.zeros((num_class,num_class))
var_n = np.zeros(num_class)
alpha = 1000
beta = 200
print('alpha = ', alpha, ' beta = ', beta)

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)
    if epoch == 30:
        alpha = 3000
        print('alpha = ', alpha, ' beta = ', beta)
    r= np.arange(len(train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = train_classifier_data[r]
    s_train_classifier_label = train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_acc, diff, var = train_half_diff_min_var(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, alpha, epoch, use_cuda, var_n,mean_class,beta)
    print ('train acc: %.4f, train loss: %.4f, train diff: %.4f, last var: %.4f'%(train_acc, train_loss, diff, var))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
    # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 40:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Texas_5layer_balance_output_partition_alpha3000_min_var_beta200_epoch_%d'%(epoch+1))

In [ ]:


# lr = 0.001
batch_size=256
best_acc = 0.0
best_epoch = 0
epochs = 100
for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)
    r= np.arange(len(train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = train_classifier_data[r]
    s_train_classifier_label = train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_acc = train(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, epoch, use_cuda)
    print ('train acc: %.4f, train loss: %.4f'%(train_acc, train_loss))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
    # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 40:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Texas100_5layer_No_defenss_epoch_%d'%(epoch+1))



In [ ]:


# lr = 0.0001
batch_size=128
best_acc = 0.0
best_epoch = 0
epochs = 100
for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)
    r= np.arange(len(small_train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = small_train_classifier_data[r]
    s_train_classifier_label = small_train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_acc = train(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, epoch, use_cuda)
    print ('train acc: %.4f, train loss: %.4f'%(train_acc, train_loss))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
    # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 58:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_1_layer_No_defenss_100_epoch_%d'%(epoch+1))

In [ ]:


# lr = 0.0001
batch_size=128
best_acc = 0.0
best_epoch = 0
epochs = 40
for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)
    r= np.arange(len(small_train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = small_train_classifier_data[r]
    s_train_classifier_label = small_train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_acc = train(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, epoch, use_cuda)
    print ('train acc: %.4f, train loss: %.4f'%(train_acc, train_loss))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
    # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 40:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_1_layer_No_defenss_epoch_%d'%(epoch+1))

In [ ]:


# lr = 0.0001
batch_size=128
# best_acc = 0.0
best_epoch = 0
epochs = 80
for epoch in range(40, epochs):
    adjust_learning_rate(optimizer, epoch)
    r= np.arange(len(small_train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = small_train_classifier_data[r]
    s_train_classifier_label = small_train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_acc = train(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, epoch, use_cuda)
    print ('train acc: %.4f, train loss: %.4f'%(train_acc, train_loss))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
    # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 40:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_1_layer_No_defenss_epoch_%d'%(epoch+1))

In [ ]:


# lr = 0.00001
batch_size=128
# best_acc = 0.0
best_epoch = 0
epochs = 120
for epoch in range(80, epochs):
    adjust_learning_rate(optimizer, epoch)
    r= np.arange(len(small_train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = small_train_classifier_data[r]
    s_train_classifier_label = small_train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_acc = train(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, epoch, use_cuda)
    print ('train acc: %.4f, train loss: %.4f'%(train_acc, train_loss))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
    # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 40:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_1_layer_No_defenss_epoch_%d'%(epoch+1))

In [ ]:


# model debug
model = net
num_batchs = 99999
alpha = 100.0

batch_size = 1000

# r= np.arange(len(train_classifier_data))
# np.random.shuffle(r)
# train_classifier_data = train_classifier_data[r]
# train_classifier_label = train_classifier_label[r]

class_num = 100
# set_index = np.argsort(train_classifier_label)
# sort_train_classifier_data = train_classifier_data[set_index]
# sort_train_classifier_label = train_classifier_label[set_index]

# class_count = np.zeros(class_num, dtype=int)
# for i in range(len(sort_train_classifier_label)):
#     class_count[sort_train_classifier_label[i]] += 1
    
# class_sum = 0
# class_index = np.zeros(class_num+1, dtype=int)
# for i in range(class_num):
#     class_index[i] = class_sum
#     class_sum += class_count[i]
# class_index[class_num] = class_sum


# train_classifier_data_tensor = torch.from_numpy(sort_train_classifier_data).type(torch.FloatTensor)
# train_classifier_label_tensor = torch.from_numpy(sort_train_classifier_label).type(torch.LongTensor)


test_data_tensor = torch.from_numpy(train_classifier_data).type(torch.FloatTensor)
test_label_tensor = torch.from_numpy(train_classifier_label).type(torch.LongTensor)
print(test_label_tensor)
model.train()
softmax = nn.Softmax()

batch_time = AverageMeter()
data_time = AverageMeter()
losses = AverageMeter()
top1 = AverageMeter()
top5 = AverageMeter()


end = time.time()
len_t = 10
# len_t =  (len(train_classifier_label_tensor)//batch_size)-1
# len_t = np.max(class_count)/10
batch_index = np.copy(class_index)
batch_data = torch.Tensor()
batch_label = torch.LongTensor()
for ind in range(len_t):
    if ind > num_batchs:
        break
#     # measure data loading time
#     inputs = train_classifier_data_tensor[ind*batch_size:(ind+1)*batch_size]
#     targets = train_classifier_label_tensor[ind*batch_size:(ind+1)*batch_size]
    inputs = test_data_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = test_label_tensor[ind*batch_size:(ind+1)*batch_size]
#     inputs = torch.Tensor()
#     targets = torch.LongTensor()
#     print('\nIndex: ',ind)
#     for i in range(class_num):
#         if (batch_index[i] + 10) > class_index[i+1]:
#         #             batch_data = sort_train_classifier_data[batch_index[i]: class_index[i+1]]
#             print('batch_index[i]: ', batch_index[i], 'i: ',i)
#             batch_index[i] = class_index[i]
#             print('reassign batch_index[i]: ', batch_index[i], 'i: ',i)
#             print('class_index: ',class_index)
#             inputs = torch.cat((inputs, train_classifier_data_tensor[batch_index[i]: batch_index[i] + 10]), 0)
#             targets = torch.cat((targets, train_classifier_label_tensor[batch_index[i]: batch_index[i] + 10]),0)
#             batch_index[i] = batch_index[i] + 10

#         else:
#             inputs = torch.cat((inputs, train_classifier_data_tensor[batch_index[i]: batch_index[i] + 10]), 0)
#             targets= torch.cat((targets, train_classifier_label_tensor[batch_index[i]: batch_index[i] + 10]),0)
#             batch_index[i] = batch_index[i] + 10
    
    data_time.update(time.time() - end)
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

    # compute output
    outputs,y1,y2,y3,y4 = model(inputs)

    soft_outputs = softmax(outputs)
    max_outputs = torch.max(soft_outputs,1).values
#     var = []
    
#     print('soft_outputs.shape: ', soft_outputs.shape)
#     print(soft_outputs[99*10:(99+1)*10])
#     for i in range(class_num):
#         mean_outputs = torch.mean(soft_outputs[i*10:(i+1)*10],0)
#         var.append(torch.mean(torch.sum((soft_outputs[i*10:(i+1)*10] - mean_outputs).pow(2),1)))
#     batch_var = torch.stack(var,0)
#     print("batch_var:", batch_var)
#     loss_var = alpha*(-torch.mean(batch_var))
#     loss = criterion(outputs, targets) #+ loss_var
# #     print("loss_var:" ,loss_var)

    # measure accuracy and record loss
    prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
#     losses.update(loss.data, inputs.size()[0])
#     top1.update(prec1, inputs.size()[0])
#     top5.update(prec5, inputs.size()[0])

    print(prec1)
    # compute gradient and do SGD step
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

    # measure elapsed time
    batch_time.update(time.time() - end)
    end = time.time()
    if ind == 10:
        break
#     break

In [ ]:


torch.mean(torch.var(soft_outputs,0))

y2.shape

In [ ]:


probs = soft_outputs
true_labels = targets
print(probs)
log_probs = _log_value(probs)
reverse_probs = 1-probs
log_reverse_probs = _log_value(reverse_probs)
modified_probs = probs.detach().clone()
modified_probs[range(true_labels.size(0)), true_labels] = reverse_probs[range(true_labels.size(0)), true_labels]
modified_log_probs = log_reverse_probs.detach().clone()
modified_log_probs[range(true_labels.size(0)), true_labels] = log_probs[range(true_labels.size(0)), true_labels]
me = torch.sum(torch.mul(modified_probs, modified_log_probs),axis=1)

In [ ]:


torch.var(me)

In [ ]:


fig = plt.figure(figsize=(20,15))

plt.hist(me.data.cpu().numpy(), bins=10, label = 'me')

In [ ]:


s_tr_m_entr = _m_entr_comp(soft_outputs.data.cpu().numpy(), targets.data.cpu().numpy())
loss = criterion(outputs, targets) 

In [ ]:


mean1 = np.mean(s_tr_m_entr)
mean1
np.var(s_tr_m_entr)

inputs.size()[0]

In [ ]:


np.mean(np.power(np.abs(s_tr_m_entr - mean1),2))

In [ ]:


out_list = [y1,y2,y3,y4,y5, outputs]
out_list[0].shape

In [ ]:


out_list = [y1,y2,y3,y4,y5, outputs]
out_list[0].shape
sum_diff = torch.zeros(out_list[0].shape[0]).cuda()
for out_layer in out_list:
    
    # hidden_outputs.shape
    hidden_map = torch.ones(out_layer.shape[1]).cuda()
    hidden_map[out_layer.shape[1]//2:] = -1
#     torch.sum(hidden_map)
    hd_diff_map = out_layer * hidden_map
    # hd_diff_map.shape
    hd_diff = torch.sum(hd_diff_map, 1)
    var_hd = hd_diff ** 2 / hd_diff_map.shape[1]
    print(hd_diff_map.shape[1])
    sum_diff += var_hd
# output_map = torch.ones(10).cuda()
# output_map[5:] = -1
# output_diff_map = outputs * output_map
# # hd_diff_map.shape
# output_diff = torch.mean(torch.sum(output_diff_map , 1))
# output_diff 

# diff_var = (hd_diff ** 2 + output_diff ** 2)/2
# diff_var

In [ ]:


var_hd = hd_diff ** 2 / y5.shape[1]
var_hd[0]
sum_diff.mean() * alpha

In [ ]:


outputs.shape
output_map = torch.ones(10).cuda()
output_map[5:] = -1
output_diff_map = outputs * output_map
# hd_diff_map.shape
output_diff = torch.mean(torch.sum(output_diff_map , 1))
output_diff 

In [ ]:


r= np.arange(len(train_classifier_data))
np.random.shuffle(r)
train_classifier_data = train_classifier_data[r]
train_classifier_label = train_classifier_label[r]
set_index = np.argsort(train_classifier_label)
set_index

In [ ]:


print(train_classifier_label[set_index[105]])
print(np.where(train_classifier_data[set_index[105]]==1))

In [ ]:


fortrain_classifier_data[set_index[0]]

In [ ]:


sf_output = soft_outputs[38].data.cpu().numpy()
pyplot.plot(sf_output)

In [ ]:


r1= np.arange(len(train_classifier_data)*0.75)
np.random.shuffle(np.intc(r1))
r1

In [ ]:


r2= np.arange(len(train_classifier_data)*0.25)
# np.random.shuffle(r2)
r2 = np.intc(r2) + np.intc(len(train_classifier_data)*0.75)
r2

In [ ]:


max(r2)

In [ ]:


model = net
num_batchs = 99999
alpha = 1.0

r1= np.arange(len(train_classifier_data)*0.75)
np.random.shuffle(r1)
r1 = np.intc(r1)
train_data = train_classifier_data[r1]
train_label = train_classifier_label[r1]

train_classifier_data_tensor = torch.from_numpy(train_classifier_data).type(torch.FloatTensor)
train_classifier_label_tensor = torch.from_numpy(train_classifier_label).type(torch.LongTensor)

r2= np.arange(len(train_classifier_data)*0.25)
r2 = np.intc(r2) + np.intc(len(train_classifier_data)*0.75)
untrain_classifier_data = train_classifier_data[r2]
untrain_classifier_label = train_classifier_label[r2]

untrain_classifier_data_tensor = torch.from_numpy(untrain_classifier_data).type(torch.FloatTensor)
untrain_classifier_label_tensor = torch.from_numpy(untrain_classifier_label).type(torch.LongTensor)


train_data = train_classifier_data_tensor
labels = train_classifier_label_tensor
untrain_data = untrain_classifier_data_tensor
untrain_labels = untrain_classifier_label_tensor


model.train()
softmax = nn.Softmax()

batch_time = AverageMeter()
data_time = AverageMeter()
losses = AverageMeter()
top1 = AverageMeter()
top5 = AverageMeter()

end = time.time()
len_t =  (len(train_classifier_label_tensor)//batch_size)-1

for ind in range(len_t):
    if ind > num_batchs:
        break
    # measure data loading time
    inputs = train_data[ind*batch_size:(ind+1)*batch_size]
    targets = labels[ind*batch_size:(ind+1)*batch_size]
    r= np.arange(untrain_data.shape[0])
    np.random.shuffle(r)
    untrain_inputs = untrain_data[r[:batch_size]]
    untrain_targets = untrain_labels[r[:batch_size]]

    data_time.update(time.time() - end)
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

    # compute output
    outputs,_ = model(inputs)
    untrain_outputs,_ = model(untrain_inputs)

    soft_outputs = softmax(outputs)
    soft_untrain_outputs = softmax(untrain_outputs)

#         max_outputs = torch.max(soft_outputs,1).values
    sort_soft_outputs,_ = torch.sort(soft_outputs)
    sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)

    loss = criterion(outputs, targets) + alpha*(torch.mean(torch.sum(torch.abs(sort_soft_outputs - sort_soft_untrain_outputs),1)))
    break

In [ ]:


a = torch.abs(sort_soft_outputs - sort_soft_untrain_outputs)

In [ ]:


r2= np.arange(len(train_classifier_data)*0.25)

r2.shape

In [ ]:


# train_classifier_data = X[:int(train_classifier_ratio*len_train)]
# train_attack_data = X[int(train_classifier_ratio*len_train):int((train_classifier_ratio+train_attack_ratio)*len_train)]
# test_data = X[int((train_classifier_ratio+train_attack_ratio)*len_train):]

# train_classifier_label = Y[:int(train_classifier_ratio*len_train)]
# train_attack_label = Y[int(train_classifier_ratio*len_train):int((train_classifier_ratio+train_attack_ratio)*len_train)]
# test_label = Y[int((train_classifier_ratio+train_attack_ratio)*len_train):]

In [ ]:


len(train_attack_data)

r = np.arange(len(train_attack_data))
r1 = np.arange(len(train_classifier_data))t
np.random.shuffle(r)
np.random.shuffle(r1)
r2 = r1[r]

print(len(r))
print(len(r1))
print(len(r2))

print(max(r2))

In [ ]:
# load saved model

# test_data_c = X[int((train_classifier_ratio+train_attack_ratio)*len_train):]
# test_label_c = Y[int((train_classifier_ratio+train_attack_ratio)*len_train):]
# test_data_c = X[40000:60000]
# test_label_c = Y[40000:60000]
# test_data_c = class10_data[14000:]
# test_label_c = class10_label[14000:]
test_data_c = test_data
test_label_c = test_label
epoch=0
batch_size=128
# net = Texas()
# net = Texas_drop(p)
# net = Texas_2layer()
# net = Texas_1layer(10)
net = Texas_layer_out(100)
# net = Texas_layer_out_scale(100, q = 50, alpha = 20)
# net = Texas_layer_out_scale2(100, q = 50, alpha = 200)


net = torch.nn.DataParallel(net).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)
print(len(test_data_c))

In [ ]:



# resume='./checkpoints_texas_10class_little_model_defense/Class10_1layer_Modify_min_sortvar_beta100_max_last8_alpha30_defense_epoch_9'
resume='./checkpoints_texas/Texas_5layer_balance_output_partition_alpha100_min_var_beta3000_epoch_50'
# resume='./checkpoints_texas_10class_little_model_defense/Texas_5layer_balance_output_partition_alpha300_min_var_beta3000_epoch_84'
# resume='./checkpoints_texas_10class_little_model_defense/Texas_5layer_balance_output_partition_alpha500_min_var_beta3000_epoch_90'
# resume='./checkpoints_texas_10class_little_model_defense/Texas_5layer_balance_output_partition_alpha20_min_var_beta0_epoch_13'
# resume='./checkpoints_texas_10class_little_model_defense/Texas_5layer_balance_output_partition_alpha100_min_var_beta0_epoch_59'
# resume='./checkpoints_texas_10class_little_model_defense/Texas_5layer_balance_output_partition_alpha300_min_var_beta0_epoch_100'

checkpoint = os.path.dirname(resume)
checkpoint = torch.load(resume)
net.load_state_dict(checkpoint['state_dict'])
print('==> Resumed from checkpoint..')
epoch = 0

In [ ]:



test_loss, final_test_acc = test(torch.from_numpy(test_data_c).type(torch.FloatTensor) ,torch.from_numpy(test_label_c).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

# trainset_loss, trainset_acc = test(torch.from_numpy(small_train_classifier_data).type(torch.FloatTensor) ,torch.from_numpy(small_train_classifier_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

trainset_loss, trainset_acc = test(torch.from_numpy(train_classifier_data).type(torch.FloatTensor) ,torch.from_numpy(train_classifier_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('trainset_loss: {trainset_loss: .4f}, trainset_acc: {trainset_acc: .4f}'.format(trainset_loss=trainset_loss, trainset_acc=trainset_acc))

In [ ]:


print(train_classifier_data.shape)
print(test_data_c.shape)

In [ ]:


x_evaluate_attacker_tensor = torch.from_numpy(train_classifier_data).type(torch.FloatTensor)
y_evaluate_attacker_tensor = torch.from_numpy(train_classifier_label).type(torch.LongTensor)

batch_size = 256

net.eval()

f_evaluate_logits = []
f_evaluate_prob = []
hidden_layer_logits = []
softmax = nn.Softmax()
len_t =  (len(x_evaluate_attacker_tensor)//batch_size)+1
print(len_t)
for ind in range(len_t):
    # measure data loading time
    inputs = x_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = y_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
    
    # compute output
    outputs,_,_,_,hidden_layer_outputs = net(inputs)
    outputs_value = outputs.data.cpu().numpy()
    hidden_layer_outputs_value = hidden_layer_outputs.data.cpu().numpy()
#     loss = criterion(outputs, targets)
    prob_outputs = softmax(outputs)
    prob_outputs_value = prob_outputs.data.cpu().numpy()
    for i in range(len(prob_outputs_value)):
        f_evaluate_logits.append(outputs_value[i])
        f_evaluate_prob.append(prob_outputs_value[i])
        hidden_layer_logits.append(hidden_layer_outputs_value[i])

In [ ]:


f_evaluate_logits_array = np.asarray(f_evaluate_logits)
f_evaluate_prob_array = np.asarray(f_evaluate_prob)
hidden_layer_logits_array = np.asarray(hidden_layer_logits)
print(f_evaluate_logits_array.shape, f_evaluate_prob_array.shape)

sort_f_evaluate=np.sort(f_evaluate_prob_array,axis=1)
sort_f_evaluate_logits=np.sort(f_evaluate_logits_array,axis=1)
sort_hidden_layer_logits = np.sort(hidden_layer_logits_array, axis=1)

In [ ]:


x_evaluate_attacker_tensor = torch.from_numpy(test_data_c).type(torch.FloatTensor)
y_evaluate_attacker_tensor = torch.from_numpy(test_label_c).type(torch.LongTensor)

batch_size = 128

net.eval()

test_prob = []
test_logits = []
test_hidden_layer_logits = []
softmax = nn.Softmax()
len_t =  (len(x_evaluate_attacker_tensor)//batch_size)+1
print(len_t)
for ind in range(len_t):
    # measure data loading time
    inputs = x_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = y_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
    
    # compute output
    outputs,_,_,_,hidden_layer_outputs = net(inputs)
    outputs_value = outputs.data.cpu().numpy()
    hidden_layer_outputs_value = hidden_layer_outputs.data.cpu().numpy()
#     loss = criterion(outputs, targets)
    prob_outputs = softmax(outputs)
    prob_outputs_value = prob_outputs.data.cpu().numpy()
    for i in range(len(prob_outputs_value)):
        test_logits.append(outputs_value[i])
        test_prob.append(prob_outputs_value[i])
        test_hidden_layer_logits.append(hidden_layer_outputs_value[i])

In [ ]:


test_prob_array = np.asarray(test_prob)
test_logits_array = np.asarray(test_logits)
test_hidden_layer_logits_array = np.asarray(test_hidden_layer_logits)
print(test_prob_array.shape, test_hidden_layer_logits_array.shape)

sort_test_prob=np.sort(test_prob_array,axis=1)
sort_test_logits=np.sort(test_logits_array,axis=1)
sort_test_hidden_layer_logits = np.sort(test_hidden_layer_logits_array, axis=1)

In [ ]:


len(f_evaluate_prob)

In [ ]:


test_sum = np.sum(test_logits_array,1)#.shape
train_sum = np.sum(f_evaluate_logits_array,1) #.shape
print(np.mean(test_sum), np.mean(train_sum))

print(np.ptp(test_sum), np.ptp(train_sum))
train_sum.shape

In [ ]:


test_mean = np.mean(test_logits_array,0)#.shape
train_mean = np.mean(f_evaluate_logits_array,0)#.shape
train_mean.shape

# fig = plt.figure(figsize=(30,20))
x = np.arange(100)
plt.plot(x,train_mean, label='train_mean')
plt.plot(x,test_mean, label='test_mean')
plt.legend(loc='best')
plt.show()

In [ ]:


test_mean = np.mean(test_prob_array,0)#.shape
train_mean = np.mean(f_evaluate_prob_array,0)#.shape
train_mean.shape

# fig = plt.figure(figsize=(30,20))
x = np.arange(100)
plt.plot(x,train_mean, label='train_mean')
plt.plot(x,test_mean, label='test_mean')
plt.legend(loc='best')
plt.show()

In [ ]:


test_sum = np.sum(test_hidden_layer_logits_array,1)#.shape
train_sum = np.sum(hidden_layer_logits_array,1) #.shape
print(np.mean(test_sum), np.mean(train_sum))
print(np.ptp(test_sum), np.ptp(train_sum))

In [ ]:


test_mean = np.mean(test_hidden_layer_logits_array,0)#.shape
train_mean = np.mean(hidden_layer_logits_array,0)#.shape
train_mean 

fig = plt.figure(figsize=(30,20))
x = np.arange(128)
plt.plot(x,train_mean)
plt.plot(x,test_mean)
plt.show()

In [ ]:




fig = plt.figure(figsize=(30,20))
x = np.arange(len(train_sum))
plt.plot(x,train_sum)
plt.plot(x,test_sum[:len(train_sum)])
plt.show()

In [ ]:


test_hidden_layer_logits_array.shape

In [ ]:


test_sum
np.sum(test_hidden_layer_logits_array[2])

x = np.arange(128)
plt.plot(x,test_hidden_layer_logits_array[2])
plt.plot(x,np.sort(test_hidden_layer_logits_array[2]))

np.sum(test_hidden_layer_logits_array[2])

In [ ]:


from matplotlib import pyplot
from matplotlib.pylab import plt
# pyplot.plot(predict_origin)
# pyplot.plot(f_evaluate_origin[3])
# pyplot.plot(f_evaluate_origin[4])
# plt.plot(f_evaluate_origin[0])
# plt.plot(f_evaluate_origin[1])
# plt.plot(f_evaluate_origin[2])
for i in range(100):
#     plt.plot(f_evaluate_origin[i])
    plt.plot(f_evaluate_logits[i])
#     plt.plot(result_array[i])
#     plt.plot(sort_result_array[i])
# for i in range(f_evaluate_origin.shape[0]):
#     pyplot.plot(f_evaluate_origin[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


for i in range(len(f_evaluate_prob)):
    plt.plot(hidden_layer_logits[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


for i in range(len(f_evaluate_prob)):
    plt.plot(sort_hidden_layer_logits[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


for i in range(len(sort_test_hidden_layer_logits)):
    plt.plot(sort_test_hidden_layer_logits[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


fig = plt.figure(figsize=(30,20))
for i in range(len(f_evaluate_prob)):
    plt.plot(sort_f_evaluate[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


fig = plt.figure(figsize=(30,20))
for i in range(10000):
    plt.plot(sort_test_prob[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


for i in range(len(f_evaluate_prob)):
    plt.plot(sort_f_evaluate_logits[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


for i in range(len(sort_test_hidden_layer_logits)):
    plt.plot(sort_test_logits[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


fig = plt.figure(figsize=(30,20))
for i in range(len(f_evaluate_prob_array)):
    plt.plot(f_evaluate_prob_array[i])
    
# plt.ylim(0, 0.7)
plt.show()

i

In [ ]:


fig = plt.figure(figsize=(30,20))
for i in range(10000):
    plt.plot(test_prob_array[i])
    
# plt.ylim(0, 0.7)
plt.show()
i

In [ ]:
# load membership inference attack

# r= np.arange(len(small_train_classifier_data))
# # np.random.shuffle(r)

# train_classifier_data_tr_attack = small_train_classifier_data[r[:int(0.5*len(r))]]
# train_classifier_label_tr_attack = small_train_classifier_label[r[:int(0.5*len(r))]]

# train_classifier_data_te_attack = small_train_classifier_data[r[int(0.5*len(r)):]]
# train_classifier_label_te_attack = small_train_classifier_label[r[int(0.5*len(r)):]]

r= np.arange(len(train_classifier_data))
np.random.shuffle(r)

train_classifier_data_tr_attack = train_classifier_data[r[:int(0.5*len(r))]]
train_classifier_label_tr_attack = train_classifier_label[r[:int(0.5*len(r))]]

train_classifier_data_te_attack = train_classifier_data[r[int(0.5*len(r)):]]
train_classifier_label_te_attack = train_classifier_label[r[int(0.5*len(r)):]]

len(train_classifier_data_te_attack)

In [ ]:


attack_model = InferenceAttack_HZ(100)
attack_model = torch.nn.DataParallel(attack_model).cuda()
attack_criterion = nn.MSELoss()
attack_optimizer = optim.Adam(attack_model.parameters(),lr=0.0001)

In [ ]:
resume_defense='./checkpoints_texas/Texas_softmax_NSH_attack_best'
print('==> Resuming from checkpoint..')
assert os.path.isfile(resume), 'Error: no checkpoint directory found!'
checkpoint_defense = os.path.dirname(resume_defense)
checkpoint_defense = torch.load(resume_defense)
attack_model.load_state_dict(checkpoint_defense['state_dict'])

In [ ]:
r= np.arange(len(train_classifier_data_te_attack))
np.random.shuffle(r)
train_classifier_data_te_attack = train_classifier_data_te_attack[r]
train_classifier_label_te_attack = train_classifier_label_te_attack[r]
test_data_attack = test_data[r]
test_label_attack = test_label[r]
train_classifier_data_te_attack_tensor = torch.from_numpy(train_classifier_data_te_attack).type(torch.FloatTensor)
train_classifier_label_te_attack_tensor = torch.from_numpy(train_classifier_label_te_attack).type(torch.LongTensor)
test_data_tensor = torch.from_numpy(test_data_attack).type(torch.FloatTensor)
test_label_tensor = torch.from_numpy(test_label_attack).type(torch.LongTensor)
test_loss, test_acc, sum_correct = test_attack_softmax(train_classifier_data_te_attack_tensor,train_classifier_label_te_attack_tensor
                                     ,test_data_tensor,test_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)
print ('test acc', test_acc, 'test_loss: ',test_loss)

In [ ]:



# train_attack_data = train_attack_data

# train_attack_label = train_attack_label

# test_data = test_data
# test_label = test_label

print('train_attack_data: ', len(train_attack_data), '  test_data: ',len(test_data))

In [ ]:


train_classifier_data_te_attack.shape

In [ ]:


test_loss, final_test_acc = test(torch.from_numpy(train_classifier_data_tr_attack).type(torch.FloatTensor) ,torch.from_numpy(train_classifier_label_tr_attack).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

test_loss, final_test_acc = test(torch.from_numpy(train_classifier_data_te_attack).type(torch.FloatTensor) ,torch.from_numpy(train_classifier_label_te_attack).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

test_loss, final_test_acc = test(torch.from_numpy(train_attack_data).type(torch.FloatTensor) ,torch.from_numpy(train_attack_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

test_loss, final_test_acc = test(torch.from_numpy(test_data).type(torch.FloatTensor) ,torch.from_numpy(test_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))



In [ ]:


best_acc= 0.0
batch_size=128
epochs = 200
for epoch in range(0, epochs):
#     adjust_learning_rate_attack(attack_optimizer, epoch)
    
    r= np.arange(len(train_classifier_data_tr_attack))
    np.random.shuffle(r)
    r1= np.arange(len(train_attack_data))
    np.random.shuffle(r1)
    r2 = r1[r]

    train_classifier_data_tr_attack = train_classifier_data_tr_attack[r]
    train_classifier_label_tr_attack = train_classifier_label_tr_attack[r]

    tr_attack_data = train_attack_data[r2]
    tr_attack_label = train_attack_label[r2]


    r= np.arange(len(train_classifier_data_te_attack))
    np.random.shuffle(r)
#     r1= np.arange(len(test_data)*0.5)
#     np.random.shuffle(r1)


    train_classifier_data_te_attack = train_classifier_data_te_attack[r]
    train_classifier_label_te_attack = train_classifier_label_te_attack[r]

    test_data_attack = test_data[r]
    test_label_attack = test_label[r]
    
    train_classifier_data_tr_attack_tensor = torch.from_numpy(train_classifier_data_tr_attack).type(torch.FloatTensor)
    train_classifier_label_tr_attack_tensor = torch.from_numpy(train_classifier_label_tr_attack).type(torch.LongTensor)

    train_classifier_data_te_attack_tensor = torch.from_numpy(train_classifier_data_te_attack).type(torch.FloatTensor)
    train_classifier_label_te_attack_tensor = torch.from_numpy(train_classifier_label_te_attack).type(torch.LongTensor)


#     r= np.arange(len(train_attack_data))
#     np.random.shuffle(r)

#     train_attack_data = train_attack_data[r]
#     train_attack_label = train_attack_label[r]

    train_attack_data_tensor = torch.from_numpy(tr_attack_data).type(torch.FloatTensor)
    train_attack_label_tensor = torch.from_numpy(tr_attack_label).type(torch.LongTensor)

    test_data_tensor = torch.from_numpy(test_data_attack).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label_attack).type(torch.LongTensor)
    print('\nEpoch: [%d | %d] , lr : 0.0001'% (epoch + 1, epochs))


    train_loss, train_acc = train_attack_softmax(train_classifier_data_tr_attack_tensor,train_classifier_label_tr_attack_tensor
                                         ,train_attack_data_tensor,train_attack_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    print ('train acc',train_acc)
    test_loss, test_acc, sum_correct = test_attack_softmax(train_classifier_data_te_attack_tensor,train_classifier_label_te_attack_tensor
                                         ,test_data_tensor,test_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    is_best = test_acc>best_acc or (1-test_acc)>best_acc
    
    best_acc = max(test_acc, best_acc)
    best_acc = max((1-test_acc), best_acc)
    if is_best or epoch+1 == epochs:
        save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': attack_model.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : attack_optimizer.state_dict(),
                }, False, filename='Texas_softmax_NSH_attack_best')
    
    print ('test acc',test_acc,best_acc)

print('Best classification acc:%.4f'%(final_test_acc))
print('Best attack acc:%.4f'%(best_acc))

In [ ]:


# Texas_5layer_balance_output_partition_alpha500_min_var_beta3000_epoch_90

best_acc= 0.0
batch_size=128
epochs = 200
for epoch in range(0, epochs):
#     adjust_learning_rate_attack(attack_optimizer, epoch)
    
    r= np.arange(len(train_classifier_data_tr_attack))
    np.random.shuffle(r)
    r1= np.arange(len(train_attack_data))
    np.random.shuffle(r1)
    r2 = r1[r]

    train_classifier_data_tr_attack = train_classifier_data_tr_attack[r]
    train_classifier_label_tr_attack = train_classifier_label_tr_attack[r]

    tr_attack_data = train_attack_data[r2]
    tr_attack_label = train_attack_label[r2]


    r= np.arange(len(train_classifier_data_te_attack))
    np.random.shuffle(r)
#     r1= np.arange(len(test_data)*0.5)
#     np.random.shuffle(r1)


    train_classifier_data_te_attack = train_classifier_data_te_attack[r]
    train_classifier_label_te_attack = train_classifier_label_te_attack[r]

    test_data_attack = test_data[r]
    test_label_attack = test_label[r]
    
    train_classifier_data_tr_attack_tensor = torch.from_numpy(train_classifier_data_tr_attack).type(torch.FloatTensor)
    train_classifier_label_tr_attack_tensor = torch.from_numpy(train_classifier_label_tr_attack).type(torch.LongTensor)

    train_classifier_data_te_attack_tensor = torch.from_numpy(train_classifier_data_te_attack).type(torch.FloatTensor)
    train_classifier_label_te_attack_tensor = torch.from_numpy(train_classifier_label_te_attack).type(torch.LongTensor)


#     r= np.arange(len(train_attack_data))
#     np.random.shuffle(r)

#     train_attack_data = train_attack_data[r]
#     train_attack_label = train_attack_label[r]

    train_attack_data_tensor = torch.from_numpy(tr_attack_data).type(torch.FloatTensor)
    train_attack_label_tensor = torch.from_numpy(tr_attack_label).type(torch.LongTensor)

    test_data_tensor = torch.from_numpy(test_data_attack).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label_attack).type(torch.LongTensor)
    print('\nEpoch: [%d | %d] , lr : 0.0001'% (epoch + 1, epochs))


    train_loss, train_acc = train_attack_softmax(train_classifier_data_tr_attack_tensor,train_classifier_label_tr_attack_tensor
                                         ,train_attack_data_tensor,train_attack_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    print ('train acc',train_acc)
    test_loss, test_acc, sum_correct = test_attack_softmax(train_classifier_data_te_attack_tensor,train_classifier_label_te_attack_tensor
                                         ,test_data_tensor,test_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    is_best = test_acc>best_acc or (1-test_acc)>best_acc
    
    best_acc = max(test_acc, best_acc)
    best_acc = max((1-test_acc), best_acc)
    if is_best or epoch+1 == epochs:
        save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': attack_model.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : attack_optimizer.state_dict(),
                }, False, filename='Texas_5layer_balance_output_partition_alpha500_min_var_beta3000_epoch_90_softmax_NSH_attack_best')
    
    print ('test acc',test_acc,best_acc)

print('Best classification acc:%.4f'%(final_test_acc))
print('Best attack acc:%.4f'%(best_acc))







In [ ]:
def adjust_learning_rate_attack(optimizer, epoch):
    global state
    if epoch in [40, 90]:
#     if (epoch+1)%100 == 0:
        state['lr'] *= 0.1 
#         state['lr'] *= 1
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [ ]:
#  NN attack model
defense_model = Defense_Model(100)
defense_model = torch.nn.DataParallel(defense_model).cuda()
defense_criterion = nn.MSELoss()
# defense_criterion = nn.CrossEntropyLoss()
at_lr = 0.001
state={}
state['lr']=at_lr
defense_optimizer = optim.Adam(defense_model.parameters(),lr=at_lr)
defense_model

In [ ]:
resume_defense='./checkpoints_texas/Texas_softmax_sort_NN_best'
print('==> Resuming from checkpoint..')
assert os.path.isfile(resume), 'Error: no checkpoint directory found!'
checkpoint_defense = os.path.dirname(resume_defense)
checkpoint_defense = torch.load(resume_defense)
defense_model.load_state_dict(checkpoint_defense['state_dict'])

In [ ]:
r= np.arange(len(train_classifier_data_te_attack))
np.random.shuffle(r)
#     r1= np.arange(len(test_data)*0.5)
#     np.random.shuffle(r1)


train_classifier_data_te_attack = train_classifier_data_te_attack[r]
train_classifier_label_te_attack = train_classifier_label_te_attack[r]

test_data_attack = test_data[r]
test_label_attack = test_label[r]
train_classifier_data_te_attack_tensor = torch.from_numpy(train_classifier_data_te_attack).type(torch.FloatTensor)
train_classifier_label_te_attack_tensor = torch.from_numpy(train_classifier_label_te_attack).type(torch.LongTensor)

test_data_tensor = torch.from_numpy(test_data_attack).type(torch.FloatTensor)
test_label_tensor = torch.from_numpy(test_label_attack).type(torch.LongTensor)
test_loss, test_acc, sum_correct = test_defense_softmax(train_classifier_data_te_attack_tensor,train_classifier_label_te_attack_tensor
                                     ,test_data_tensor,test_label_tensor,net,defense_model,criterion,defense_criterion,optimizer,defense_optimizer,epoch,use_cuda)


print ('test acc', test_acc, 'test_loss: ',test_loss)

In [ ]:

# 

best_acc= 0.0
batch_size=128
epochs = 100
for epoch in range(0, epochs):
    adjust_learning_rate_attack(defense_optimizer, epoch)
    
    r= np.arange(len(train_classifier_data_tr_attack))
    np.random.shuffle(r)
    r1= np.arange(len(train_attack_data))
    np.random.shuffle(r1)
    r2 = r1[r]

    train_classifier_data_tr_attack = train_classifier_data_tr_attack[r]
    train_classifier_label_tr_attack = train_classifier_label_tr_attack[r]

    tr_attack_data = train_attack_data[r2]
    tr_attack_label = train_attack_label[r2]


    r= np.arange(len(train_classifier_data_te_attack))
    np.random.shuffle(r)
#     r1= np.arange(len(test_data)*0.5)
#     np.random.shuffle(r1)


    train_classifier_data_te_attack = train_classifier_data_te_attack[r]
    train_classifier_label_te_attack = train_classifier_label_te_attack[r]

    test_data_attack = test_data[r]
    test_label_attack = test_label[r]

    
    train_classifier_data_tr_attack_tensor = torch.from_numpy(train_classifier_data_tr_attack).type(torch.FloatTensor)
    train_classifier_label_tr_attack_tensor = torch.from_numpy(train_classifier_label_tr_attack).type(torch.LongTensor)

    train_classifier_data_te_attack_tensor = torch.from_numpy(train_classifier_data_te_attack).type(torch.FloatTensor)
    train_classifier_label_te_attack_tensor = torch.from_numpy(train_classifier_label_te_attack).type(torch.LongTensor)


#     r= np.arange(len(train_attack_data))
#     np.random.shuffle(r)

#     train_attack_data = train_attack_data[r]
#     train_attack_label = train_attack_label[r]

    train_attack_data_tensor = torch.from_numpy(tr_attack_data).type(torch.FloatTensor)
    train_attack_label_tensor = torch.from_numpy(tr_attack_label).type(torch.LongTensor)

    test_data_tensor = torch.from_numpy(test_data_attack).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label_attack).type(torch.LongTensor)
    print('\nEpoch: [%d | %d] , lr : %f'% (epoch + 1, epochs, state['lr']))


    train_loss, train_acc = train_defense_softmax(train_classifier_data_tr_attack_tensor,train_classifier_label_tr_attack_tensor
                                         ,train_attack_data_tensor,train_attack_label_tensor,net,defense_model,criterion,defense_criterion,optimizer,defense_optimizer,epoch,use_cuda)

    print ('train acc',train_acc)
    test_loss, test_acc, sum_correct = test_defense_softmax(train_classifier_data_te_attack_tensor,train_classifier_label_te_attack_tensor
                                         ,test_data_tensor,test_label_tensor,net,defense_model,criterion,defense_criterion,optimizer,defense_optimizer,epoch,use_cuda)

    is_best = test_acc>best_acc or (1-test_acc)>best_acc
    
    best_acc = max(test_acc, best_acc)
    best_acc = max((1-test_acc), best_acc)
    if is_best or epoch+1 == epochs:
        save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': defense_model.state_dict(),                                       
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : defense_optimizer.state_dict(),
                }, False, filename='Texas_softmax_sort_NN_best')
    
    print ('test acc',test_acc,best_acc)

print('Best classification acc:%.4f'%(final_test_acc))
print('Best attack acc:%.4f'%(best_acc))



In [ ]:


class10_500_testset_te_label.shape

In [ ]:


# memg uard implementation

# evaluate_train_data = X[5000:10000]
# evaluate_test_data = X[40000:45000]
# evaluate_train_label = Y[5000:10000]
# evaluate_test_label = Y[40000:45000]

# evaluate_train_data = X[10000:20000]
# evaluate_test_data = X[40000:50000]
# evaluate_train_label = Y[10000:20000]
# evaluate_test_label = Y[40000:50000]

evaluate_train_data = class10_train_classifier_data[3500:]
evaluate_train_label = class10_train_classifier_label[3500:]

evaluate_test_data = class10_test_data[:3500]
evaluate_test_label = class10_test_label[:3500]

# evaluate_train_data = train_classifier_data_te_attack
# evaluate_train_label = train_classifier_label_te_attack

# evaluate_test_data = class10_500_testset_te_data
# evaluate_test_label = class10_500_testset_te_label


x_evaluate_attacker=np.concatenate((evaluate_train_data,evaluate_test_data),axis=0)
y_evaluate_attacker=np.concatenate((evaluate_train_label,evaluate_test_label),axis=0)

label_evaluate_attacker=np.zeros([x_evaluate_attacker.shape[0]],dtype=np.int)
label_evaluate_attacker[0:evaluate_train_data.shape[0]]=1

x_evaluate_attacker_tensor = torch.from_numpy(x_evaluate_attacker).type(torch.FloatTensor)
y_evaluate_attacker_tensor = torch.from_numpy(y_evaluate_attacker).type(torch.LongTensor)
print(x_evaluate_attacker_tensor.shape, y_evaluate_attacker_tensor.shape)

In [ ]:


batch_size = 128

net.eval()

f_evaluate_logits = []
f_evaluate_prob = []
softmax = nn.Softmax()
len_t =  (len(x_evaluate_attacker_tensor)//batch_size)+1
print(len_t)
for ind in range(len_t):
    # measure data loading time
    inputs = x_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = y_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
    
    # compute output
    outputs,_ = net(inputs)
    outputs_value = outputs.data.cpu().numpy()
#     loss = criterion(outputs, targets)
    prob_outputs = softmax(outputs)
    prob_outputs_value = prob_outputs.data.cpu().numpy()
    for i in range(len(prob_outputs_value)):
        f_evaluate_logits.append(outputs_value[i])
        f_evaluate_prob.append(prob_outputs_value[i])

In [ ]:


f_evaluate_logits_array = np.asarray(f_evaluate_logits)
f_evaluate_prob_array = np.asarray(f_evaluate_prob)
print(f_evaluate_logits_array.shape, f_evaluate_prob_array.shape)

In [ ]:


f_evaluate_origin=np.copy(f_evaluate_prob_array)  #keep a copy of original one
f_evaluate_logits_origin=np.copy(f_evaluate_logits_array)
#############as we sort the prediction sscores, back_index is used to get back original scores#############
sort_index=np.argsort(f_evaluate_prob_array,axis=1)
back_index=np.copy(sort_index)
for i in np.arange(back_index.shape[0]):
    back_index[i,sort_index[i,:]]=np.arange(back_index.shape[1])

# sort the output for attack model
f_evaluate=np.sort(f_evaluate_prob_array,axis=1)
f_evaluate_logits=np.sort(f_evaluate_logits_array,axis=1)

# f_evaluate=f_evaluate_prob_array
# f_evaluate_logits=f_evaluate_logits_array

In [ ]:


print("f evaluate shape: {}".format(f_evaluate.shape))
print("f evaluate logits shape: {}".format(f_evaluate_logits.shape))

In [ ]:


input_shape=f_evaluate.shape[1:]
input_shape

In [ ]:


defense_model = Defense_Model(10)
defense_model = torch.nn.DataParallel(defense_model).cuda()
defense_criterion = nn.MSELoss()
# defense_criterion = nn.CrossEntropyLoss()
at_lr = 0.0001
state={}
state['lr']=at_lr
defense_optimizer = optim.Adam(defense_model.parameters(),lr=at_lr)
defense_model

In [ ]:


resume_defense='./checkpoints_texas_10class_little_model_defense/Class10_1layer_No_defenss_200_epoch_200_softmax_defensemodel_epoch77'
# resume_defense='./checkpoints_texas_prune_little_model_defense/Modify2_var_max_alpha1_beta3_defense_epoch_5_softmax_defensemodel_epoch28'
print('==> Resuming attack model from checkpoint..')
assert os.path.isfile(resume_defense), 'Error: no checkpoint directory found!'
checkpoint_defense = os.path.dirname(resume_defense)
checkpoint_defense = torch.load(resume_defense)
defense_model.load_state_dict(checkpoint_defense['state_dict'])

In [ ]:


batch_size

In [ ]:


########evaluate the performance of defense's attack model on undefended data#######

defense_model.eval()

# softmax = nn.Softmax()

losses = AverageMeter()
top1 = AverageMeter()

sum_correct = 0.0

# f_evaluate_logits_tensor = torch.from_numpy(f_evaluate_logits).type(torch.FloatTensor)
f_evaluate_tensor = torch.from_numpy(f_evaluate).type(torch.FloatTensor)

att_labels = np.zeros(f_evaluate_tensor.size()[0])

# att_labels = np.zeros(f_evaluate_prob_tensor.size()[0])
# att_labels [:10000] =1.0
# att_labels [10000:] =0.0
att_labels [:len(evaluate_train_label)] =1.0
att_labels [len(evaluate_train_label):] =0.0
is_member_labels_tensor = torch.from_numpy(att_labels).type(torch.FloatTensor)
    
len_t =  (len(f_evaluate_tensor)//batch_size)+1
print('len_t: ', len_t)

list_attack_output = []
for ind in range(len_t):
    # measure data loading time
    inputs = f_evaluate_tensor[ind*batch_size:(ind+1)*batch_size]
#     targets = labels[ind*batch_size:(ind+1)*batch_size]

    attack_output, attack_output_h = defense_model(inputs)
#     .view([-1])
#     break
    is_member_labels = is_member_labels_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        is_member_labels = is_member_labels.cuda()

    v_is_member_labels = torch.autograd.Variable(is_member_labels)

    loss = defense_criterion(attack_output, v_is_member_labels)

    # measure accuracy and record loss
    #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
    list_attack_output.append(attack_output.data.cpu().numpy())
    prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
    losses.update(loss.data, inputs.size()[0])
    top1.update(prec1, inputs.size()[0])
#     print('attack_output: ', attack_output.shape, 'v_is_member_labels: ', v_is_member_labels.shape)
    correct = np.sum(np.equal((attack_output.data.cpu().numpy().reshape(-1) >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
#     print("correct: ", correct)
    sum_correct += correct
#     break
    # plot progress
    if ind%100==0:
        print  ('({batch}/{size}) | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                batch=ind + 1,
                size=len_t,
                loss=losses.avg,
                top1=top1.avg,
                ))

test_loss = losses.avg
test_acc = top1.avg
accurate_acc = sum_correct/len(f_evaluate)
print ('evaluate loss: {test_loss: .4f}, evaluate accuracy: {test_acc: .4f}, accurate accuracy: {accurate_acc: .4f}, sum correct: {sum_correct: .1f}'.format(test_loss=test_loss, test_acc=test_acc, accurate_acc=accurate_acc,sum_correct=sum_correct))

# (1/157) | Loss: 0.2280 | top1:  0.7109 
# (101/157) | Loss: 0.2244 | top1:  0.6838 
# evaluate loss:  0.2300, evaluate accuracy:  0.6151, accurate accuracy:  0.6155

In [ ]:


evaluate_train_data
evaluate_test_data
evaluate_train_label
evaluate_test_label
evaluate_train_data_tensor = torch.from_numpy(evaluate_train_data).type(torch.FloatTensor)
evaluate_train_label_tensor = torch.from_numpy(evaluate_train_label).type(torch.LongTensor)
evaluate_test_data_tensor = torch.from_numpy(evaluate_test_data).type(torch.FloatTensor)
evaluate_test_label_tensor = torch.from_numpy(evaluate_test_label).type(torch.LongTensor)

In [ ]:


batch_size = 128
attack_train_loss, attack_train_acc, attack_correct_sum = test_defense_softmax(evaluate_train_data_tensor,evaluate_train_label_tensor
                                                                               ,evaluate_test_data_tensor,evaluate_test_label_tensor,
                                                                               net,defense_model,criterion,defense_criterion,
                                                                               optimizer,defense_optimizer,epoch,use_cuda)

   

print ('attack loss: {test_loss: .4f}, attack acc: {test_acc: .4f}, attack correct sum: {attack_correct_sum: .1f}'.format(test_loss=attack_train_loss, test_acc=attack_train_acc,attack_correct_sum=attack_correct_sum))

In [ ]:


defense_model = InferenceAttack_HZ(10)
defense_model = torch.nn.DataParallel(defense_model).cuda()
defense_criterion = nn.MSELoss()
at_lr = 0.0001
state={}
state['lr']=at_lr
defense_optimizer = optim.Adam(defense_model.parameters(),lr=at_lr)

In [ ]:


resume_defense='./checkpoints_texas_10class_little_model_defense/Class10_1layer_No_defenss_200_epoch_200_softmax_epoch73'
print('==> Resuming attack model from checkpoint..')
assert os.path.isfile(resume_defense), 'Error: no checkpoint directory found!'
# checkpoint_defense = os.path.dirname(resume_defense)
checkpoint_defense = torch.load(resume_defense)
defense_model.load_state_dict(checkpoint_defense['state_dict'])

In [ ]:


batch_size = 128
attack_train_loss, attack_train_acc, attack_correct_sum = test_attack_softmax(evaluate_train_data_tensor,evaluate_train_label_tensor
                                         ,evaluate_test_data_tensor,evaluate_test_label_tensor,net,defense_model,criterion,defense_criterion,optimizer,defense_optimizer,epoch,use_cuda)

print ('attack loss: {test_loss: .4f}, attack acc: {test_acc: .4f}, attack correct sum: {attack_correct_sum: .1f}'.format(test_loss=attack_train_loss, test_acc=attack_train_acc,attack_correct_sum=attack_correct_sum))

In [ ]:


attack_train_loss, attack_train_acc, attack_correct_sum = test_attack_softmax_sort_mod(evaluate_train_data_tensor,evaluate_train_label_tensor
                                         ,evaluate_test_data_tensor,evaluate_test_label_tensor,net,defense_model,criterion,defense_criterion,optimizer,defense_optimizer,epoch,use_cuda)

print ('attack loss: {test_loss: .4f}, attack acc: {test_acc: .4f}, attack correct sum: {attack_correct_sum: .1f}'.format(test_loss=attack_train_loss, test_acc=attack_train_acc,attack_correct_sum=attack_correct_sum))

In [ ]:


attack_output_h.shape

In [ ]:


batch_size = 100
defense_model.eval()

# softmax = nn.Softmax()

losses = AverageMeter()
top1 = AverageMeter()

sum_correct = 0.0

# f_evaluate_logits_tensor = torch.from_numpy(result_array_logits).type(torch.FloatTensor)

# f_evaluate=np.sort(result_array,axis=1)
# f_evaluate_tensor = torch.from_numpy(f_evaluate).type(torch.FloatTensor)
f_evaluate_tensor = torch.from_numpy(f_evaluate_origin).type(torch.FloatTensor)

# comb_targets= torch.cat((check_test_trainset_label_tensor,check_test_label_tensor)).view([-1,1]).type(torch.cuda.FloatTensor)
comb_targets = y_evaluate_attacker_tensor.cuda()
att_labels = np.zeros(f_evaluate_tensor.size()[0])

# att_labels = np.zeros(f_evaluate_prob_tensor.size()[0])
att_labels [:len(evaluate_train_label)] =1.0
att_labels [len(evaluate_train_label):] =0.0
is_member_labels_tensor = torch.from_numpy(att_labels).type(torch.FloatTensor)
    
len_t =  (len(f_evaluate_tensor)//batch_size)+1
print('len_t: ', len_t)

# attack_criterion
# attack_optimizer

list_attack_output = []
for ind in range(len_t):
    # measure data loading time
    inputs = f_evaluate_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = comb_targets[ind*batch_size:(ind+1)*batch_size]
    
    sort_inputs, indices= torch.sort(inputs)
    
    one_hot_tr = torch.from_numpy((np.zeros((inputs.size()[0],inputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
    target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)

    for i in range(len(indices)):
        target_one_hot_tr[i] = target_one_hot_tr[i][indices[i]]
    
    infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
    
#     attack_output = attack_model(inputs,_,infer_input_one_hot).view([-1])

    attack_output, _ = defense_model(sort_inputs,_,infer_input_one_hot)
#     attack_output, _ = defense_model(inputs,_,infer_input_one_hot)
    attack_output = attack_output.view([-1])
#     .view([-1])
#     break
    is_member_labels = is_member_labels_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        is_member_labels = is_member_labels.cuda()

    v_is_member_labels = torch.autograd.Variable(is_member_labels)

    loss = defense_criterion(attack_output, v_is_member_labels)

    # measure accuracy and record loss
    #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
    list_attack_output.append(attack_output.data.cpu().numpy())
    prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
    losses.update(loss.data, inputs.size()[0])
    top1.update(prec1, inputs.size()[0])
#     print('attack_output: ', attack_output.shape, 'v_is_member_labels: ', v_is_member_labels.shape)
    correct = np.sum(np.equal((attack_output.data.cpu().numpy().reshape(-1) >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
#     print("correct: ", correct)
    sum_correct += correct
#     break
    # plot progress
    if ind%100==0:
        print  ('({batch}/{size}) | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                batch=ind + 1,
                size=len_t,
                loss=losses.avg,
                top1=top1.avg,
                ))

test_loss = losses.avg
test_acc = top1.avg
accurate_acc = sum_correct/len(f_evaluate)
print ('evaluate loss: {test_loss: .4f}, evaluate accuracy: {test_acc: .4f}, accurate accuracy: {accurate_acc: .4f}, sum correct: {sum_correct: .1f}'.format(test_loss=test_loss, test_acc=test_acc, accurate_acc=accurate_acc,sum_correct=sum_correct))

In [ ]:


infer_input_one_hot.shape

In [ ]:


result_folder = './checkpoints_texas_10class_little_model_defense'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)
if not os.path.exists(result_folder+"/attack"):
    os.makedirs(result_folder+"/attack")

In [ ]:


############################ Start-to calculate the defense logits for each input ##############################
user_label_dim=10
num_classes=1

label_mask_array=np.zeros([1,user_label_dim],dtype=np.float)
##########################################################
result_array=np.zeros(f_evaluate.shape,dtype=np.float)
result_array_logits=np.zeros(f_evaluate.shape,dtype=np.float)
success_fraction=0.0
max_iteration=300   #max iteration if can't find adversarial example that satisfies requirements
np.random.seed(1000)

# outputs = defense_model_intermidiate(inputs)

c1=1.0  #used to find adversarial examples 
c2=10.0    #penalty such that the index of max score is keeped
c3=0.1

In [ ]:


# Get the saved location npz model res

evaluation_noise_filepath="./checkpoints_texas_prune_little_model_defense/attack/Small_1_layer_No_defenss_60_epoch_54_softmax_epoch90_NSH_noise_data_strong.npz"

npz_defense=np.load(evaluation_noise_filepath)
npz_defense.files

In [ ]:


result_array=npz_defense['defense_output']
result_array_logits=npz_defense['defense_output_logits']
# origin_array=npz_defense['tc_output']
# origin_array_logits=npz_defense['tc_output_logits']

In [ ]:


# origin_array.shape

# x_evaluate_attacker

one_hot_tr = torch.from_numpy((np.zeros((f_evaluate_tensor.size()[0],f_evaluate_tensor.size(1))))).cuda().type(torch.cuda.FloatTensor)
target_one_hot_tr = one_hot_tr.scatter_(1, comb_targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)

infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)

In [ ]:


infer_input_one_hot.size()

In [ ]:


infer_input_one_hot[0]

In [ ]:


max_label=np.argmax(f_evaluate[0,:])
# result_array[0]

In [ ]:


# Sort NSH

# f_evaluate.shape[0]
start_time = time.time()
modified_predict_scores = np.array([])
success_fraction = 0.

for test_sample_id in np.arange(0,f_evaluate.shape[0]):
    max_label=np.argmax(f_evaluate[test_sample_id,:])
    origin_value=np.copy(f_evaluate[test_sample_id,:]).reshape(1,user_label_dim)
    origin_value_logits=np.copy(f_evaluate_logits[test_sample_id,:]).reshape(1,user_label_dim)
    label_mask_array[0,:]=0.0
    label_mask_array[0,max_label]=1.0
    label_mask_tonser = torch.from_numpy(label_mask_array).type(torch.FloatTensor).cuda()
#     label_mask_tonser = torch.from_numpy(label_mask_array).type(torch.FloatTensor)
    sample_f_logits=np.copy(origin_value_logits)
    sample_f_logits_tensor = torch.from_numpy(sample_f_logits).type(torch.FloatTensor).requires_grad_()
    sample_f_logits_tensor = sample_f_logits_tensor.cuda()
    sample_f_tensor = softmax(sample_f_logits_tensor)
    
    target_one_hot_tr = torch.from_numpy((np.zeros((sample_f_tensor.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
    target_one_hot_tr[:,y_evaluate_attacker[test_sample_id]] = 1

    target_one_hot_tr[0] = target_one_hot_tr[0][sort_index[test_sample_id]]
    infer_input_one_hot_tensor = torch.autograd.Variable(target_one_hot_tr)

#     print(sample_f_tensor)
    result_predict_scores_initial, sample_f_h =defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)
    result_predict_scores_initial = result_predict_scores_initial.item()
    
    ########## if the output score is already very close to 0.5, we can just use it for numerical reason
    if np.abs(result_predict_scores_initial-0.5)<=1e-5:
        success_fraction+=1.0
        result_array[test_sample_id,:]=origin_value[0,back_index[test_sample_id,:]]
        result_array_logits[test_sample_id,:]=origin_value_logits[0,back_index[test_sample_id,:]]
        continue
    last_iteration_result=np.copy(origin_value)[0,back_index[test_sample_id,:]]
    last_iteration_result_logits=np.copy(origin_value_logits)[0,back_index[test_sample_id,:]]
    success=True
    c3=0.1
    iterate_time=1
    while success==True: 
        sample_f=np.copy(origin_value_logits)
        j=1
        result_max_label=-1
        result_predict_scores=result_predict_scores_initial
    #         sample_f_tensor = torch.from_numpy(sample_f).type(torch.FloatTensor).requires_grad_()
    #         sample_f_tensor = softmax(sample_f_tensor).cuda()
        while j<max_iteration and (max_label!=result_max_label or (result_predict_scores-0.5)*(result_predict_scores_initial-0.5)>0):
            _, sample_f_h =defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)
            loss1 = torch.abs(sample_f_h)
#             print(loss1)
            correct_label = torch.sum(label_mask_tonser * sample_f_tensor, 1)
            wrong_label = torch.max((1-label_mask_tonser) * sample_f_tensor - 1e8*label_mask_tonser,1).values
            loss2 = F.relu(wrong_label-correct_label)
#             print(loss2)
            loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor).cuda())) #L-1 norm
#             loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor))) #L-1 norm
#             print(loss3)
            loss = c1 * loss1 + c2 * loss2 + c3 * loss3
            gradient_values = torch.autograd.grad(loss, sample_f_tensor)[0]
            gradient_valuesx = gradient_values/torch.norm(gradient_values)

            sample_f_logits_tensor = sample_f_logits_tensor-0.1*gradient_valuesx
            sample_f_tensor = softmax(sample_f_logits_tensor)
            result_predict_scores, _ = defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)
            result_predict_scores = result_predict_scores.item()
            result_max_label = torch.argmax(sample_f_tensor).item()
            j+=1
        if max_label!=result_max_label:
            if iterate_time==1:
                print("failed sample for label not same for id: {},c3:{} not add noise".format(test_sample_id,c3))
                success_fraction-=1.0
            break 
        result_predict_scores, _ = defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)
        if ((result_predict_scores.item()-0.5)*(result_predict_scores_initial-0.5))>0:
            if iterate_time==1:
                success_fraction-=1.0
                print("max iteration reached with id: {}, max score: {}, prediction_score: {}, c3: {}, not add noise".format(test_sample_id,torch.max(sample_f_tensor),result_predict_scores.item(),c3))
            break
        last_iteration_result[:]=sample_f_tensor.data.cpu().numpy()[0,back_index[test_sample_id,:]]
        last_iteration_result_logits[:]=sample_f_logits_tensor.data.cpu().numpy()[0,back_index[test_sample_id,:]]
        iterate_time+=1 
        c3=c3*10
        if c3>100000:
            break
            
#     modified_predict_scores = np.concatenate((modified_predict_scores, result_predict_scores.item()), axis=None)
    success_fraction+=1.0
    result_array[test_sample_id,:]=last_iteration_result[:]
    result_array_logits[test_sample_id,:]=last_iteration_result_logits[:]
    if test_sample_id%100==0:
        end = time.time() - start_time
        start_time = time.time()
        np.savez(result_folder+"/attack/"+"Class10_1layer_No_defenss_100_epoch_32_sort_mod_softmax_epoch94_noise_data_{}.npz".format("strong"),defense_output=result_array,defense_output_logits=result_array_logits)
        print("test sample id: {}, time spend per 100 test sample: {}".format(test_sample_id, end))
        
print("Success fraction: {}".format(success_fraction/float(f_evaluate.shape[0])))

In [ ]:


success_fraction

In [ ]:


# NSH attack model

defense_model.eval()
# f_evaluate.shape[0]
start_time = time.time()
modified_predict_scores = np.array([])
success_fraction = 0.

for test_sample_id in np.arange(0,f_evaluate.shape[0]):
# for test_sample_id in np.arange(0,101):

    max_label=np.argmax(f_evaluate[test_sample_id,:])
    origin_value=np.copy(f_evaluate[test_sample_id,:]).reshape(1,user_label_dim)
    origin_value_logits=np.copy(f_evaluate_logits[test_sample_id,:]).reshape(1,user_label_dim)
    infer_input_one_hot_tensor = infer_input_one_hot[test_sample_id,:]
    label_mask_array[0,:]=0.0
    label_mask_array[0,max_label]=1.0
    label_mask_tonser = torch.from_numpy(label_mask_array).type(torch.FloatTensor).cuda()
#     label_mask_tonser = torch.from_numpy(label_mask_array).type(torch.FloatTensor)
    sample_f_logits=np.copy(origin_value_logits)
    sample_f_logits_tensor = torch.from_numpy(sample_f_logits).type(torch.FloatTensor).requires_grad_()
    sample_f_logits_tensor = sample_f_logits_tensor.cuda()
    sample_f_tensor = softmax(sample_f_logits_tensor)
    infer_input_one_hot_tensor = infer_input_one_hot_tensor.view(sample_f_tensor.shape)
#     print(sample_f_tensor)
    result_predict_scores_initial, sample_f_h =defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)

    result_predict_scores_initial = result_predict_scores_initial.item()
#     break
    ########## if the output score is already very close to 0.5, we can just use it for numerical reason
    if np.abs(result_predict_scores_initial-0.5)<=1e-5:
        success_fraction+=1.0
        result_array[test_sample_id,:]=origin_value[0,back_index[test_sample_id,:]]
        result_array_logits[test_sample_id,:]=origin_value_logits[0,back_index[test_sample_id,:]]
        continue
    last_iteration_result=np.copy(origin_value)[0,back_index[test_sample_id,:]]
    last_iteration_result_logits=np.copy(origin_value_logits)[0,back_index[test_sample_id,:]]
    success=True
    c3=0.1
    iterate_time=1
    while success==True: 
        sample_f=np.copy(origin_value_logits)
        j=1
        result_max_label=-1
        result_predict_scores=result_predict_scores_initial
    #         sample_f_tensor = torch.from_numpy(sample_f).type(torch.FloatTensor).requires_grad_()
    #         sample_f_tensor = softmax(sample_f_tensor).cuda()
        while j<max_iteration and (max_label!=result_max_label or (result_predict_scores-0.5)*(result_predict_scores_initial-0.5)>0):
            _, sample_f_h =defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)
            loss1 = torch.abs(sample_f_h)
#             print(loss1)
            correct_label = torch.sum(label_mask_tonser * sample_f_tensor, 1)
            wrong_label = torch.max((1-label_mask_tonser) * sample_f_tensor - 1e8*label_mask_tonser,1).values
            loss2 = F.relu(wrong_label-correct_label)
#             print(loss2)
            loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor).cuda())) #L-1 norm
#             loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor))) #L-1 norm
#             print(loss3)
            loss = c1 * loss1 + c2 * loss2 + c3 * loss3
            gradient_values = torch.autograd.grad(loss, sample_f_tensor)[0]
            gradient_valuesx = gradient_values/torch.norm(gradient_values)
            
            sample_f_logits_tensor = sample_f_logits_tensor-0.1*gradient_valuesx
            sample_f_tensor = softmax(sample_f_logits_tensor)
            result_predict_scores, _ = defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)
            result_predict_scores = result_predict_scores.item()
            result_max_label = torch.argmax(sample_f_tensor).item()
            j+=1
        if max_label!=result_max_label:
            if iterate_time==1:
                print("failed sample for label not same for id: {},c3:{} not add noise".format(test_sample_id,c3))
                success_fraction-=1.0
            break 
        result_predict_scores, _ = defense_model(sample_f_tensor,_,infer_input_one_hot_tensor)
        if ((result_predict_scores.item()-0.5)*(result_predict_scores_initial-0.5))>0:
            if iterate_time==1:
                success_fraction-=1.0
                print("max iteration reached with id: {}, max score: {}, prediction_score: {}, c3: {}, not add noise".format(test_sample_id,torch.max(sample_f_tensor),result_predict_scores.item(),c3))
            break
        last_iteration_result[:]=sample_f_tensor.data.cpu().numpy()
        last_iteration_result_logits[:]=sample_f_logits_tensor.data.cpu().numpy()
        iterate_time+=1 
        c3=c3*10
        if c3>100000:
            break
            
#     modified_predict_scores = np.concatenate((modified_predict_scores, result_predict_scores.item()), axis=None)
    success_fraction+=1.0
    result_array[test_sample_id,:]=last_iteration_result[:]
    result_array_logits[test_sample_id,:]=last_iteration_result_logits[:]
    if test_sample_id%100==0:
        end = time.time() - start_time
        start_time = time.time()
        np.savez(result_folder+"/attack/"+"Small_1_layer_No_defenss_60_epoch_54_softmax_epoch90_NSH_noise_data_{}.npz".format("strong"),defense_output=result_array,defense_output_logits=result_array_logits)
        print("test sample id: {}, time spend per 100 test sample: {}".format(test_sample_id, end))
#         break
        
print("Success fraction: {}".format(success_fraction/float(f_evaluate.shape[0])))

In [ ]:


success_fraction

In [ ]:


# f_evaluate.shape[0]
start_time = time.time()
success_fraction = 0.0
modified_predict_scores = np.array([])

for test_sample_id in np.arange(0,f_evaluate.shape[0]):
    max_label=np.argmax(f_evaluate[test_sample_id,:])
    origin_value=np.copy(f_evaluate[test_sample_id,:]).reshape(1,user_label_dim)
    origin_value_logits=np.copy(f_evaluate_logits[test_sample_id,:]).reshape(1,user_label_dim)
    label_mask_array[0,:]=0.0
    label_mask_array[0,max_label]=1.0
    label_mask_tonser = torch.from_numpy(label_mask_array).type(torch.FloatTensor).cuda()
#     label_mask_tonser = torch.from_numpy(label_mask_array).type(torch.FloatTensor)
    sample_f_logits=np.copy(origin_value_logits)
    sample_f_logits_tensor = torch.from_numpy(sample_f_logits).type(torch.FloatTensor).requires_grad_()
    sample_f_logits_tensor = sample_f_logits_tensor.cuda()
    sample_f_tensor = softmax(sample_f_logits_tensor)
#     print(sample_f_tensor)
    result_predict_scores_initial, sample_f_h =defense_model(sample_f_tensor)
    result_predict_scores_initial = result_predict_scores_initial.item()
    
    ########## if the output score is already very close to 0.5, we can just use it for numerical reason
    if np.abs(result_predict_scores_initial-0.5)<=1e-5:
        success_fraction+=1.0
        result_array[test_sample_id,:]=origin_value[0,back_index[test_sample_id,:]]
        result_array_logits[test_sample_id,:]=origin_value_logits[0,back_index[test_sample_id,:]]
        continue
    last_iteration_result=np.copy(origin_value)[0,back_index[test_sample_id,:]]
    last_iteration_result_logits=np.copy(origin_value_logits)[0,back_index[test_sample_id,:]]
    success=True
    c3=0.1
    iterate_time=1
    while success==True: 
        sample_f=np.copy(origin_value_logits)
        j=1
        result_max_label=-1
        result_predict_scores=result_predict_scores_initial
    #         sample_f_tensor = torch.from_numpy(sample_f).type(torch.FloatTensor).requires_grad_()
    #         sample_f_tensor = softmax(sample_f_tensor).cuda()
        while j<max_iteration and (max_label!=result_max_label or (result_predict_scores-0.5)*(result_predict_scores_initial-0.5)>0):
            _, sample_f_h =defense_model(sample_f_tensor)
            loss1 = torch.abs(sample_f_h)
#             print(loss1)
            correct_label = torch.sum(label_mask_tonser * sample_f_tensor, 1)
            wrong_label = torch.max((1-label_mask_tonser) * sample_f_tensor - 1e8*label_mask_tonser,1).values
            loss2 = F.relu(wrong_label-correct_label)
#             print(loss2)
            loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor).cuda())) #L-1 norm
#             loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor))) #L-1 norm
#             print(loss3)
            loss = c1 * loss1 + c2 * loss2 + c3 * loss3
            gradient_values = torch.autograd.grad(loss, sample_f_tensor)[0]
            gradient_valuesx = gradient_values/torch.norm(gradient_values)

            sample_f_logits_tensor = sample_f_logits_tensor-0.1*gradient_valuesx
            sample_f_tensor = softmax(sample_f_logits_tensor)
            result_predict_scores, _ = defense_model(sample_f_tensor)
            result_predict_scores = result_predict_scores.item()
            result_max_label = torch.argmax(sample_f_tensor).item()
            j+=1
        if max_label!=result_max_label:
            if iterate_time==1:
                print("failed sample for label not same for id: {},c3:{} not add noise".format(test_sample_id,c3))
                success_fraction-=1.0
            break 
        result_predict_scores, _ = defense_model(sample_f_tensor)
        if ((result_predict_scores.item()-0.5)*(result_predict_scores_initial-0.5))>0:
            if iterate_time==1:
                success_fraction-=1.0
                print("max iteration reached with id: {}, max score: {}, prediction_score: {}, c3: {}, not add noise".format(test_sample_id,torch.max(sample_f_tensor),result_predict_scores.item(),c3))
            break
        last_iteration_result[:]=sample_f_tensor.data.cpu().numpy()[0,back_index[test_sample_id,:]]
        last_iteration_result_logits[:]=sample_f_logits_tensor.data.cpu().numpy()[0,back_index[test_sample_id,:]]
        iterate_time+=1 
        c3=c3*10
        if c3>100000:
            break
            
#     modified_predict_scores = np.concatenate((modified_predict_scores, result_predict_scores.item()), axis=None)
    success_fraction+=1.0
    result_array[test_sample_id,:]=last_iteration_result[:]
    result_array_logits[test_sample_id,:]=last_iteration_result_logits[:]
    if test_sample_id%100==0:
        end = time.time() - start_time
        start_time = time.time()
        np.savez(result_folder+"/attack/"+"Class10_1layer_No_defenss_100_epoch_32_softmax_defensemodel_epoch77_noise_data_{}.npz".format("modified"),defense_output=result_array,defense_output_logits=result_array_logits)
        print("test sample id: {}, time spend per 100 test sample: {}".format(test_sample_id, end))
        
print("Success fraction: {}".format(success_fraction/float(f_evaluate.shape[0])))

In [ ]:


success_fraction

In [ ]:


result_array[9999]

In [ ]:


test_array = result_array[0:100, :]
test_array_logits = result_array_logits[0:100, :]

In [ ]:


test_array[0]

In [ ]:


test_array_logits[0]

In [ ]:


one_hot_tr = torch.from_numpy((np.zeros((f_evaluate_tensor.size()[0],f_evaluate_tensor.size(1))))).cuda().type(torch.cuda.FloatTensor)
target_one_hot_tr = one_hot_tr.scatter_(1, y_evaluate_attacker_tensor.type(torch.cuda.LongTensor).view([-1,1]).data,1)
infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)

In [ ]:


# memg test
defense_model.eval()

# softmax = nn.Softmax()

# f_evaluate_logits_tensor = torch.from_numpy(f_evaluate_logits).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(np.sort(f_evaluate_origin,axis=1)).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(f_evaluate_origin).type(torch.FloatTensor)
f_evaluate_tensor = torch.from_numpy(f_evaluate).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(np.sort(origin_array,axis=1)).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(origin_array).type(torch.FloatTensor)

batch_size = 128
len_t =  (len(f_evaluate_tensor)//batch_size)+1
print('len_t: ', len_t)
# predict_origin = np.zeros(f_evaluate.shape[0],dtype=np.float)
predict_origin= np.array([])
for ind in range(len_t):
    # measure data loading time
    inputs = f_evaluate_tensor[ind*batch_size:(ind+1)*batch_size]
    input_one_hot_tensor = infer_input_one_hot[ind*batch_size:(ind+1)*batch_size]
    attack_output, attack_output_h = defense_model(inputs)
#     attack_output, attack_output_h = defense_model(inputs, _ ,input_one_hot_tensor)
    predict_origin = np.concatenate((predict_origin, attack_output.data.cpu().numpy()), axis=None)

In [ ]:


defense_model.eval()

# softmax = nn.Softmax()

# f_evaluate_logits_tensor = torch.from_numpy(f_evaluate_logits).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(np.sort(f_evaluate_origin,axis=1)).type(torch.FloatTensor)
f_evaluate_tensor = torch.from_numpy(f_evaluate_origin).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(f_evaluate).type(torch.FloatTensor)

# f_evaluate_tensor = torch.from_numpy(np.sort(origin_array,axis=1)).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(origin_array).type(torch.FloatTensor)

batch_size = 128
len_t =  (len(f_evaluate_tensor)//batch_size)+1
print('len_t: ', len_t)
# predict_origin = np.zeros(f_evaluate.shape[0],dtype=np.float)
predict_origin= np.array([])
for ind in range(len_t):
    # measure data loading time
    inputs = f_evaluate_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = y_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    
    sort_inputs, indices= torch.sort(inputs)
        
    one_hot_tr = torch.from_numpy((np.zeros((inputs.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
    target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)

    for i in range(len(indices)):
        target_one_hot_tr[i] = target_one_hot_tr[i][indices[i]]

    infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
    
#     attack_output, attack_output_h = defense_model(inputs)
    attack_output, attack_output_h = defense_model(sort_inputs, _ ,infer_input_one_hot)
    predict_origin = np.concatenate((predict_origin, attack_output.data.cpu().numpy()), axis=None)
#     if infer_input_one_hot[:,99] == 0:
#         break

In [ ]:


result_array.shape

In [ ]:


f_evaluate[3499]

In [ ]:


test_data = f_evaluate[3499]
# test_data = [0.09613917, 0.09760948, 0.09762944, 0.09764805, 0.09819929,0.09827515, 0.09849446, 0.09960721, 0.10178852, 0.1016092]
np.sum(test_data)

In [ ]:


test_data_tensor = torch.from_numpy(np.sort(test_data)).type(torch.FloatTensor).cuda()
test_data_tensor = test_data_tensor.view(1,-1)
test_data_tensor.shape
attack_output, attack_output_h = defense_model(test_data_tensor)
attack_output

In [ ]:


max_data = f_evaluate[:,9]
max_data.shape
max_data_train = max_data[:3500]
max_data_test = max_data[3500:]
sort_max_data_train = np.sort(max_data_train)
sort_max_data_test = np.sort(max_data_test)

In [ ]:


sort_max_data_train[3500-2483]

In [ ]:


sort_max_data_test[1750]
np.sum(sort_max_data_test>0.85)

In [ ]:


predict_modified= np.array([])
# np.sort(result_array,axis=1)
# result_array_tensor = torch.from_numpy(np.sort(result_array,axis=1)).type(torch.FloatTensor).cuda()
result_array_tensor = torch.from_numpy(result_array).type(torch.FloatTensor).cuda()

for ind in range(len_t):
    # measure data loading time
    inputs = result_array_tensor[ind*batch_size:(ind+1)*batch_size]
    input_one_hot_tensor = infer_input_one_hot[ind*batch_size:(ind+1)*batch_size]
#     attack_output, attack_output_h = defense_model(inputs)
    attack_output, attack_output_h = defense_model(inputs, _ ,input_one_hot_tensor)

    predict_modified = np.concatenate((predict_modified, attack_output.data.cpu().numpy()), axis=None)
#     break

# predict_modified.shape

In [ ]:


predict_modified= np.array([])
# np.sort(result_array,axis=1)
# result_array_tensor = torch.from_numpy(np.sort(result_array,axis=1)).type(torch.FloatTensor).cuda()
result_array_tensor = torch.from_numpy(result_array).type(torch.FloatTensor).cuda()

for ind in range(len_t):
    # measure data loading time
    inputs = result_array_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = y_evaluate_attacker_tensor[ind*batch_size:(ind+1)*batch_size]
    
    sort_inputs, indices= torch.sort(inputs)
        
    one_hot_tr = torch.from_numpy((np.zeros((inputs.size()[0],outputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
    target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)

    for i in range(len(indices)):
        target_one_hot_tr[i] = target_one_hot_tr[i][indices[i]]

    infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
    
#     attack_output, attack_output_h = defense_model(inputs)
    attack_output, attack_output_h = defense_model(sort_inputs, _ ,infer_input_one_hot)

    predict_modified = np.concatenate((predict_modified, attack_output.data.cpu().numpy()), axis=None)

In [ ]:


predict_origin

In [ ]:


# f_evaluate_noise = f_evaluate_origin
# predict_modified = predict_origin

In [ ]:


# memg test
scenario='full'
if scenario=='full':
    predict_result_origin=np.where(predict_origin > 0.5, 1, 0)
#     predict_result_defense=np.where(predict_modified > 0.5, 1, 0)
else:
    raise NotImplementedError

epsilon_value_list=["1.0","0.7","0.5","0.4","0.3","0.2","0.1","0.0"] # "0.05","0.03","0.02","0.01",
# epsilon_value_list=["0.2","0.1","0.07","0.05","0.03","0.01","0.0"]
# epsilon_value_list=["0.0"]
epsilon_value_list=[float(t) for t in epsilon_value_list]

In [ ]:


f_evaluate_noise = result_array
# f_evaluate_origin = origin_array
predict_result_defense.shape

In [ ]:


predict_origin.shape

In [ ]:


count = 0
for i in np.arange(f_evaluate_origin.shape[0]):
    if np.equal(predict_result_origin[i] , label_evaluate_attacker[i]):
        count+=1
    
count

In [ ]:


count_train = 0
for i in np.arange(int(f_evaluate_origin.shape[0]*0.5)):
    if np.equal(predict_result_origin[i] , label_evaluate_attacker[i]):
        count_train+=1
    
count_test = 0
for i in np.arange(int(f_evaluate_origin.shape[0]*0.5), f_evaluate_origin.shape[0]):
    if np.equal(predict_result_origin[i] , label_evaluate_attacker[i]):
        count_test+=1
    
print(count_train, count_test)

In [ ]:


count = 0
for i in np.arange(f_evaluate_origin.shape[0]):
    if np.equal(predict_result_defense[i] , label_evaluate_attacker[i]):
        count+=1
    
count

In [ ]:


count_train = 0
for i in np.arange(int(f_evaluate_origin.shape[0]*0.5)):
    if np.equal(predict_result_defense[i] , label_evaluate_attacker[i]):
        count_train+=1
    
count_test = 0
for i in np.arange(int(f_evaluate_origin.shape[0]*0.5), f_evaluate_origin.shape[0]):
    if np.equal(predict_result_defense[i] , label_evaluate_attacker[i]):
        count_test+=1
    
print(count_train, count_test)

In [ ]:


# scenario='full'
# if scenario=='full':
#     predict_result_origin=np.where(predict_origin > 0.5, 1, 0)
#     predict_result_defense=np.where(predict_modified > 0.5, 1, 0)
# else:
#     raise NotImplementedError

# epsilon_value_list=["1.0","0.7","0.5","0.3","0.1","0.0"]
# epsilon_value_list=[float(t) for t in epsilon_value_list]

inference_accuracy_list=[]
ori_list = []
noise_list = []
for epsilon_value in epsilon_value_list: 
    inference_accuracy=0.0
    ori = 0.0
    noi = 0.0
    max_noise = 0.0
    np.random.seed(100)  
    for i in np.arange(f_evaluate_origin.shape[0]):
        distortion_noise=np.sum(np.abs(f_evaluate_origin[i,:]-f_evaluate_noise[i,:]))
        if distortion_noise > max_noise:
            max_noise = distortion_noise
        p_value=0.0
        if np.abs(predict_origin[i]-0.5)<=np.abs(predict_modified[i]-0.5):
            p_value=0.0
        else:
            p_value=min(epsilon_value/distortion_noise,1.0)
        
        if predict_result_origin[i]==label_evaluate_attacker[i]:
            ori += 1
            inference_accuracy+=1.0-p_value
        if predict_result_defense[i]==label_evaluate_attacker[i]:
            noi += 1
            inference_accuracy+=p_value
#         print('distortion_noise: ', distortion_noise, ' p_value: ',p_value,' inference_accuracy: ',inference_accuracy)
    inference_accuracy_list.append(inference_accuracy/(float(f_evaluate_origin.shape[0])))
    ori_list.append(ori)
    noise_list.append(noi)
#     break
    
print("Budget list: {}".format(epsilon_value_list))
print("inference accuracy list: {}".format(inference_accuracy_list))
print("ori_list: {}".format(ori_list))
print("noise_list: {}".format(noise_list))
print("max_noise: {}".format(max_noise))
# Budget list: [1.0, 0.7, 0.5, 0.3, 0.1, 0.0]
# inference accuracy list: [0.5021911142220747, 0.5048992830892228, 0.5120250427638868, 0.5286259040624035, 0.5731544218872452, 0.60825]

In [ ]:


from matplotlib import pyplot
from matplotlib.pylab import plt
# pyplot.plot(predict_origin)
# pyplot.plot(f_evaluate_origin[3])
# pyplot.plot(f_evaluate_origin[4])
# plt.plot(f_evaluate_origin[0])
# plt.plot(f_evaluate_origin[1])
# plt.plot(f_evaluate_origin[2])
for i in range(100):
#     plt.plot(f_evaluate_origin[i])
    plt.plot(f_evaluate[i])
#     plt.plot(result_array[i])
#     plt.plot(sort_result_array[i])
# for i in range(f_evaluate_origin.shape[0]):
#     pyplot.plot(f_evaluate_origin[i])
    
# plt.ylim(0, 0.7)
plt.show()

In [ ]:


predict_train = []
predict_test = []

for i in range(len(predict_origin)):
    if predict_origin[i] > 0.5:
        predict_train.append(i)
    else:
        predict_test.append(i)
        
print(len(predict_train))
print(len(predict_test))

In [ ]:



fig = plt.figure(figsize=(30,20))
plt.ylim(0.05,0.25)
for i in range(len(predict_train)):
#     plt.plot(f_evaluate_origin[i])
    plt.plot(f_evaluate[predict_train[i]])

plt.show()
# fig.savefig('Class10_1layer_Modify_min_sort_diff55_alpha0_beta10_defense_epoch_7_predict_train.png')

In [ ]:


fig = plt.figure(figsize=(30,20))
plt.ylim(0.05,0.25)
for i in range(len(predict_test)):
#     plt.plot(f_evaluate_origin[i])
    plt.plot(f_evaluate[predict_test[i]])

plt.show()
# fig.savefig('Class10_1layer_Modify_min_sort_diff55_alpha0_beta10_defense_epoch_7_predict_test.png')

In [ ]:


# No_defense_20_epoch_2_prune_99_retrain_epoch_99

fig = plt.figure(figsize=(30,20))
plt.ylim(0.05,0.25)
# for i in range(3500,7000):
for i in range(2500,5000):
#     plt.plot(result_array[i])
    pyplot.plot(f_evaluate_origin[i])
# fig.savefig('Class10_1layer_Modify_min_sort_diff55_alpha0_beta10_defense_epoch_7_testset.png')

In [ ]:



fig = plt.figure(figsize=(30,20))
plt.ylim(0.05,0.25)
# for i in range(0,3500):
for i in range(0,2500):
#     plt.plot(result_array[i])
    pyplot.plot(f_evaluate_origin[i])
# fig.savefig('Class10_1layer_Modify_min_sort_diff55_alpha0_beta10_defense_epoch_7_traintset.png')

In [ ]:


sort_result_array = np.sort(result_array,axis=1)

In [ ]:


f_evaluate[:,9].shape

In [ ]:


hsit_train = np.histogram(f_evaluate[0:2500,9],range=[0.1,0.25],bins=10)
sum(hsit_train[0])
hsit_train
# plt.hist(f_evaluate[0:2500,9],range=[0.1,0.25],bins=20)
plt.hist(f_evaluate[0:2500,9],bins=50)

In [ ]:


hsit_test = np.histogram(f_evaluate[2500:5000,9],range=[0.1,0.25],bins=10)
hsit_test
# sum(hsit_test[0])
# plt.hist(f_evaluate[2500:5000,9],range=[0.1,0.25],bins=20)
plt.hist(f_evaluate[2500:5000,9],bins=50)

In [ ]:


fig = plt.figure(figsize=(30,20))
plt.ylim(0.05,0.25)
for i in range(2500,5000):
#     plt.plot(sort_result_array[i])
    pyplot.plot(f_evaluate[i])
# fig.savefig('Class10_1layer_Modify_min_sort_diff55_alpha0_beta10_defense_epoch_7_sort_testset.png')

In [ ]:


fig = plt.figure(figsize=(30,20))
plt.ylim(0.05,0.25)
for i in range(0,2500):
#     plt.plot(sort_result_array[i])
    pyplot.plot(f_evaluate[i])
# fig.savefig('Class10_1layer_Modify_min_sort_diff55_alpha0_beta10_defense_epoch_7_sort_trainset.png')

In [ ]:


# Modify6_var_max_alpha5_defense_epoch_15

fig = plt.figure(figsize=(30,20))
# plt.ylim(-0.05, 0.65)
for i in range(10000,20000):
    pyplot.plot(f_evaluate_origin[i])
#     pyplot.plot(f_evaluate_logits_origin[i])
fig.savefig('Modify6_var_max_alpha5_defense_epoch_15_testset.png')

In [ ]:


# save the results

# np.savez(result_folder+"/attack/"+"noise_data_{}.npz".format("No_defense_20_epoch_6_40000_evaluation"),defense_output=result_array,defense_output_logits=result_array_logits,tc_output=f_evaluate_origin,tc_output_logits=f_evaluate_logits_origin,predict_origin=predict_origin,predict_modified=predict_modified, label_evaluate=att_labels)

In [ ]:


attack_model = InferenceAttack_HZ(100)
attack_model = torch.nn.DataParallel(attack_model).cuda()
attack_criterion = nn.MSELoss()
at_lr = 0.0001
state={}
state['lr']=at_lr
attack_optimizer = optim.Adam(attack_model.parameters(),lr=at_lr)

In [ ]:


resume_att='./checkpoints_texas_prune_little_model_defense/Small_No_defense_20_epoch_20_softmax_epoch97'
print('==> Resuming attack model from checkpoint..')
assert os.path.isfile(resume_att), 'Error: no checkpoint directory found!'
checkpoint_att = os.path.dirname(resume_att)
checkpoint_att = torch.load(resume_att)
attack_model.load_state_dict(checkpoint_att['state_dict'])

In [ ]:


batch_size = 128

r1 = np.arange(len(train_classifier_data_tr_attack))
# np.random.shuffle(r1)

check_train_trainset_data_tensor = torch.from_numpy(train_classifier_data_tr_attack[r1]).type(torch.FloatTensor)
check_train_trainset_label_tensor = torch.from_numpy(train_classifier_label_tr_attack[r1]).type(torch.LongTensor)

r2 = np.arange(len(train_attack_data))
# np.random.shuffle(r2)
check_train_attack_data_tensor = torch.from_numpy(train_attack_data[r2]).type(torch.FloatTensor)
check_train_attack_label_tensor = torch.from_numpy(train_attack_label[r2]).type(torch.LongTensor)

r3 = np.arange(len(train_classifier_data_te_attack))
# np.random.shuffle(r3)

check_test_trainset_data_tensor = torch.from_numpy(train_classifier_data_te_attack[r3]).type(torch.FloatTensor)
check_test_trainset_label_tensor = torch.from_numpy(train_classifier_label_te_attack[r3]).type(torch.LongTensor)

r4 = np.arange(len(test_data))
# np.random.shuffle(r4)

check_test_data_tensor = torch.from_numpy(test_data[r4]).type(torch.FloatTensor)
check_test_label_tensor = torch.from_numpy(test_label[r4]).type(torch.LongTensor)

print(len(check_train_trainset_data_tensor))
print(len(check_train_attack_data_tensor))
print(len(check_test_trainset_data_tensor))
print(len(check_test_data_tensor))

In [ ]:


# check train accuracy and test accuracy of attack model with correspond data

attack_train_loss, attack_train_acc, attack_train_correct_sum = test_attack_softmax(check_train_trainset_data_tensor,check_train_trainset_label_tensor
                                         ,check_train_attack_data_tensor,check_train_attack_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

print ('attack_train_loss: {test_loss: .4f}, attack_train_acc: {test_acc: .4f}'.format(test_loss=attack_train_loss, test_acc=attack_train_acc))

attack_test_loss, attack_test_acc, attack_test_correct_sum = test_attack_softmax(check_test_trainset_data_tensor,check_test_trainset_label_tensor
                                         ,check_test_data_tensor,check_test_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

print ('attack_test_loss: {test_loss: .4f}, attack_test_acc: {test_acc: .4f}'.format(test_loss=attack_test_loss, test_acc=attack_test_acc))

In [ ]:


check_test_trainset_data_tensor = torch.from_numpy(evaluate_train_data).type(torch.FloatTensor)
check_test_trainset_label_tensor = torch.from_numpy(evaluate_train_label).type(torch.LongTensor)
result_array_trainset_logits_tensor = torch.from_numpy(result_array_logits[:5000]).type(torch.FloatTensor)

check_test_data_tensor = torch.from_numpy(evaluate_test_data).type(torch.FloatTensor)
check_test_label_tensor = torch.from_numpy(evaluate_test_label).type(torch.LongTensor)
result_array_testset_logits_tensor = torch.from_numpy(result_array_logits[5000:]).type(torch.FloatTensor)


print(len(check_test_trainset_data_tensor))
print(len(check_test_data_tensor))
print(len(result_array_testset_logits_tensor))

In [ ]:


check_test_trainset_data_tensor.shape

In [ ]:


attack_test_loss, attack_test_acc, attack_test_correct_sum = test_attack_softmax_modify(check_test_trainset_data_tensor,check_test_trainset_label_tensor,result_array_trainset_logits_tensor
                                         ,check_test_data_tensor,check_test_label_tensor,result_array_testset_logits_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

print ('attack_test_loss: {test_loss: .4f}, attack_test_acc: {test_acc: .4f}'.format(test_loss=attack_test_loss, test_acc=attack_test_acc))

In [ ]:


net.eval()
inputs = x_evaluate_attacker_tensor[0:50]
inputs = inputs.cuda()
inputs = torch.autograd.Variable(inputs)
outputs,h_layer = net(inputs)

In [ ]:


outputs.size(1)

In [ ]:


one_hot_tr.shape

In [ ]:


one_hot_tr = torch.from_numpy((np.zeros((inputs.size()[0],inputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
target_one_hot_tr = one_hot_tr.scatter_(1, torch.cat((check_test_trainset_label_tensor,check_test_label_tensor)).type(torch.cuda.LongTensor).view([-1,1]).data,1)

infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)

In [ ]:


predict_origin.shape

In [ ]:


batch_size = 100
attack_model.eval()

# softmax = nn.Softmax()

losses = AverageMeter()
top1 = AverageMeter()

sum_correct = 0.0

# f_evaluate_logits_tensor = torch.from_numpy(result_array_logits).type(torch.FloatTensor)

# f_evaluate=np.sort(result_array,axis=1)
f_evaluate_tensor = torch.from_numpy(result_array).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(origin_array).type(torch.FloatTensor)
# f_evaluate_tensor = torch.from_numpy(f_evaluate_origin).type(torch.FloatTensor)

# comb_targets= torch.cat((check_test_trainset_label_tensor,check_test_label_tensor)).view([-1,1]).type(torch.cuda.FloatTensor)
comb_targets = y_evaluate_attacker_tensor.cuda()
att_labels = np.zeros(f_evaluate_tensor.size()[0])

# att_labels = np.zeros(f_evaluate_prob_tensor.size()[0])
att_labels [:5000] =1.0
att_labels [5000:] =0.0
# att_labels [:2000] =1.0
# att_labels [2000:] =0.0
is_member_labels_tensor = torch.from_numpy(att_labels).type(torch.FloatTensor)
    
len_t =  (len(f_evaluate_tensor)//batch_size)
print('len_t: ', len_t)


list_attack_output = []
for ind in range(len_t):
    # measure data loading time
    inputs = f_evaluate_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = comb_targets[ind*batch_size:(ind+1)*batch_size]

    
    one_hot_tr = torch.from_numpy((np.zeros((inputs.size()[0],inputs.size(1))))).cuda().type(torch.cuda.FloatTensor)
    target_one_hot_tr = one_hot_tr.scatter_(1, targets.type(torch.cuda.LongTensor).view([-1,1]).data,1)

    infer_input_one_hot = torch.autograd.Variable(target_one_hot_tr)
    
#     attack_output = attack_model(inputs,_,infer_input_one_hot).view([-1])
    attack_output, _ = attack_model(inputs,_,infer_input_one_hot)
    attack_output = attack_output.view([-1])
#     .view([-1])
#     break
    is_member_labels = is_member_labels_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        is_member_labels = is_member_labels.cuda()

    v_is_member_labels = torch.autograd.Variable(is_member_labels)

    loss = attack_criterion(attack_output, v_is_member_labels)

    # measure accuracy and record loss
    #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
    list_attack_output.append(attack_output.data.cpu().numpy())
    prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
    losses.update(loss.data, inputs.size()[0])
    top1.update(prec1, inputs.size()[0])
#     print('attack_output: ', attack_output.shape, 'v_is_member_labels: ', v_is_member_labels.shape)
    correct = np.sum(np.equal((attack_output.data.cpu().numpy().reshape(-1) > 0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
#     print("correct: ", correct)
    sum_correct += correct
#     break
    # plot progress
    if ind%100==0:
        print  ('({batch}/{size}) | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                batch=ind + 1,
                size=len_t,
                loss=losses.avg,
                top1=top1.avg,
                ))

test_loss = losses.avg
test_acc = top1.avg
accurate_acc = sum_correct/10000.0
print ('evaluate loss: {test_loss: .4f}, evaluate accuracy: {test_acc: .4f}, accurate accuracy: {accurate_acc: .4f}, sum correct: {sum_correct: .1f}'.format(test_loss=test_loss, test_acc=test_acc, accurate_acc=accurate_acc,sum_correct=sum_correct))

In [ ]:


list_attack_output_array = np.asarray(list_attack_output)
v_is_member_labels

In [ ]:


list_attack_output_arrayx = list_attack_output_array.reshape(-1)
list_attack_output_arrayx.shape

In [ ]:


list_attack_output_array[1]

In [ ]:


list_attack_output_arrayx[0,100:200]

In [ ]:



pyplot.plot(list_attack_output_arrayx.reshape(-1))

In [ ]:


########evaluate the performance of defense's attack model on modified data#######

defense_model.eval()

# softmax = nn.Softmax()

losses = AverageMeter()
top1 = AverageMeter()

sum_correct = 0.0

# f_evaluate_logits_tensor = torch.from_numpy(result_array_logits).type(torch.FloatTensor)

f_evaluate=np.sort(result_array,axis=1)
# f_evaluate=np.sort(origin_array,axis=1)
# f_evaluate=np.sort(f_evaluate_origin,axis=1) 
f_evaluate_tensor = torch.from_numpy(f_evaluate).type(torch.FloatTensor)

att_labels = np.zeros(f_evaluate_tensor.size()[0])

# att_labels = np.zeros(f_evaluate_prob_tensor.size()[0])
att_labels [:5000] =1.0
att_labels [5000:] =0.0
is_member_labels_tensor = torch.from_numpy(att_labels).type(torch.FloatTensor)
    
len_t =  (len(f_evaluate_tensor)//batch_size)
print('len_t: ', len_t)

list_attack_output = []
for ind in range(len_t):
    # measure data loading time
    inputs = f_evaluate_tensor[ind*batch_size:(ind+1)*batch_size]
#     targets = labels[ind*batch_size:(ind+1)*batch_size]

    attack_output, attack_output_h = defense_model(inputs)
#     .view([-1])
#     break
    is_member_labels = is_member_labels_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        is_member_labels = is_member_labels.cuda()

    v_is_member_labels = torch.autograd.Variable(is_member_labels)

    loss = defense_criterion(attack_output, v_is_member_labels)

    # measure accuracy and record loss
    #prec1,p5 = accuracy(attack_output.data, v_is_member_labels.data, topk=(1,2))
    list_attack_output.append(attack_output.data.cpu().numpy())
    prec1=np.mean(np.equal((attack_output.data.cpu().numpy() >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
    losses.update(loss.data, inputs.size()[0])
    top1.update(prec1, inputs.size()[0])
#     print('attack_output: ', attack_output.shape, 'v_is_member_labels: ', v_is_member_labels.shape)
    correct = np.sum(np.equal((attack_output.data.cpu().numpy().reshape(-1) >0.5),(v_is_member_labels.data.cpu().numpy()> 0.5)))
#     print("correct: ", correct)
    sum_correct += correct
#     break
    # plot progress
    if ind%100==0:
        print  ('({batch}/{size}) | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                batch=ind + 1,
                size=len_t,
                loss=losses.avg,
                top1=top1.avg,
                ))

test_loss = losses.avg
test_acc = top1.avg
accurate_acc = sum_correct/10000.0
print ('evaluate loss: {test_loss: .4f}, evaluate accuracy: {test_acc: .4f}, accurate accuracy: {accurate_acc: .4f}, sum correct: {sum_correct: .1f}'.format(test_loss=test_loss, test_acc=test_acc, accurate_acc=accurate_acc,sum_correct=sum_correct))

# (1/157) | Loss: 0.2482 | top1:  0.6953 
# (101/157) | Loss: 0.2485 | top1:  0.6144 
# evaluate loss:  0.2496, evaluate accuracy:  0.5018, accurate accuracy:  0.5021

In [ ]:


# Get the saved location npz model res

# evaluation_noise_filepath="/home/nux219/ML_Privacy_Regularization_master/checkpoints_texas_prune_little_model_Memguard/attack/noise_data_No_defense_20_epoch_6_40000_evaluation.npz"

# npz_defense=np.load(evaluation_noise_filepath)
# f_evaluate_defense=npz_defense['defense_output']
# f_evaluate_origin=npz_defense['tc_output']

# f_evaluate_defense_logits=npz_defense['defense_output_logits']
# f_evaluate_logits_origin=npz_defense['tc_output_logits']

# f_evaluate_origin_score=npz_defense['predict_origin']
# f_evaluate_defense_score=npz_defense['predict_modified']

dir_path = './model_logits/Modify1_defense_100_epoch_78/'
# txt_path = '/media/SSD_NVMe_1T/prune_model/model_params/resnet18_v2_ori_2_50/'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
# txt_path = dir_path + 'f_evaluate_defense_logits.txt'
# np.savetxt(txt_path, f_evaluate_defense_logits, fmt='%f', delimiter=',')
# print(f_evaluate_defense_logits.shape)

txt_path = dir_path + 'f_evaluate_origin_logits.txt'
np.savetxt(txt_path, f_evaluate_logits_origin, fmt='%f', delimiter=',')
print(f_evaluate_logits_origin.shape)

# txt_path = dir_path + 'f_evaluate_defense.txt'
# np.savetxt(txt_path, f_evaluate_defense, fmt='%f', delimiter=',')
# print(f_evaluate_defense.shape)

txt_path = dir_path + 'f_evaluate_origin.txt'
np.savetxt(txt_path, f_evaluate_origin, fmt='%f', delimiter=',')
print(f_evaluate_origin.shape)

In [ ]:


txt_path = dir_path + 'predict_origin.txt'
np.savetxt(txt_path, predict_origin, fmt='%f', delimiter=',')
print(predict_origin.shape)

# txt_path = dir_path + 'predict_defense.txt'
# np.savetxt(txt_path, predict_modified, fmt='%f', delimiter=',')
# print(predict_modified.shape)

txt_path = dir_path + 'y_evaluate.txt'
np.savetxt(txt_path, y_evaluate_attacker, fmt='%f', delimiter=',')
print(y_evaluate_attacker.shape)

In [ ]:


scenario='full'
if scenario=='full':
    predict_result_origin=np.where(predict_origin > 0.5, 1, 0)
    predict_result_defense=np.where(predict_modified > 0.5, 1, 0)
else:
    raise NotImplementedError

epsilon_value_list=["1.0","0.7","0.5","0.3","0.1","0.0"]
# epsilon_value_list=["0.0"]
epsilon_value_list=[float(t) for t in epsilon_value_list]

In [ ]:


f_evaluate_noise = result_array
# f_evaluate_origin = origin_array

In [ ]:


count = 0
for i in np.arange(f_evaluate_origin.shape[0]):
    if np.equal(predict_result_origin[i] , label_evaluate_attacker[i]):
        count+=1
    
count

In [ ]:


f_evaluate_origin[i]

In [ ]:


pyplot.plot(predict_result_origin)

In [ ]:


# scenario='full'
# if scenario=='full':
#     predict_result_origin=np.where(predict_origin > 0.5, 1, 0)
#     predict_result_defense=np.where(predict_modified > 0.5, 1, 0)
# else:
#     raise NotImplementedError

# epsilon_value_list=["1.0","0.7","0.5","0.3","0.1","0.0"]
# epsilon_value_list=[float(t) for t in epsilon_value_list]

inference_accuracy_list=[]

for epsilon_value in epsilon_value_list: 
    inference_accuracy=0.0

    np.random.seed(100)  
    for i in np.arange(f_evaluate_origin.shape[0]):
        distortion_noise=np.sum(np.abs(f_evaluate_origin[i,:]-f_evaluate_noise[i,:]))
        
        p_value=0.0
        if np.abs(predict_origin[i]-0.5)<=np.abs(predict_modified[i]-0.5):
            p_value=0.0
        else:
            p_value=min(epsilon_value/distortion_noise,1.0)
        
        if predict_result_origin[i]==label_evaluate_attacker[i]:
            inference_accuracy+=1.0-p_value
        if predict_result_defense[i]==label_evaluate_attacker[i]:
            inference_accuracy+=p_value
#         print('distortion_noise: ', distortion_noise, ' p_value: ',p_value,' inference_accuracy: ',inference_accuracy)
    inference_accuracy_list.append(inference_accuracy/(float(f_evaluate_origin.shape[0])))
#     break
    
print("Budget list: {}".format(epsilon_value_list))
print("inference accuracy list: {}".format(inference_accuracy_list))

# Budget list: [1.0, 0.7, 0.5, 0.3, 0.1, 0.0]
# inference accuracy list: [0.5021911142220747, 0.5048992830892228, 0.5120250427638868, 0.5286259040624035, 0.5731544218872452, 0.60825]

In [ ]:


# strong sort noise result
# NN model
# Budget list: [1.0, 0.7, 0.5, 0.3, 0.1, 0.0]
# inference accuracy list: [0.646396630378386, 0.6569747528528351, 0.6640105377520257, 0.6710463226512157, 0.6780821075504057, 0.6816]

# NSH model
# Budget list: [1.0, 0.7, 0.5, 0.3, 0.1, 0.0]
# inference accuracy list: [0.5397111203340036, 0.5949816170842677, 0.631815299292593, 0.6689281014857144, 0.7064427004952366, 0.7252]

In [ ]:


predict_result_origin

In [ ]:


# test_sample_id = 1
# max_label=np.argmax(f_evaluate[test_sample_id,:])
# origin_value=np.copy(f_evaluate[test_sample_id,:]).reshape(1,user_label_dim)
# origin_value_logits=np.copy(f_evaluate_logits[test_sample_id,:]).reshape(1,user_label_dim)
# label_mask_array[0,:]=0.0
# label_mask_array[0,max_label]=1.0
# label_mask_tonser = torch.from_numpy(label_mask_array).type(torch.FloatTensor).cuda()
# sample_f_logits=np.copy(origin_value_logits)

In [ ]:


# sample_f_logits_tensor = torch.from_numpy(sample_f_logits).type(torch.FloatTensor).requires_grad_()
# sample_f_logits_tensor = sample_f_logits_tensor.cuda()
# sample_f_tensor = softmax(sample_f_logits_tensor)
# result_predict_scores_initial, sample_f_h =defense_model(sample_f_tensor)
# result_predict_scores_initial = result_predict_scores_initial.item()

In [ ]:


# loss1 = torch.abs(sample_f_h)
# loss1

In [ ]:


# correct_label = torch.sum(label_mask_tonser * sample_f_tensor, 1)
# wrong_label = torch.max((1-label_mask_tonser) * sample_f_tensor - 1e8*label_mask_tonser,1).values
# loss2 = F.relu(wrong_label-correct_label)
# loss2

In [ ]:


# loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor).cuda())) #L-1 norm
# loss3

In [ ]:


# loss = c1 * loss1 + c2 * loss2 + c3 * loss3
# gradient_values = torch.autograd.grad(loss, sample_f_tensor)[0]

In [ ]:


# if np.abs(result_predict_scores_initial-0.5)<=1e-5:
#         success_fraction+=1.0
#         result_array[test_sample_id,:]=origin_value[0,back_index[test_sample_id,:]]
#         result_array_logits[test_sample_id,:]=origin_value_logits[0,back_index[test_sample_id,:]]
# #         continue
# last_iteration_result=np.copy(origin_value)[0,back_index[test_sample_id,:]]
# last_iteration_result_logits=np.copy(origin_value_logits)[0,back_index[test_sample_id,:]]
# success=True
# c3=0.1
# iterate_time=1

In [ ]:


# while success==True: 
#     sample_f=np.copy(origin_value_logits)
#     j=1
#     result_max_label=-1
#     result_predict_scores=result_predict_scores_initial
# #         sample_f_tensor = torch.from_numpy(sample_f).type(torch.FloatTensor).requires_grad_()
# #         sample_f_tensor = softmax(sample_f_tensor).cuda()
#     while j<max_iteration and (max_label!=result_max_label or (result_predict_scores-0.5)*(result_predict_scores_initial-0.5)>0):
#         _, sample_f_h =defense_model(sample_f_tensor)
#         loss1 = torch.abs(sample_f_h)

#         correct_label = torch.sum(label_mask_tonser * sample_f_tensor, 1)
#         wrong_label = torch.max((1-label_mask_tonser) * sample_f_tensor - 1e8*label_mask_tonser,1).values
#         loss2 = F.relu(wrong_label-correct_label)

#         loss3= torch.sum(torch.abs(sample_f_tensor - torch.from_numpy(origin_value).type(torch.FloatTensor).cuda())) #L-1 norm
#         loss = c1 * loss1 + c2 * loss2 + c3 * loss3
#         gradient_values = torch.autograd.grad(loss, sample_f_tensor)[0]
#         gradient_valuesx = gradient_values/torch.norm(gradient_values)

#         sample_f_logits_tensor = sample_f_logits_tensor-0.1*gradient_valuesx
#         sample_f_tensor = softmax(sample_f_logits_tensor)
#         result_predict_scores, _ = defense_model(sample_f_tensor)
#         result_predict_scores = result_predict_scores.item()
#         result_max_label = torch.argmax(sample_f_tensor).item()
#         j+=1
#     if max_label!=result_max_label:
#         if iterate_time==1:
#             print("failed sample for label not same for id: {},c3:{} not add noise".format(test_sample_id,c3))
#             success_fraction-=1.0
#         break 
#     result_predict_scores, _ = defense_model(sample_f_tensor)
#     if ((result_predict_scores.item()-0.5)*(result_predict_scores_initial-0.5))>0:
#         if iterate_time==1:
#             print("max iteration reached with id: {}, max score: {}, prediction_score: {}, c3: {}, not add noise".format(test_sample_id,np.amax(softmax(sample_f)),result_predict_scores,c3))
#         break
#     last_iteration_result[:]=sample_f_tensor.data.cpu().numpy()[0,back_index[test_sample_id,:]]
#     last_iteration_result_logits[:]=sample_f_logits_tensor.data.cpu().numpy()[0,back_index[test_sample_id,:]]
#     iterate_time+=1 
#     c3=c3*10
#     if c3>100000:
#         break

In [ ]:


last_iteration_result_logits

In [ ]:


# test_data = X[int((train_classifier_ratio+train_attack_ratio)*len_train):]
# test_label = Y[int((train_classifier_ratio+train_attack_ratio)*len_train):]
# test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
# test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)

# print('test_data_tensor: ',len(test_data_tensor))

In [ ]:


model = Texas()
model = torch.nn.DataParallel(model).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
batch_size = 128

In [ ]:


train_classifier_data_tensor = torch.from_numpy(train_classifier_data).type(torch.FloatTensor)
train_classifier_label_tensor = torch.from_numpy(train_classifier_label).type(torch.LongTensor)

test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)

print('train_classifier_data_tensor: ',len(train_classifier_data_tensor))
print('test_data_tensor: ',len(test_data_tensor))

In [ ]:


resume='./checkpoints_texas_prune_little_model_test/No_defense_20_epoch_6'
print('==> Resuming from checkpoint..')
assert os.path.isfile(resume), 'Error: no checkpoint directory found!'
checkpoint = os.path.dirname(resume)
checkpoint = torch.load(resume)
model.load_state_dict(checkpoint['state_dict'])
epoch = 0
test_loss, final_test_acc = test(torch.from_numpy(test_data).type(torch.FloatTensor) ,torch.from_numpy(test_label).type(torch.LongTensor), model, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

trainset_loss, trainset_acc = test(torch.from_numpy(train_classifier_data).type(torch.FloatTensor) ,torch.from_numpy(train_classifier_label).type(torch.LongTensor), model, criterion, epoch, use_cuda)

print ('trainset_loss: {trainset_loss: .4f}, trainset_acc: {trainset_acc: .4f}'.format(trainset_loss=trainset_loss, trainset_acc=trainset_acc))

In [ ]:


sort_train_label_index = np.argsort(train_classifier_label)
sort_train_label = train_classifier_label[sort_train_label_index]

sort_train_data = train_classifier_data[sort_train_label_index]

sort_train_label_bin = np.bincount(sort_train_label)

In [ ]:


sort_test_label_index = np.argsort(test_label)
sort_test_label = test_label[sort_test_label_index]

sort_test_data = test_data[sort_test_label_index]

sort_test_label_bin = np.bincount(sort_test_label)

In [ ]:


sort_test_data.shape

In [ ]:


sort_train_data_tensor = torch.from_numpy(sort_train_data).type(torch.FloatTensor)
sort_train_label_tensor = torch.from_numpy(sort_train_label).type(torch.LongTensor)

sort_test_data_tensor = torch.from_numpy(sort_test_data).type(torch.FloatTensor)
sort_test_label_tensor = torch.from_numpy(sort_test_label).type(torch.LongTensor)

print('train_classifier_data_tensor: ',len(sort_train_data_tensor))
print('test_data_tensor: ',len(sort_test_data_tensor))

In [ ]:


trainset_loss, trainset_acc = test(sort_train_data_tensor ,sort_train_label_tensor, model, criterion, epoch, use_cuda)

test_loss, final_test_acc = test(sort_test_data_tensor ,sort_test_label_tensor, model, criterion, epoch, use_cuda)

print ('trainset_loss: {trainset_loss: .4f}, trainset_acc: {trainset_acc: .4f}'.format(trainset_loss=trainset_loss, trainset_acc=trainset_acc))

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

In [ ]:


# load-attack model 

attack_model = InferenceAttack_HZ(100)
attack_model = torch.nn.DataParallel(attack_model).cuda()
attack_criterion = nn.MSELoss()
attack_optimizer = optim.Adam(attack_model.parameters(),lr=0.0001)

In [ ]:


resume_att='./checkpoints_texas_prune_little_model_test/No_defense_20_epoch_6_longtrain_attack_epoch50'
print('==> Resuming attack model from checkpoint..')
assert os.path.isfile(resume_att), 'Error: no checkpoint directory found!'
checkpoint_att = os.path.dirname(resume_att)
checkpoint_att = torch.load(resume_att)
attack_model.load_state_dict(checkpoint_att['state_dict'])

In [ ]:


# Epoch: [49 | 100] , lr : 0.000100
# (1/106) Data: 0.000s | Batch: 0.045s | | Loss: 0.2478 | top1:  0.5195 
# (101/106) Data: 0.000s | Batch: 0.054s | | Loss: 0.2374 | top1:  0.5755 
# train acc 0.5738897965245804
# (1/106) Data: 0.000s | Batch: 0.033s | | Loss: 0.2637 | top1:  0.5312 
# (101/106) Data: 0.000s | Batch: 0.031s | | Loss: 0.2585 | top1:  0.5075 
# test acc 0.5080573295707709 0.5080573295707709

In [ ]:


batch_size = 128

r1 = np.arange(len(train_classifier_data_tr_attack))
np.random.shuffle(r1)

check_train_trainset_data_tensor = torch.from_numpy(train_classifier_data_tr_attack[r1]).type(torch.FloatTensor)
check_train_trainset_label_tensor = torch.from_numpy(train_classifier_label_tr_attack[r1]).type(torch.LongTensor)

r2 = np.arange(len(train_attack_data))
np.random.shuffle(r2)
check_train_attack_data_tensor = torch.from_numpy(train_attack_data[r2]).type(torch.FloatTensor)
check_train_attack_label_tensor = torch.from_numpy(train_attack_label[r2]).type(torch.LongTensor)

r3 = np.arange(len(train_classifier_data_te_attack))
np.random.shuffle(r3)

check_test_trainset_data_tensor = torch.from_numpy(train_classifier_data_te_attack[r3]).type(torch.FloatTensor)
check_test_trainset_label_tensor = torch.from_numpy(train_classifier_label_te_attack[r3]).type(torch.LongTensor)

r4 = np.arange(len(test_data))
np.random.shuffle(r4)

check_test_data_tensor = torch.from_numpy(test_data[r4]).type(torch.FloatTensor)
check_test_label_tensor = torch.from_numpy(test_label[r4]).type(torch.LongTensor)

print(len(check_train_trainset_data_tensor))
print(len(check_train_attack_data_tensor))
print(len(check_test_trainset_data_tensor))
print(len(check_test_data_tensor))

In [ ]:


# check train accuracy and test accuracy of attack model with correspond data

attack_train_loss, attack_train_acc, attack_train_correct_sum = test_attack(check_train_trainset_data_tensor,check_train_trainset_label_tensor
                                         ,check_train_attack_data_tensor,check_train_attack_label_tensor,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

print ('attack_train_loss: {test_loss: .4f}, attack_train_acc: {test_acc: .4f}'.format(test_loss=attack_train_loss, test_acc=attack_train_acc))

attack_test_loss, attack_test_acc, attack_test_correct_sum = test_attack(check_test_trainset_data_tensor,check_test_trainset_label_tensor
                                         ,check_test_data_tensor,check_test_label_tensor,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

print ('attack_test_loss: {test_loss: .4f}, attack_test_acc: {test_acc: .4f}'.format(test_loss=attack_test_loss, test_acc=attack_test_acc))

In [ ]:


batch_size = 32 
attack_test_loss, attack_test_acc, correct_sum = test_attack(check_test_trainset_data_tensor,check_test_trainset_label_tensor
                                         ,check_test_data_tensor,check_test_label_tensor,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

print ('attack_test_loss: {test_loss: .4f}, attack_test_acc: {test_acc: .4f}'.format(test_loss=attack_test_loss, test_acc=attack_test_acc))
print('attack_test_acc: ', correct_sum / (len(check_test_trainset_label_tensor)*2))

In [ ]:


attack_train_correct_sum 

In [ ]:


attack_train_correct_sum/len(check_train_trainset_data_tensor)/2

In [ ]:


len(check_test_data_tensor)

In [ ]:


# trainset attack accuracy for trained set
batch_size = 32 
attack_trainset_loss, attack_trainset_acc, trainset_correct = attack_test_trainset(check_train_trainset_data_tensor,check_train_trainset_label_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)
# attack_trainset_loss = sum_correct / len(check_train_data_tensor)
print ('attack_trainset_loss: {test_loss: .4f}, attack_trainset_acc: {test_acc: .4f}'.format(test_loss=attack_trainset_loss, test_acc=attack_trainset_acc))
print('attack_trainset_acc: ', trainset_correct / len(check_train_trainset_data_tensor))

In [ ]:


# trainset attack accuracy for trainset not trained in attack model
batch_size = 32 
attack_trainset_loss, attack_trainset_acc, trainset_correct = attack_test_trainset(check_test_trainset_data_tensor,check_test_trainset_label_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)
# attack_trainset_loss = sum_correct / len(check_train_data_tensor)
print ('attack_trainset_loss: {test_loss: .4f}, attack_trainset_acc: {test_acc: .4f}'.format(test_loss=attack_trainset_loss, test_acc=attack_trainset_acc))
print('attack_testset_acc: ', trainset_correct / len(check_test_trainset_data_tensor))

In [ ]:


trainset_correct

In [ ]:


#attack accuracy for dataset not in trainset but trained in attack model
batch_size = 32
attack_testset_loss, attack_testset_acc, testset_correct = attack_test_testset(check_train_attack_data_tensor,check_train_attack_label_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)
# attack_trainset_loss = sum_correct / len(check_train_data_tensor)
print ('attack_testset_loss: {test_loss: .4f}, attack_testset_acc: {test_acc: .4f}'.format(test_loss=attack_testset_loss, test_acc=attack_testset_acc))
print('testset_correct_acc: ', testset_correct / len(check_train_attack_label_tensor))

In [ ]:


#attack accuracy for testset not trained in attack model
batch_size = 32
attack_testset_loss, attack_testset_acc, testset_correct = attack_test_testset(check_test_data_tensor,check_test_label_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)
# attack_trainset_loss = sum_correct / len(check_train_data_tensor)
print ('attack_testset_loss: {test_loss: .4f}, attack_testset_acc: {test_acc: .4f}'.format(test_loss=attack_testset_loss, test_acc=attack_testset_acc))
print('testset_correct_acc: ', testset_correct / len(check_test_data_tensor))

In [ ]:


testset_correct

In [ ]:


len(check_test_label_tensor) + len(check_train_attack_label_tensor)

In [ ]:


# train_classifier_label_tr_attack
# train_attack_label
# train_classifier_label_te_attack
# test_label

In [ ]:


sort_train_label_tr_attack_index = np.argsort(train_classifier_label_tr_attack)
sort_train_label_tr_attack = train_classifier_label_tr_attack[sort_train_label_tr_attack_index]

sort_train_data_tr_attack = train_classifier_data_tr_attack[sort_train_label_tr_attack_index]

sort_train_label_tr_attack_bin = np.bincount(sort_train_label_tr_attack)
# sort_train_label_tr_attack_bin

In [ ]:


sort_train_label_tr_attack_bin_list = []
for i in range(len(sort_train_label_tr_attack_bin)):
    sort_train_label_tr_attack_bin_list.append(sort_train_label_tr_attack_bin[i])
print(sort_train_label_tr_attack_bin_list)

# [73, 61, 203, 70, 83, 78, 44, 39, 53, 61, 92, 132, 84, 43, 485, 53, 114, 61, 162, 109, 69, 317, 94, 48, 178, 64, 54, 78, 92, 74, 466, 88, 132, 93, 302, 70, 193, 116, 98, 107, 83, 76, 106, 106, 50, 242, 54, 434, 38, 180, 184, 264, 53, 57, 58, 54, 59, 245, 135, 186, 61, 87, 61, 58, 78, 518, 247, 52, 97, 52, 58, 85, 529, 55, 68, 133, 60, 56, 184, 67, 61, 210, 69, 68, 103, 52, 62, 68, 69, 57, 88, 254, 103, 351, 493, 149, 49, 441, 563, 53]

In [ ]:


sort_train_label_te_attack_index = np.argsort(train_classifier_label_te_attack)
sort_train_label_te_attack = train_classifier_label_te_attack[sort_train_label_te_attack_index]

sort_train_data_te_attack = train_classifier_data_te_attack[sort_train_label_te_attack_index]

sort_train_label_te_attack_bin = np.bincount(sort_train_label_te_attack)
# sort_train_label_te_attack_bin

In [ ]:


sort_train_label_te_attack_bin_list = []
for i in range(len(sort_train_label_te_attack_bin)):
    sort_train_label_te_attack_bin_list.append(sort_train_label_te_attack_bin[i])
print(sort_train_label_te_attack_bin_list)

# [71, 64, 164, 63, 83, 81, 65, 57, 46, 66, 81, 149, 102, 51, 490, 50, 105, 44, 172, 115, 56, 320, 88, 43, 197, 45, 49, 81, 86, 71, 470, 47, 121, 100, 307, 73, 183, 125, 58, 94, 73, 102, 100, 114, 48, 221, 60, 447, 55, 194, 181, 269, 61, 65, 52, 68, 70, 275, 136, 151, 59, 94, 60, 47, 97, 573, 217, 56, 99, 53, 70, 49, 574, 39, 69, 142, 78, 41, 174, 61, 66, 214, 64, 71, 98, 55, 62, 75, 53, 74, 83, 276, 113, 296, 448, 158, 55, 417, 606, 55]

In [ ]:


sort_train_attack_label_index = np.argsort(train_attack_label)
sort_train_attack_label = train_attack_label[sort_train_attack_label_index]

sort_train_attack_data = train_attack_data[sort_train_attack_label_index]

sort_train_attack_label_bin = np.bincount(sort_train_attack_label)
# sort_train_attack_label_bin

In [ ]:


sort_train_attack_label_bin_list = []
for i in range(len(sort_train_attack_label_bin)):
    sort_train_attack_label_bin_list.append(sort_train_attack_label_bin[i])
print(sort_train_attack_label_bin_list)

# [102, 107, 261, 108, 146, 127, 84, 80, 92, 97, 135, 174, 147, 69, 790, 93, 149, 72, 233, 162, 91, 521, 121, 80, 303, 111, 93, 113, 124, 96, 702, 121, 174, 137, 437, 104, 293, 160, 114, 175, 125, 130, 143, 160, 80, 376, 87, 643, 81, 274, 328, 427, 71, 87, 81, 84, 97, 361, 210, 246, 78, 106, 81, 75, 128, 778, 342, 87, 150, 70, 95, 87, 790, 65, 86, 187, 121, 88, 278, 81, 108, 335, 119, 108, 121, 77, 74, 91, 74, 98, 128, 371, 182, 458, 713, 218, 85, 613, 1003, 61]

In [ ]:


sort_test_label_index = np.argsort(test_label)
sort_test_label = test_label[sort_test_label_index]

sort_test_data = test_data[sort_test_label_index]

sort_test_label_bin = np.bincount(sort_test_label)
# sort_test_label_bin

In [ ]:


sort_test_label_bin_list = []
for i in range(len(sort_test_label_bin)):
    sort_test_label_bin_list.append(sort_test_label_bin[i])
print(sort_test_label_bin_list)

# [95, 97, 248, 99, 122, 95, 75, 87, 71, 94, 151, 163, 172, 73, 723, 60, 157, 67, 278, 160, 118, 478, 140, 70, 310, 89, 114, 104, 126, 101, 679, 92, 187, 139, 462, 95, 320, 150, 115, 183, 133, 144, 161, 172, 95, 404, 103, 668, 84, 243, 320, 376, 77, 94, 80, 103, 87, 320, 223, 270, 93, 136, 97, 64, 145, 817, 345, 100, 170, 87, 121, 95, 794, 78, 90, 206, 111, 72, 261, 81, 77, 313, 105, 120, 110, 90, 101, 106, 80, 90, 122, 398, 183, 494, 691, 201, 81, 609, 874, 75]

In [ ]:


# trainset and testset NOT used in attack training

print("sort_train_label_te_attack_bin: ", sort_train_label_te_attack_bin_list)
print("sort_test_label_bin_list: ", sort_test_label_bin_list)

In [ ]:


# attack accuracy for each class for trainset and testset NOT used in attack training

batch_size = 32

len_class =  len(sort_train_label_te_attack_bin)
sort_train_data_te_attack_acc = []
sort_testset_class_attack_acc = []

sort_train_data_te_attack_loss = []
sort_testset_class_attack_loss = []

sort_train_data_te_attack_correct = []
sort_testset_class_attack_correct = []

print('len_class: ', len_class)

index_trainset = 0
index_testset = 0

for ind in range(len_class):

    class_train_data_te_tensor = torch.from_numpy(sort_train_data_te_attack[index_trainset : (index_trainset+sort_train_label_te_attack_bin[ind])]).type(torch.FloatTensor)
    class_train_label_te_tensor = torch.from_numpy(sort_train_label_te_attack[index_trainset:(index_trainset+sort_train_label_te_attack_bin[ind])]).type(torch.LongTensor)

    class_test_data_tensor = torch.from_numpy(sort_test_data[index_testset : (index_testset+sort_test_label_bin[ind])]).type(torch.FloatTensor)
    class_test_label_tensor = torch.from_numpy(sort_test_label[index_testset:(index_testset+sort_test_label_bin[ind])]).type(torch.LongTensor)
    
    index_trainset += sort_train_label_te_attack_bin[ind]
    index_testset += sort_test_label_bin[ind]
    
    print('index_trainset: ', index_trainset)
    print('index_testset: ', index_testset)
    
    attack_trainset_loss, attack_trainset_acc, trainset_correct = attack_test_trainset(class_train_data_te_tensor,class_train_label_te_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    attack_testset_loss, attack_testset_acc, testset_correct = attack_test_testset(class_test_data_tensor,class_test_label_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)
    
    sort_train_data_te_attack_acc.append(attack_trainset_acc)
    sort_testset_class_attack_acc.append(attack_testset_acc)

    sort_train_data_te_attack_loss.append(attack_trainset_loss)
    sort_testset_class_attack_loss.append(attack_testset_loss)

    sort_train_data_te_attack_correct.append(trainset_correct)
    sort_testset_class_attack_correct.append(testset_correct)
    
total_trainset_correct = sum(sort_train_data_te_attack_correct)
total_testset_correct = sum(sort_testset_class_attack_correct)

print('total_trainset_correct: ', total_trainset_correct)
print('total_testset_correct: ', total_testset_correct)

print('sort_trainset_class_attack_correct: ', sort_train_data_te_attack_correct)
print('sort_testset_class_attack_correct: ', sort_testset_class_attack_correct)

In [ ]:


# attack accuracy on trainset NOT used in attack training

sort_acc_traindata_te_attack_class = sort_train_data_te_attack_correct/sort_train_label_te_attack_bin

sort_acc_traindata_te_attack_class_list = []
for i in range(len(sort_acc_traindata_te_attack_class)):
    sort_acc_traindata_te_attack_class_list.append(sort_acc_traindata_te_attack_class[i])

print(sort_acc_traindata_te_attack_class_list)

In [ ]:


# attack accuracy on testset NOT used in attack training

sort_acc_test_te_class_attack_class = sort_testset_class_attack_correct/sort_test_label_bin

sort_acc_test_te_class_attack_class_list = []
for i in range(len(sort_acc_test_te_class_attack_class)):
    sort_acc_test_te_class_attack_class_list.append(sort_acc_test_te_class_attack_class[i])

print(sort_acc_test_te_class_attack_class_list)

In [ ]:


print("sort_train_tr_attack_label_bin: ", sort_train_label_tr_attack_bin_list)
print("sort_test_tr_attack_label_bin: ", sort_train_attack_label_bin_list)

In [ ]:


# attack accuracy for each class for trainset and testset trained for the attack

batch_size = 32

len_class =  len(sort_train_label_tr_attack_bin)
sort_train_data_tr_attack_acc = []
sort_test_tr_attack_acc = []

sort_train_data_tr_attack_loss = []
sort_test_tr_attack_loss = []

sort_train_data_tr_attack_correct = []
sort_test_tr_attack_correct = []

print('len_class: ', len_class)

index_trainset = 0
index_testset = 0

for ind in range(len_class):

    class_train_data_tr_tensor = torch.from_numpy(sort_train_data_tr_attack[index_trainset : (index_trainset+sort_train_label_tr_attack_bin[ind])]).type(torch.FloatTensor)
    class_train_label_tr_tensor = torch.from_numpy(sort_train_label_tr_attack[index_trainset:(index_trainset+sort_train_label_tr_attack_bin[ind])]).type(torch.LongTensor)

    class_test_data_tr_tensor = torch.from_numpy(sort_train_attack_data[index_testset : (index_testset+sort_train_attack_label_bin[ind])]).type(torch.FloatTensor)
    class_test_label_tr_tensor = torch.from_numpy(sort_train_attack_label[index_testset:(index_testset+sort_train_attack_label_bin[ind])]).type(torch.LongTensor)
    
    index_trainset += sort_train_label_tr_attack_bin[ind]
    index_testset += sort_train_attack_label_bin[ind]
    
    print('index_trainset: ', index_trainset)
    print('index_testset: ', index_testset)
    
    attack_trainset_loss, attack_trainset_acc, trainset_correct = attack_test_trainset(class_train_data_tr_tensor,class_train_label_tr_tensor, model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    attack_testset_loss, attack_testset_acc, testset_correct = attack_test_testset(class_test_data_tr_tensor,class_test_label_tr_tensor, model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    
    sort_train_data_tr_attack_acc.append(attack_trainset_acc)
    sort_test_tr_attack_acc.append(attack_testset_acc)

    sort_train_data_tr_attack_loss.append(attack_trainset_loss)
    sort_test_tr_attack_loss.append(attack_testset_loss)

    sort_train_data_tr_attack_correct.append(trainset_correct)
    sort_test_tr_attack_correct.append(testset_correct)
    
total_trainset_tr_correct = sum(sort_train_data_tr_attack_correct)
total_testset_tr_correct = sum(sort_test_tr_attack_correct)

print('total_trainset_tr_correct: ', total_trainset_tr_correct)
print('total_testset_tr_correct: ', total_testset_tr_correct)

print('sort_trainset_class_tr_attack_correct: ', sort_train_data_tr_attack_correct)
print('sort_testset_class_tr_attack_correct: ', sort_test_tr_attack_correct)

In [ ]:


# attack accuracy on trainset used in attack training
sort_acc_traindata_tr_attack_class = sort_train_data_tr_attack_correct/sort_train_label_tr_attack_bin

sort_acc_traindata_tr_attack_class_list = []
for i in range(len(sort_acc_traindata_tr_attack_class)):
    sort_acc_traindata_tr_attack_class_list.append(sort_acc_traindata_tr_attack_class[i])

print(sort_acc_traindata_tr_attack_class_list)

In [ ]:


# attack accuracy on testset used in attack training
sort_acc_test_tr_class_attack_class = sort_test_tr_attack_correct/sort_train_attack_label_bin

sort_acc_test_tr_class_attack_class_list = []
for i in range(len(sort_acc_test_tr_class_attack_class)):
    sort_acc_test_tr_class_attack_class_list.append(sort_acc_test_tr_class_attack_class[i])

print(sort_acc_test_tr_class_attack_class_list)

In [ ]:


# attack accuracy for each class for all trainset and testset

batch_size = 32

len_class =  len(sort_train_label_bin)
sort_trainset_class_attack_acc = []
sort_testset_class_attack_acc = []

sort_trainset_class_attack_loss = []
sort_testset_class_attack_loss = []

sort_trainset_class_attack_correct = []
sort_testset_class_attack_correct = []

print('len_class: ', len_class)

index_trainset = 0
index_testset = 0

for ind in range(len_class):

    class_train_data_tensor = torch.from_numpy(sort_train_data[index_trainset : (index_trainset+sort_train_label_bin[ind])]).type(torch.FloatTensor)
    class_train_label_tensor = torch.from_numpy(sort_train_label[index_trainset:(index_trainset+sort_train_label_bin[ind])]).type(torch.LongTensor)

    class_test_data_tensor = torch.from_numpy(sort_test_data[index_testset : (index_testset+sort_test_label_bin[ind])]).type(torch.FloatTensor)
    class_test_label_tensor = torch.from_numpy(sort_test_label[index_testset:(index_testset+sort_test_label_bin[ind])]).type(torch.LongTensor)
    
    index_trainset += sort_train_label_bin[ind]
    index_testset += sort_test_label_bin[ind]
    
    print('index_trainset: ', index_trainset)
    print('index_testset: ', index_testset)
    
    attack_trainset_loss, attack_trainset_acc, trainset_correct = attack_test_trainset(class_train_data_tensor,class_train_label_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    attack_testset_loss, attack_testset_acc, testset_correct = attack_test_testset(class_test_data_tensor,class_test_label_tensor
                                         ,model,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)
    
    sort_trainset_class_attack_acc.append(attack_trainset_acc)
    sort_testset_class_attack_acc.append(attack_testset_acc)

    sort_trainset_class_attack_loss.append(attack_trainset_loss)
    sort_testset_class_attack_loss.append(attack_testset_loss)

    sort_trainset_class_attack_correct.append(trainset_correct)
    sort_testset_class_attack_correct.append(testset_correct)
    
total_trainset_correct = sum(sort_trainset_class_attack_correct)
total_testset_correct = sum(sort_testset_class_attack_correct)

print('total_trainset_correct: ', total_trainset_correct)
print('total_testset_correct: ', total_testset_correct)

print('sort_trainset_class_attack_correct: ', sort_trainset_class_attack_correct)
print('sort_testset_class_attack_correct: ', sort_testset_class_attack_correct)

In [ ]:


sort_acc_train_class = sort_trainset_class_attack_correct/sort_train_label_bin

In [ ]:


# sort_acc_train_class
sort_acc_train_class_list = []
for i in range(len(sort_acc_train_class)):
    sort_acc_train_class_list.append(sort_acc_train_class[i])

print(sort_acc_train_class_list)

In [ ]:


sort_acc_test_class = sort_testset_class_attack_correct/sort_test_label_bin

In [ ]:


# sort_acc_test_class
sort_acc_test_class_list = []
for item in sort_acc_test_class:
    sort_acc_test_class_list.append(item)
    
print(sort_acc_test_class_list)

In [ ]:


sort_class_attack_correct = sort_trainset_class_attack_correct + sort_testset_class_attack_correct
print(sum(sort_class_attack_correct))

In [ ]:


from math import log

# train data logits, prob, out

batch_size = 128
model.eval()
sort_train_total_out = []
softmax = nn.Softmax()
len_t =  (len(sort_train_data_tensor)//batch_size)+1
sort_train_logits = []
sort_train_prob = []
print('len_t: ', len_t)
for ind in range(len_t):
    # measure data loading time
    inputs = sort_train_data_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = sort_train_label_tensor[ind*batch_size:(ind+1)*batch_size]
#     print((ind+1)*batch_size)
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

    # compute output
    outputs,_ = model(inputs)
    outputs_value = outputs.data.cpu().numpy()
#     for i in range(len(outputs_value)):
#         sort_train_logits.append(outputs_value[i])
#     print('outputs.shape: ',outputs_value.shape)
    loss = criterion(outputs, targets)
    prob_outputs = softmax(outputs)
    prob_outputs_value = prob_outputs.data.cpu().numpy()
#     for i in range(len(prob_outputs_value)):
#         sort_train_prob.append(prob_outputs_value[i])
#     print('prob_outputs_value: ', prob_outputs_value.shape)
#     print(prob_outputs_value.shape)
    for i in range(len(prob_outputs_value)):
        sort_train_logits.append(outputs_value[i])
        sort_train_prob.append(prob_outputs_value[i])
        score = prob_outputs_value[i]
        sum_s = 0.
#         print('score.shape: ', score.shape)
        for s in score:
            sum_s = sum_s + s*log(s)
#         print(sum_s)
        ne = -1/log(100) * sum_s
#         print(ne)
        sort_train_total_out.append(ne)
#     break

In [ ]:


train_count = np.histogram(sort_train_total_out, bins=20)
np.set_printoptions(suppress=True)
print('train_count: ',train_count[0])
print('train_count_bins: ' ,train_count[1])

In [ ]:


train_total_out_sort = np.sort(sort_train_total_out)
train_count_center = []
count = 0
train_count_index = train_count[0]
for i in range(len(train_count_index)):
    if train_count_index[i] == 1:
        train_count_center.append(train_total_out_sort[count])
    else:
        train_count_center.append(np.average(train_total_out_sort[count:(count+train_count_index[i]-1)]))
    print(count, count+train_count_index[i]-1)
    count += train_count_index[i]

print(train_count_center)

In [ ]:


len(sort_train_logits)
sort_train_logits_array = np.asarray(sort_train_logits)
sort_train_logits_array.shape

In [ ]:


sort_train_prob_array = np.asarray(sort_train_prob)
sort_train_prob_array.shape

In [ ]:


max(sort_train_logits_array[0])

In [ ]:


print(len(sort_train_total_out))

In [ ]:


sort_train_label_bin = np.bincount(sort_train_label)
print(sort_train_label_bin)

In [ ]:


sort_train_label_bin_list = []
for i in range(len(sort_train_label_bin)):
    sort_train_label_bin_list.append(sort_train_label_bin[i])
print(sort_train_label_bin_list)

In [ ]:


index = 0
dir_path = './model_logits/No_defense_20_epoch_2_prune_99_retrain_epoch_99/'
# txt_path = '/media/SSD_NVMe_1T/prune_model/model_params/resnet18_v2_ori_2_50/'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
txt_path = dir_path + 'sort_train_logits_array/'
if not os.path.exists(txt_path):
    os.mkdir(txt_path)
for i in range(len(sort_train_label_bin)): #len(sort_train_label_bin)
    print(index)
    temp_array = []
    temp_range = sort_train_label_bin[i]
    temp_array = sort_train_logits_array[index:index+temp_range]
    index += temp_range
    np.savetxt(txt_path+'class_'+str(i)+'.txt', temp_array, fmt='%f', delimiter=',')
    print(temp_array.shape)

In [ ]:


index = 0
dir_path = './model_logits/No_defense_20_epoch_2_prune_99_retrain_epoch_99/'
# txt_path = '/media/SSD_NVMe_1T/prune_model/model_params/resnet18_v2_ori_2_50/'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
txt_path = dir_path + 'sort_train_prob_array/'
if not os.path.exists(txt_path):
    os.mkdir(txt_path)
for i in range(len(sort_train_label_bin)): #len(sort_train_label_bin)
    print(index)
    temp_array = []
    temp_range = sort_train_label_bin[i]
    temp_array = sort_train_prob_array[index:index+temp_range]
    index += temp_range
    np.savetxt(txt_path+'class_'+str(i)+'.txt', temp_array, fmt='%f', delimiter=',')
    print(temp_array.shape)

In [ ]:


batch_size = 128
model.eval()

sort_train_acc = []
sort_train_loss = []
sort_correct = []

index = 0
losses = AverageMeter()
top1 = AverageMeter()
top5 = AverageMeter()

for ind in range(len(sort_train_label_bin)):
    # measure data loading time
    inputs = sort_train_data_tensor[index:(index+sort_train_label_bin[ind])]
    targets = sort_train_label_tensor[index:(index+sort_train_label_bin[ind])]
    index += sort_train_label_bin[ind]
#     print('inputs.shape: ', inputs.shape)
#     print('targets.shape: ', targets.shape)
#     print('index: ', index)
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

    # compute output
    outputs,_ = model(inputs)
    
    loss = criterion(outputs, targets)
    
    # measure accuracy and record loss
    prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
    
    output = outputs.data
    target = targets.data
    _, pred = output.topk(1, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred)) 
    correct_number = correct[0].view(-1).float().sum(0).item()
    sort_correct.append(correct_number)
    
    sort_train_acc.append(prec1.item())
    sort_train_loss.append(loss.item())
    
    losses.update(loss.data, inputs.size()[0])
    top1.update(prec1, inputs.size()[0])
    top5.update(prec5, inputs.size()[0])
#     print('prec1: ', prec1)
#     print('loss: ', loss)
#     break

print('sort_train_acc: ', sort_train_acc)
print('sort_train_loss: ', sort_train_loss)
print('sort_correct: ', sort_correct)

In [ ]:


sum(sort_correct)/len(sort_train_total_out)

In [ ]:


output = outputs.data
target = targets.data
batch_size = target.size(0)
_, pred = output.topk(5, 1, True, True)
pred = pred.t()
correct = pred.eq(target.view(1, -1).expand_as(pred)) 

In [ ]:


correct.shape

In [ ]:


correct[0].view(-1).float().sum(0).item()

In [ ]:


sort_train_label[143]

In [ ]:


len(sort_train_label_bin)

In [ ]:


np.std(sort_train_total_out)

In [ ]:


len(sort_train_total_out)

In [ ]:


batch_size

In [ ]:


len(sort_test_label_tensor)

In [ ]:


# test data logits, prob, out
batch_size = 128

model.eval()
sort_test_total_out = []
sort_test_logits = []
sort_test_prob = []
softmax = nn.Softmax()
len_t =  (len(sort_test_label_tensor)//batch_size)+1
print(len_t)
for ind in range(len_t):
    # measure data loading time
    inputs = sort_test_data_tensor[ind*batch_size:(ind+1)*batch_size]
    targets = sort_test_label_tensor[ind*batch_size:(ind+1)*batch_size]
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
    
    # compute output
    outputs,_ = model(inputs)
    outputs_value = outputs.data.cpu().numpy()
#     loss = criterion(outputs, targets)
    prob_outputs = softmax(outputs)
    prob_outputs_value = prob_outputs.data.cpu().numpy()
    for i in range(len(prob_outputs_value)):
        sort_test_logits.append(outputs_value[i])
        sort_test_prob.append(prob_outputs_value[i])
        score = prob_outputs_value[i]
        sum_s = 0.
#         print('score.shape: ', score.shape)
        for s in score:
            sum_s = sum_s + s*log(s)
#         print(sum_s)
        ne = -1/log(100) * sum_s
#         print(ne)
        sort_test_total_out.append(ne)
#     break

In [ ]:


test_count = np.histogram(sort_test_total_out, bins=20)
np.set_printoptions(suppress=True)
print('test_count: ',test_count[0])
print('test_count_center: ' ,test_count[1])

In [ ]:


test_total_out_sort = np.sort(sort_test_total_out)
test_count_center = []
count = 0
test_count_index = test_count[0]
for i in range(len(test_count_index)):
    if test_count_index[i] == 1:
        test_count_center.append(test_total_out_sort[count])
    else:
        test_count_center.append(np.average(test_total_out_sort[count:(count+test_count_index[i]-1)]))
    print(count, count+test_count_index[i]-1)
    count += test_count_index[i]

print(test_count_center)

In [ ]:


sort_test_logits_array = np.asarray(sort_test_logits)
sort_test_logits_array.shape

In [ ]:


sort_test_prob_array = np.asarray(sort_test_prob)
sort_test_prob_array.shape

In [ ]:


max(sort_test_logits_array[0])

In [ ]:


sort_test_label_bin = np.bincount(sort_test_label)
print(sort_test_label_bin)

In [ ]:


sum(sort_test_label_bin)

In [ ]:


sort_test_label_bin_list = []
for i in range(len(sort_test_label_bin)):
    sort_test_label_bin_list.append(sort_test_label_bin[i])
print(sort_test_label_bin_list)

In [ ]:


sort_test_label[95:192]

In [ ]:


index = 0
dir_path = './model_logits/No_defense_20_epoch_2_prune_99_retrain_epoch_99/'
# txt_path = '/media/SSD_NVMe_1T/prune_model/model_params/resnet18_v2_ori_2_50/'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
txt_path = dir_path + 'sort_test_logits_array/'
if not os.path.exists(txt_path):
    os.mkdir(txt_path)
for i in range(len(sort_test_label_bin)): #len(sort_train_label_bin)
    print(index)
    temp_array = []
    temp_range = sort_test_label_bin[i]
    temp_array = sort_test_logits_array[index:index+temp_range]
    index += temp_range
    np.savetxt(txt_path+'class_'+str(i)+'.txt', temp_array, fmt='%f', delimiter=',')
    print(temp_array.shape)

In [ ]:


index = 0
dir_path = './model_logits/No_defense_20_epoch_2_prune_99_retrain_epoch_99/'
# txt_path = '/media/SSD_NVMe_1T/prune_model/model_params/resnet18_v2_ori_2_50/'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
txt_path = dir_path + 'sort_test_prob_array/'
if not os.path.exists(txt_path):
    os.mkdir(txt_path)
for i in range(len(sort_test_label_bin)): #len(sort_train_label_bin)
    print(index)
    temp_array = []
    temp_range = sort_test_label_bin[i]
    temp_array = sort_test_prob_array[index:index+temp_range]
    index += temp_range
    np.savetxt(txt_path+'class_'+str(i)+'.txt', temp_array, fmt='%f', delimiter=',')
    print(temp_array.shape)

In [ ]:


batch_size = 128
model.eval()

sort_test_acc = []
sort_test_loss = []
sort_test_correct = []

index = 0
losses = AverageMeter()
top1 = AverageMeter()
top5 = AverageMeter()

for ind in range(len(sort_test_label_bin)):
    # measure data loading time
    inputs = sort_test_data_tensor[index:(index+sort_test_label_bin[ind])]
    targets = sort_test_label_tensor[index:(index+sort_test_label_bin[ind])]
    index += sort_test_label_bin[ind]
#     print('inputs.shape: ', inputs.shape)
#     print('targets.shape: ', targets.shape)
#     print('index: ', index)
    if use_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

    # compute output
    outputs,_ = model(inputs)
    
    loss = criterion(outputs, targets)
    
    # measure accuracy and record loss
    prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
    
    output = outputs.data
    target = targets.data
    _, pred = output.topk(1, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred)) 
    correct_number = correct[0].view(-1).float().sum(0).item()
    sort_test_correct.append(correct_number)
    
    sort_test_acc.append(prec1.item())
    sort_test_loss.append(loss.item())
    
    losses.update(loss.data, inputs.size()[0])
    top1.update(prec1, inputs.size()[0])
    top5.update(prec5, inputs.size()[0])
#     print('prec1: ', prec1)
#     print('loss: ', loss)
#     break

print('sort_test_acc: ', sort_test_acc)
print('sort_test_loss: ', sort_test_loss)
print('sort_test_correct: ', sort_test_correct)

In [ ]:


# Get the saved location npz model res

evaluation_noise_filepath="/home/nux219/ML_Privacy_Regularization_master/checkpoints_texas_prune_little_model_Memguard/attack/noise_data_No_defense_20_epoch_6_40000_evaluation.npz"

npz_defense=np.load(evaluation_noise_filepath)
f_evaluate_defense=npz_defense['defense_output']
f_evaluate_origin=npz_defense['tc_output']

f_evaluate_defense_logits=npz_defense['defense_output_logits']
f_evaluate_origin_logits=npz_defense['tc_output_logits']

f_evaluate_origin_score=npz_defense['predict_origin']
f_evaluate_defense_score=npz_defense['predict_modified']

In [ ]:


f_evaluate_defense_logits

In [ ]:


dir_path = './model_logits/location/'
# txt_path = '/media/SSD_NVMe_1T/prune_model/model_params/resnet18_v2_ori_2_50/'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
txt_path = dir_path + 'f_evaluate_defense_logits.txt'
np.savetxt(txt_path, f_evaluate_defense_logits, fmt='%f', delimiter=',')
print(f_evaluate_defense_logits.shape)

txt_path = dir_path + 'f_evaluate_origin_logits.txt'
np.savetxt(txt_path, f_evaluate_origin_logits, fmt='%f', delimiter=',')
print(f_evaluate_origin_logits.shape)

txt_path = dir_path + 'f_evaluate_defense.txt'
np.savetxt(txt_path, f_evaluate_defense, fmt='%f', delimiter=',')
print(f_evaluate_defense.shape)

txt_path = dir_path + 'f_evaluate_origin.txt'
np.savetxt(txt_path, f_evaluate_origin, fmt='%f', delimiter=',')
print(f_evaluate_origin.shape)

In [ ]:


np.mean(f_evaluate_origin_score)

In [ ]:


np.std(sort_test_total_out)

In [ ]:


len(sort_test_total_out)

In [ ]:


# prune train

import numpy as np
from sklearn.cluster import KMeans
from pruned_layers import *
import torch.nn as nn
from summary import summary


def prune(net, method='std', q=5.0, s=0.25):
    # Before the training started, generate the mask
    assert isinstance(net, nn.Module)
    for n, m in net.named_modules():
        if isinstance(m, PrunedConv) or isinstance(m, PruneLinear):
            if method == 'percentage':
                m.prune_by_percentage(q)
            elif method == 'std':
                m.prune_by_std(s)

In [ ]:


# from pruned_layers import *
def prune_mod(net, method='std', q=5.0, s=0.25):
    # Before the training started, generate the mask
    assert isinstance(net, nn.Module)
    for n, m in net.named_modules():
        if isinstance(m, PrunedConv) or isinstance(m, PruneLinear):
            if method == 'percentage':
                m.prune_by_percentage_mod(q)
            elif method == 'std':
                m.prune_by_std(s)

In [ ]:


def prune_train(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999):
    # switch to train mode
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        
        for n, m in model.named_modules():
            if isinstance(m, PrunedConv):
                # tensor = m.mask
                weight = m.conv.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('mask.shape: ', tensor.shape)
                grad_tensor = m.conv.weight.grad.data.cpu().numpy()
                # print('grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.conv.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
            elif isinstance(m, PruneLinear):
                # tensor = m.mask
                weight = m.linear.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('linear mask.shape: ', tensor.shape)
                grad_tensor = m.linear.weight.grad.data.cpu().numpy()
                # print('linear grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.linear.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
        
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg)

In [ ]:


def prune_train_with_mask(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999):
    # switch to train mode
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        
        for n, m in model.named_modules():
            if isinstance(m, PrunedConv):
                tensor = m.mask
#                 weight = m.conv.weight.data.cpu().numpy()
#                 # weight_shape = weight.shape
# #                 tensor = np.ones_like(weight)
# #                 tensor = np.where(weight==0, 0, tensor)
# #                 print('mask.shape: ', tensor.shape)
                grad_tensor = m.conv.weight.grad.data.cpu().numpy()
#                 # print('grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.conv.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
#                 m.conv.weight.grad.data = torch.mul(m.conv.weight.grad.data, m.mask)
            elif isinstance(m, PruneLinear):
                tensor = m.mask
#                 weight = m.linear.weight.data.cpu().numpy()
                # weight_shape = weight.shape
#                 tensor = np.ones_like(weight)
#                 tensor = np.where(weight==0, 0, tensor)
#                 print('linear mask.shape: ', tensor.shape)
                grad_tensor = m.linear.weight.grad.data.cpu().numpy()
                # print('linear grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.linear.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
#                 m.linear.weight.grad.data = torch.mul(m.linear.weight.grad.data, m.mask)
            else:
                pass
        
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg, top1.avg)

In [ ]:


def prune_train_max_var_mod5(train_data,labels, model, criterion, optimizer, mean_class,var_n, epoch, use_cuda,num_batchs=999999,num_class=100, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)
    
#     mean_class = torch.from_numpy(np.zeros((num_class,num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_class = np.zeros((num_class,num_class))
#     var_n = np.zeros(num_class)
#     mean_var = torch.from_numpy(np.zeros((num_class))).cuda().type(torch.cuda.FloatTensor)
#     mean_var = np.zeros((num_class))
 
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
#         mean_outputs = torch.mean(soft_outputs,0)
#         var = -torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))

        for i in range(batch_size):
#         var_n[targets[i]] = var_n[targets[i]] + 1
            var_n[targets[i]] += 1
    #         mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i]/var_n[targets[i]]
            mean_class[targets[i]] = mean_class[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + soft_outputs[i].data.cpu().numpy()/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - mean_class[targets[i]]).pow(2))/var_n[targets[i]]
    #         mean_var[targets[i]] = mean_var[targets[i]]*(var_n[targets[i]]-1)/var_n[targets[i]] + torch.sum((soft_outputs[i] - torch.from_numpy(mean_class[targets[i]]).cuda()).pow(2))/var_n[targets[i]]

        temp_mean = torch.from_numpy(mean_class).cuda()
        mean_var = torch.sum((soft_outputs - temp_mean[targets]).pow(2),1)
    
        var = torch.mean(mean_var)*alpha
#         var = -torch.mean(torch.from_numpy(mean_var)).cuda()*alpha
#         var = torch.autograd.Variable(var)
        loss = criterion(outputs, targets) + var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward() #retain_graph=True)
        
                
        for n, m in model.named_modules():
            if isinstance(m, PrunedConv):
                # tensor = m.mask
                weight = m.conv.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('mask.shape: ', tensor.shape)
                grad_tensor = m.conv.weight.grad.data.cpu().numpy()
                # print('grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.conv.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
            elif isinstance(m, PruneLinear):
                # tensor = m.mask
                weight = m.linear.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('linear mask.shape: ', tensor.shape)
                grad_tensor = m.linear.weight.grad.data.cpu().numpy()
                # print('linear grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.linear.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
        
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%20==0:
            print  ('({batch}/{size}) Data: {data:.3f}s |Batch: {bt:.3f}s| Loss: {loss:.4f} |Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def prune_train_mod8(train_data,labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    min_train_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs,_ = model(inputs)
        
        soft_outputs = softmax(outputs)
        max_outputs = torch.max(soft_outputs,1).values
        min_outputs = torch.min(max_outputs)
        
        mean_outputs = torch.mean(soft_outputs)
        var = torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))
        
        loss = criterion(outputs, targets) + alpha*var 
        
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        max_train_outputs.update(torch.mean(max_outputs), inputs.size()[0])
        min_train_outputs.update(min_outputs, inputs.size()[0])
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        
        for n, m in model.named_modules():
            if isinstance(m, PrunedConv):
                # tensor = m.mask
                weight = m.conv.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('mask.shape: ', tensor.shape)
                grad_tensor = m.conv.weight.grad.data.cpu().numpy()
                # print('grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.conv.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
            elif isinstance(m, PruneLinear):
                # tensor = m.mask
                weight = m.linear.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('linear mask.shape: ', tensor.shape)
                grad_tensor = m.linear.weight.grad.data.cpu().numpy()
                # print('linear grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.linear.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
        
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if ind%100==0:
            print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s| Loss: {loss:.4f} | Var: {loss_var:.4f} | Max: {tmax:.4f} | Min: {tmin:.4f}| top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    loss_var=losses_var.avg,
                    tmax = max_train_outputs.avg,
                    tmin = min_train_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
    return (losses.avg,losses_var.avg, top1.avg)
        


In [ ]:


def prune_train_mod9(train_data,labels,untrain_data, untrain_labels, model, criterion, optimizer, epoch, use_cuda,num_batchs=999999, alpha = 1.0, beta = 1.0,lamd = 1.0):
    # switch to train mode
    model.train()
    softmax = nn.Softmax()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_d = AverageMeter()
    losses_a = AverageMeter()
    losses_var = AverageMeter()
    max_train_outputs = AverageMeter()
    max_untrain_outputs = AverageMeter()
    min_untrain_outputs = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    print('alpha: ', alpha, ' beta: ', beta, ', lamd: ',lamd)
    end = time.time()
    len_t =  (len(train_data)//batch_size)-1
    
    for ind in range(len_t):
        if ind > num_batchs:
            break
        # measure data loading time
        inputs = train_data[ind*batch_size:(ind+1)*batch_size]
        targets = labels[ind*batch_size:(ind+1)*batch_size]
        r= np.arange(untrain_data.shape[0])
        np.random.shuffle(r)
        untrain_inputs = untrain_data[r[:batch_size]]
        untrain_targets = untrain_labels[r[:batch_size]]

        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            untrain_inputs, untrain_targets = untrain_inputs.cuda(), untrain_targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        untrain_inputs, untrain_targets = torch.autograd.Variable(untrain_inputs), torch.autograd.Variable(untrain_targets)

        # compute output
        outputs,_ = model(inputs)
        untrain_outputs,_ = model(untrain_inputs)

        soft_outputs = softmax(outputs)
        soft_untrain_outputs = softmax(untrain_outputs)

        sort_soft_outputs,_ = torch.sort(soft_outputs)
        sort_soft_untrain_outputs,_ = torch.sort(soft_untrain_outputs)
        
        mean_outputs = torch.mean(soft_outputs)
        var = torch.mean(torch.sum((soft_outputs - mean_outputs).pow(2),1))
        
        loss_d = alpha*(torch.mean(torch.sum(torch.abs(sort_soft_outputs - sort_soft_untrain_outputs),1)))
#         print('loss_d: ', loss_d)
        max_soft_outputs = torch.max(soft_outputs,1).values
        max_soft_untrain_outputs = torch.mean(torch.max(sort_soft_untrain_outputs,1).values)
        min_soft_untrain_outputs = torch.min(torch.max(sort_soft_untrain_outputs,1).values)
#         print('max_soft_outputs: ', max_soft_outputs)
#         print('max_soft_untrain_outputs: ', max_soft_untrain_outputs)
#         print('min_soft_untrain_outputs: ', min_soft_untrain_outputs)
#         max_untrain_outputs = max_untrain_outputs + max_soft_untrain_outputs
        loss_a = beta*torch.mean(torch.abs(max_soft_outputs - min_soft_untrain_outputs))
#         print('loss_a: ', loss_a)
        loss = criterion(outputs, targets) + loss_d + loss_a + lamd*var
#         print('loss: ', loss)
#         print('inputs.size()[0]: ', inputs.size()[0])
#         loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.data, inputs.size()[0])
        losses_d.update(loss_d.data, untrain_inputs.size()[0])
        losses_a.update(loss_a.data, untrain_inputs.size()[0])
        max_train_outputs.update(torch.mean(max_soft_outputs), inputs.size()[0])
#         max_untrain_outputs.update(max_soft_untrain_outputs, 1)
        max_untrain_outputs.update(max_soft_untrain_outputs, inputs.size()[0])
        min_untrain_outputs.update(min_soft_untrain_outputs, inputs.size()[0])
        top1.update(prec1, inputs.size()[0])
        top5.update(prec5, inputs.size()[0])
        losses_var.update(var.data, inputs.size()[0])
        # compute gradient and do SGD step
        
        optimizer.zero_grad()
        loss.backward()
        
        for n, m in model.named_modules():
            if isinstance(m, PrunedConv):
                # tensor = m.mask
                weight = m.conv.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('mask.shape: ', tensor.shape)
                grad_tensor = m.conv.weight.grad.data.cpu().numpy()
                # print('grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.conv.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
            elif isinstance(m, PruneLinear):
                # tensor = m.mask
                weight = m.linear.weight.data.cpu().numpy()
                # weight_shape = weight.shape
                tensor = np.ones_like(weight)
                tensor = np.where(weight==0, 0, tensor)
                # print('linear mask.shape: ', tensor.shape)
                grad_tensor = m.linear.weight.grad.data.cpu().numpy()
                # print('linear grad.shape: ', grad_tensor.shape)
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                m.linear.weight.grad.data = torch.from_numpy(grad_tensor).to(device)
        
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        # plot progress
        if ind%100==0:
            print  ('({batch}/{size})| Loss: {loss:.4f}| Loss_d: {loss_d:.4f}| Loss_a:{loss_a:.4f}|max train:{max_train:.4f}|max untrain:{max_untrain:.4f}|min: {min_untrain:.4f}|top1:{top1: .4f}|top5:{top5: .4f}'.format(
                    batch=ind + 1,
                    size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
                    loss=losses.avg,
                    loss_d = losses_d.avg,
                    loss_a = losses_a.avg,
                    max_train = max_train_outputs.avg,
                    max_untrain = max_untrain_outputs.avg,    #torch.mean(max_untrain_outputs),
                    min_untrain = min_untrain_outputs.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    ))
#             print  ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} |  Loss_d: {loss_d:.4f} |  Loss_a: {loss_a:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
#                     batch=ind + 1,
#                     size=len_t,
#                     data=data_time.avg,
#                     bt=batch_time.avg,
#                     loss=losses.avg,
#                     loss_d = losses_d.avg,
#                     loss_a = losses_a.avg,

#                     top1=top1.avg,
#                     top5=top5.avg,
#                     ))
    return (losses.avg, losses_d.avg, losses_a.avg, losses_var.avg, top1.avg)
        


In [ ]:


def prune4layer(net, method='std', q=5.0, s=0.25, last_q=0.0,last_s=0.25, l=4):
    # Before the training started, generate the mask
    i = 0
    assert isinstance(net, nn.Module)
    for n, m in net.named_modules():
        if i < l:
            if isinstance(m, PrunedConv) or isinstance(m, PruneLinear):
                i += 1
                if method == 'percentage':
                    m.prune_by_percentage(q)
                elif method == 'std':
                    m.prune_by_std(s)
        elif i >= l:
            if isinstance(m, PrunedConv) or isinstance(m, PruneLinear):
                i += 1
                if method == 'percentage':
                    m.prune_by_percentage(last_q)
                elif method == 'std':
                    m.prune_by_std(last_s)

In [ ]:



train_classifier_data_length  = 10000

train_classifier_data = X[:train_classifier_data_length]
train_classifier_label = Y[:train_classifier_data_length]
test_data = X[40000:60000]
test_label = Y[40000:60000]

lr = 0.001
state={}
state['lr']=lr
# net = Texas()
net = Texas_2layer()
net = torch.nn.DataParallel(net).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
batch_size = 128

In [ ]:


resume='./checkpoints_texas_prune_little_model_defense/Small_2layer_No_defenss_epoch_40'
print('==> Resuming from checkpoint..')
assert os.path.isfile(resume), 'Error: no checkpoint directory found!'
checkpoint = os.path.dirname(resume)
checkpoint = torch.load(resume)
net.load_state_dict(checkpoint['state_dict'])
epoch = 0
test_loss, final_test_acc = test(torch.from_numpy(test_data).type(torch.FloatTensor) ,torch.from_numpy(test_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

trainset_loss, trainset_acc = test(torch.from_numpy(train_classifier_data).type(torch.FloatTensor) ,torch.from_numpy(train_classifier_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('trainset_loss: {trainset_loss: .4f}, trainset_acc: {trainset_acc: .4f}'.format(trainset_loss=trainset_loss, trainset_acc=trainset_acc))

In [ ]:


print("-----Summary before pruning-----")
summary(net)
print("-------------------------------")

In [ ]:


# prune(net, method='percentage', q=98, s=0.75)
prune4layer(net, method='percentage', q=99, last_q=0,l=2)
# prune(net, method='std', q=98, s=2.6)
test_loss, final_test_acc = test(torch.from_numpy(test_data).type(torch.FloatTensor) ,torch.from_numpy(test_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

In [ ]:


print("-----Summary After one time pruning-----")
summary(net)
print("-------------------------------")

In [ ]:


def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in [400, 1500]:
#     if epoch in [100, 90]:
        state['lr'] *= 10 
#         state['lr'] *= 1
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']
    if epoch in [ 200,40,500, 700]:
#     if epoch in [100, 90]:
        state['lr'] *= 0.1 
#         state['lr'] *= 1
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [ ]:


# lr = 0.001
batch_size=128
best_acc = 0.0
best_epoch = 0
alpha = 100
epochs = 80
gce_alpha = 0.333 
criterion = GuidedComplementEntropy(gce_alpha)
for epoch in range(0, epochs):
#     if epoch == 3:
#         alpha = 10.0
    adjust_learning_rate(optimizer, epoch)
    r= np.arange(len(train_classifier_data))
    np.random.shuffle(r)
    train_classifier_data = train_classifier_data[r]
    train_classifier_label = train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(train_classifier_label).type(torch.LongTensor)
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss,train_var, train_acc = prune_train_mod8(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, epoch, use_cuda, alpha = alpha)
    print ('train acc: %.4f, train loss: %.4f, train_var: %.4f'%(train_acc, train_loss,train_var))
    test_loss,test_var, test_acc = test_mod8(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, test_var: %.4f, best acc: %.4f'%(test_acc, test_loss, test_var, best_acc))
    
#     save model
    if is_best or epoch+1 == epochs:
        if best_acc > 54:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_GCE_Modify9_var_min_alpha100_defense_epoch_7_prune_99_4_layer_50_defense_epoch_%d'%(epoch+1))

In [ ]:


# lr = 0.00010
batch_size=128
best_acc = 0.0
best_epoch = 0
alpha = 0.0
beta = 0.0
lamd = 100.0
epochs=120
gce_alpha = 0.333 
criterion = GuidedComplementEntropy(gce_alpha)
for epoch in range(0, epochs):
#     if epoch == 2:
#         best_acc = 0.0
#         alpha = 5.0
#     adjust_learning_rate(optimizer, epoch)
#     if epoch== 30:
#         alpha = 3.0
    
    r= np.arange(len(small_train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = small_train_classifier_data[r]
    s_train_classifier_label = small_train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    r2= np.arange(len(train_attack_data)*0.125)
    r2 = np.intc(r2) 
    untrain_classifier_data = train_attack_data[r2]
    untrain_classifier_label = train_attack_label[r2]

    untrain_classifier_data_tensor = torch.from_numpy(untrain_classifier_data).type(torch.FloatTensor)
    untrain_classifier_label_tensor = torch.from_numpy(untrain_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_loss_d,train_loss_a, train_loss_var,  train_acc = prune_train_mod9(train_classifier_data_tensor,train_classifier_label_tensor,
                                                                   untrain_classifier_data_tensor, untrain_classifier_label_tensor, 
                                                                   net, criterion, optimizer, epoch, use_cuda, alpha = alpha, beta = beta, lamd = lamd)
    print ('train acc: %.4f, train loss: %.4f, train distance loss: %.4f, train var loss: %.4f'%(train_acc, train_loss, train_loss_d,train_loss_var))
    test_loss,test_var, test_acc = test_mod8(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, test_var: %.4f, best acc: %.4f'%(test_acc, test_loss, test_var, best_acc))
    
#     save model
    if is_best or epoch+1 == epochs:
        if epoch > 0:
#             best_acc = 0
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_GCE_Modify10_alpha3_beta0_lamd100_defense_prune_99_retrain1_epoch_%d'%(epoch+1))
        else : best_acc = 0

In [ ]:


# lr = 0.00010
batch_size=128
best_acc = 0.0
best_epoch = 0
alpha = 0.0
beta = 0.0
lamd = 100.0
epochs=200
gce_alpha = 0.333 
criterion = GuidedComplementEntropy(gce_alpha)
for epoch in range(120, epochs):
#     if epoch == 2:
#         best_acc = 0.0
#         alpha = 5.0
#     adjust_learning_rate(optimizer, epoch)
#     if epoch== 30:
#         alpha = 3.0
    
    r= np.arange(len(small_train_classifier_data))
    np.random.shuffle(r)
    s_train_classifier_data = small_train_classifier_data[r]
    s_train_classifier_label = small_train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(s_train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(s_train_classifier_label).type(torch.LongTensor)
    
    r2= np.arange(len(train_attack_data)*0.125)
    r2 = np.intc(r2) 
    untrain_classifier_data = train_attack_data[r2]
    untrain_classifier_label = train_attack_label[r2]

    untrain_classifier_data_tensor = torch.from_numpy(untrain_classifier_data).type(torch.FloatTensor)
    untrain_classifier_label_tensor = torch.from_numpy(untrain_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_loss_d,train_loss_a, train_loss_var,  train_acc = prune_train_mod9(train_classifier_data_tensor,train_classifier_label_tensor,
                                                                   untrain_classifier_data_tensor, untrain_classifier_label_tensor, 
                                                                   net, criterion, optimizer, epoch, use_cuda, alpha = alpha, beta = beta, lamd = lamd)
    print ('train acc: %.4f, train loss: %.4f, train distance loss: %.4f, train var loss: %.4f'%(train_acc, train_loss, train_loss_d,train_loss_var))
    test_loss,test_var, test_acc = test_mod8(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, test_var: %.4f, best acc: %.4f'%(test_acc, test_loss, test_var, best_acc))
    
#     save model
    if is_best or epoch+1 == epochs:
        if epoch > 0:
#             best_acc = 0
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_GCE_Modify10_alpha3_beta0_lamd100_defense_prune_99_retrain1_epoch_%d'%(epoch+1))
        else : best_acc = 0

In [ ]:


# finetune after pruning 98
# Small_No_defense_20_epoch_12

batch_size=128
best_acc = 0.0
best_epoch = 0
epochs = 140
for epoch in range(0, epochs):
#     adjust_learning_rate(optimizer, epoch)
#     r= np.arange(len(train_classifier_data))
    np.random.shuffle(r)
    train_classifier_data = train_classifier_data[r]
    train_classifier_label = train_classifier_label[r]
    
    train_classifier_data_tensor = torch.from_numpy(train_classifier_data).type(torch.FloatTensor)
    train_classifier_label_tensor = torch.from_numpy(train_classifier_label).type(torch.LongTensor)
    
    
    test_data_tensor = torch.from_numpy(test_data).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label).type(torch.LongTensor)
    
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
        
    train_loss, train_acc = prune_train(train_classifier_data_tensor,train_classifier_label_tensor, net, criterion, optimizer, epoch, use_cuda)
    print ('train acc: %.4f, train loss: %.4f'%(train_acc, train_loss))
    test_loss, test_acc = test(test_data_tensor,test_label_tensor, net, criterion, epoch, use_cuda)
    #privacy_loss, privacy_acc = privacy_train(trainloader,testloader,model,inferenece_model,criterion_attack,optimizer_mem,epoch,use_cuda)
    
    # append logger file
    
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    print ('test acc: %.4f, test loss: %.4f, best acc: %.4f'%(test_acc, test_loss, best_acc))
    
    # save model
    if is_best or epoch+1 == epochs:
        if best_acc > 54:
            best_epoch = epoch+1;
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_No_defense_20_epoch_12_prune_98_retrain_epoch_%d'%(epoch+1))

In [ ]:


print("-----Summary After fine tune pruning-----")
summary(net)
print("-------------------------------")

In [ ]:


attack_model = InferenceAttack_HZ(100)
attack_model = torch.nn.DataParallel(attack_model).cuda()
attack_criterion = nn.MSELoss()
attack_optimizer = optim.Adam(attack_model.parameters(),lr=0.0001)

In [ ]:


# no defense attack on model
# test_loss:  1.3502, test_acc:  61.5184
# trainset_loss:  1.0334, trainset_acc:  67.9202
# No_defense_20_epoch_2_prune_99_retrain_epoch_99

best_acc= 0.0
batch_size=128
epochs = 100
for epoch in range(0, epochs):
#     adjust_learning_rate_attack(attack_optimizer, epoch)
    
    r= np.arange(len(train_classifier_data_tr_attack))
    np.random.shuffle(r)
    r1= np.arange(len(train_attack_data))
    np.random.shuffle(r1)
    r2 = r1[r]

    train_classifier_data_tr_attack = train_classifier_data_tr_attack[r]
    train_classifier_label_tr_attack = train_classifier_label_tr_attack[r]

    tr_attack_data = train_attack_data[r2]
    tr_attack_label = train_attack_label[r2]


    r= np.arange(len(train_classifier_data_te_attack))
    np.random.shuffle(r)
    r1= np.arange(len(test_data))
    np.random.shuffle(r1)
    r2 = r1[r]

    train_classifier_data_te_attack = train_classifier_data_te_attack[r]
    train_classifier_label_te_attack = train_classifier_label_te_attack[r]

    test_data_attack = test_data[r2]
    test_label_attack = test_label[r2]

    
    train_classifier_data_tr_attack_tensor = torch.from_numpy(train_classifier_data_tr_attack).type(torch.FloatTensor)
    train_classifier_label_tr_attack_tensor = torch.from_numpy(train_classifier_label_tr_attack).type(torch.LongTensor)

    train_classifier_data_te_attack_tensor = torch.from_numpy(train_classifier_data_te_attack).type(torch.FloatTensor)
    train_classifier_label_te_attack_tensor = torch.from_numpy(train_classifier_label_te_attack).type(torch.LongTensor)


#     r= np.arange(len(train_attack_data))
#     np.random.shuffle(r)

#     train_attack_data = train_attack_data[r]
#     train_attack_label = train_attack_label[r]

    train_attack_data_tensor = torch.from_numpy(tr_attack_data).type(torch.FloatTensor)
    train_attack_label_tensor = torch.from_numpy(tr_attack_label).type(torch.LongTensor)

    test_data_tensor = torch.from_numpy(test_data_attack).type(torch.FloatTensor)
    test_label_tensor = torch.from_numpy(test_label_attack).type(torch.LongTensor)
    print('\nEpoch: [%d | %d] , lr : %f'% (epoch + 1, epochs, state['lr']))


    train_loss, train_acc = train_attack(train_classifier_data_tr_attack_tensor,train_classifier_label_tr_attack_tensor
                                         ,train_attack_data_tensor,train_attack_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    print ('train acc',train_acc)
    test_loss, test_acc = test_attack(train_classifier_data_te_attack_tensor,train_classifier_label_te_attack_tensor
                                         ,test_data_tensor,test_label_tensor,net,attack_model,criterion,attack_criterion,optimizer,attack_optimizer,epoch,use_cuda)

    is_best = test_acc>best_acc
    
    best_acc = max(test_acc, best_acc)
    if is_best:
        save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': attack_model.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : attack_model.state_dict(),
                }, False, filename='No_defense_20_epoch_2_prune_99_retrain_epoch_99_longtrain_attack_epoch%d'%(epoch+1))
    
    
    print ('test acc',test_acc,best_acc)

print('Best classification acc:%.4f'%(final_test_acc))
print('Best attack acc:%.4f'%(best_acc))

In [ ]:


# attack on model with no defense 
# No_defense_epoch_2
# prune 85% acc: 61.1128
best_acc= 0.0
batch_size=128
epochs = 50

    if is_best:
        save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': attack_model.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : attack_model.state_dict(),
                }, False, filename='No_defense_epoch_2_prune_85_attack_epoch%d'%(epoch+1))
    
    
    print ('test acc',test_acc,best_acc)

print('Best classification acc:%.4f'%(final_test_acc))
print('Best attack acc:%.4f'%(best_acc))

In [ ]:


def _quantize_layer(weight, bits=8):
    """
    :param weight: A numpy array of any shape.
    :param bits: quantization bits for weight sharing.
    :return quantized weights and centriods.
    """
    shape = weight.shape
    min = np.min(weight)
    max = np.max(weight)
    # Get zero maskings
    zero_masking = (weight == 0)
    # Consider the 0 parameters, we have to reserve a centriod for them.
    num_clusters = 2 ** bits + 1
    kmeans_init = np.linspace(min, max, num_clusters)
    kmeans = KMeans(n_clusters=num_clusters, init=kmeans_init.reshape(-1, 1), n_init=1,
                    precompute_distances=True, algorithm='full')
    kmeans.fit(weight.reshape(-1, 1))
    new_weight = kmeans.cluster_centers_[kmeans.labels_].reshape(shape)
    # Recover the zero masking
    new_weight[zero_masking] = 0.
    # Now exclude the zero center
    centers = kmeans.cluster_centers_.copy()
    zero_center_idx = np.argmin(np.abs(centers))
    centers_ = []
    for i in range(len(centers)):
        if i != zero_center_idx:
            centers_.append(centers[i])
    centers_ = np.array(centers_)
    return new_weight, centers_

def quantize_whole_model(net, bits=8):
    """
    Quantize the whole model.
    :param net: (object) network model.
    :return: centroids of each weight layer, used in the quantization codebook.
    """
    cluster_centers = []
    assert isinstance(net, nn.Module)
    layer_ind = 0
    for n, m in net.named_modules():
        if isinstance(m, PrunedConv):
            weight = m.conv.weight.data.cpu().numpy()
            weight, centers = _quantize_layer(weight, bits=bits)
            centers = centers.flatten()
            cluster_centers.append(centers)
            m.conv.weight.data = torch.from_numpy(weight).to(device)
            layer_ind += 1
            print("Complete %d layers quantization..." %layer_ind)
        elif isinstance(m, PruneLinear):
            weight = m.linear.weight.data.cpu().numpy()
            weight, centers = _quantize_layer(weight, bits=bits)
            centers = centers.flatten()
            cluster_centers.append(centers)
            m.linear.weight.data = torch.from_numpy(weight).to(device)
            layer_ind += 1
            print("Complete %d layers quantization..." %layer_ind)
    return np.array(cluster_centers)

In [ ]:


# from quantize import quantize_whole_model
centers = quantize_whole_model(net, bits=5)

In [ ]:


print("-----Summary After fine tune quantization-----")
summary(net)
print("-------------------------------")

In [ ]:


test_loss, final_test_acc = test(torch.from_numpy(test_data).type(torch.FloatTensor) ,torch.from_numpy(test_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('test_loss: {test_loss: .4f}, test_acc: {test_acc: .4f}'.format(test_loss=test_loss, test_acc=final_test_acc))

trainset_loss, trainset_acc = test(torch.from_numpy(train_classifier_data).type(torch.FloatTensor) ,torch.from_numpy(train_classifier_label).type(torch.LongTensor), net, criterion, epoch, use_cuda)

print ('trainset_loss: {trainset_loss: .4f}, trainset_acc: {trainset_acc: .4f}'.format(trainset_loss=trainset_loss, trainset_acc=trainset_acc))

In [ ]:


save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'acc': test_acc,
                    'best_acc': best_acc,
                    'optimizer' : optimizer.state_dict(),
                }, False, filename='Small_No_defense_20_epoch_12_prune_99_4layer_retrain_epoch_139_quantization_5')

In [ ]:


def test_loader(testloader, model, criterion, epoch, use_cuda):
    global best_acc
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    end = time.time()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        # measure data loading time
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs, y1, y3, y4, y5 = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
#         losses.update(loss.data[0], inputs.size(0))
        losses.update(loss.data, inputs.size(0))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if batch_idx % 20==0:
            
            print ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                        batch=batch_idx + 1,
                        size=len(testloader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        loss=losses.avg,
                        top1=top1.avg,
                        top5=top5.avg,
                        ))
    return (losses.avg, top1.avg)

In [ ]:


def testloader_by_class(testloader, model, criterion, epoch, use_cuda):
    global best_acc
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    end = time.time()
    class_count = np.zeros(100)
    class_correct = np.zeros(100)
    for batch_idx, (inputs, targets) in enumerate(testloader):
        # measure data loading time
        data_time.update(time.time() - end)
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs, y1, y3, y4, y5 = model(inputs)
#         outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        _, pred = outputs.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(targets.view(1, -1).expand_as(pred))
        correct = correct.data.cpu().numpy()
        targets = targets.data.cpu().numpy()
        for i in range(len(targets)):
            class_count[targets[i]] += 1
            class_correct[targets[i]] += correct[0,i]
#         losses.update(loss.data[0], inputs.size(0))
        losses.update(loss.data, inputs.size(0))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        if batch_idx % 20==0:
            
            print ('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                        batch=batch_idx + 1,
                        size=len(testloader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        loss=losses.avg,
                        top1=top1.avg,
                        top5=top5.avg,
                        ))
    return (losses.avg, top1.avg, class_count, class_correct)

In [ ]:


class black_box_benchmarks(object):
    
    def __init__(self, shadow_train_performance, shadow_test_performance, 
                 target_train_performance, target_test_performance, num_classes):
        '''
        each input contains both model predictions (shape: num_data*num_classes) and ground-truth labels. 
        '''
        self.num_classes = num_classes
        
        self.s_tr_outputs, self.s_tr_labels = shadow_train_performance
        self.s_te_outputs, self.s_te_labels = shadow_test_performance
        self.t_tr_outputs, self.t_tr_labels = target_train_performance
        self.t_te_outputs, self.t_te_labels = target_test_performance
        
        self.s_tr_corr = (np.argmax(self.s_tr_outputs, axis=1)==self.s_tr_labels).astype(int)
        self.s_te_corr = (np.argmax(self.s_te_outputs, axis=1)==self.s_te_labels).astype(int)
        self.t_tr_corr = (np.argmax(self.t_tr_outputs, axis=1)==self.t_tr_labels).astype(int)
        self.t_te_corr = (np.argmax(self.t_te_outputs, axis=1)==self.t_te_labels).astype(int)
        
        self.s_tr_conf = np.array([self.s_tr_outputs[i, self.s_tr_labels[i]] for i in range(len(self.s_tr_labels))])
        self.s_te_conf = np.array([self.s_te_outputs[i, self.s_te_labels[i]] for i in range(len(self.s_te_labels))])
        self.t_tr_conf = np.array([self.t_tr_outputs[i, self.t_tr_labels[i]] for i in range(len(self.t_tr_labels))])
        self.t_te_conf = np.array([self.t_te_outputs[i, self.t_te_labels[i]] for i in range(len(self.t_te_labels))])
        
        self.s_tr_entr = self._entr_comp(self.s_tr_outputs)
        self.s_te_entr = self._entr_comp(self.s_te_outputs)
        self.t_tr_entr = self._entr_comp(self.t_tr_outputs)
        self.t_te_entr = self._entr_comp(self.t_te_outputs)
        
        self.s_tr_m_entr = self._m_entr_comp(self.s_tr_outputs, self.s_tr_labels)
        self.s_te_m_entr = self._m_entr_comp(self.s_te_outputs, self.s_te_labels)
        self.t_tr_m_entr = self._m_entr_comp(self.t_tr_outputs, self.t_tr_labels)
        self.t_te_m_entr = self._m_entr_comp(self.t_te_outputs, self.t_te_labels)
        
    
    def _log_value(self, probs, small_value=1e-30):
        return -np.log(np.maximum(probs, small_value))
    
    def _entr_comp(self, probs):
        return np.sum(np.multiply(probs, self._log_value(probs)),axis=1)
    
    def _m_entr_comp(self, probs, true_labels):
        log_probs = self._log_value(probs)
        reverse_probs = 1-probs
        log_reverse_probs = self._log_value(reverse_probs)
        modified_probs = np.copy(probs)
        modified_probs[range(true_labels.size), true_labels] = reverse_probs[range(true_labels.size), true_labels]
        modified_log_probs = np.copy(log_reverse_probs)
        modified_log_probs[range(true_labels.size), true_labels] = log_probs[range(true_labels.size), true_labels]
        return np.sum(np.multiply(modified_probs, modified_log_probs),axis=1)
    
    def _thre_setting(self, tr_values, te_values):
        value_list = np.concatenate((tr_values, te_values))
        thre, max_acc = 0, 0
        for value in value_list:
            tr_ratio = np.sum(tr_values>=value)/(len(tr_values)+0.0)
            te_ratio = np.sum(te_values<value)/(len(te_values)+0.0)
            acc = 0.5*(tr_ratio + te_ratio)
            if acc > max_acc:
                thre, max_acc = value, acc
        return thre
    
    def _mem_inf_via_corr(self):
        # perform membership inference attack based on whether the input is correctly classified or not
        t_tr_acc = np.sum(self.t_tr_corr)/(len(self.t_tr_corr)+0.0)
        t_te_acc = np.sum(self.t_te_corr)/(len(self.t_te_corr)+0.0)
        mem_inf_acc = 0.5*(t_tr_acc + 1 - t_te_acc)
        print('With train acc {acc2:.3f} and test acc {acc3:.3f}\nFor membership inference attack via correctness, the attack acc is {acc1:.3f} '.format(acc1=mem_inf_acc, acc2=t_tr_acc, acc3=t_te_acc) )
        return
    
    def _mem_inf_thre(self, v_name, s_tr_values, s_te_values, t_tr_values, t_te_values):
        # perform membership inference attack by thresholding feature values: the feature can be prediction confidence,
        # (negative) prediction entropy, and (negative) modified entropy
        t_tr_mem, t_te_non_mem = 0, 0
        for num in range(self.num_classes):
            thre = self._thre_setting(s_tr_values[self.s_tr_labels==num], s_te_values[self.s_te_labels==num])
            t_tr_mem += np.sum(t_tr_values[self.t_tr_labels==num]>=thre)
            t_te_non_mem += np.sum(t_te_values[self.t_te_labels==num]<thre)
        mem_inf_acc = 0.5*(t_tr_mem/(len(self.t_tr_labels)+0.0) + t_te_non_mem/(len(self.t_te_labels)+0.0))
        print('For membership inference attack via {n}, the attack acc is {acc:.3f}'.format(n=v_name,acc=mem_inf_acc))
        return
    

    
    def _mem_inf_benchmarks(self, all_methods=True, benchmark_methods=[]):
        if (all_methods) or ('correctness' in benchmark_methods):
            self._mem_inf_via_corr()
        if (all_methods) or ('confidence' in benchmark_methods):
            self._mem_inf_thre('confidence', self.s_tr_conf, self.s_te_conf, self.t_tr_conf, self.t_te_conf)
        if (all_methods) or ('entropy' in benchmark_methods):
            self._mem_inf_thre('entropy', -self.s_tr_entr, -self.s_te_entr, -self.t_tr_entr, -self.t_te_entr)
        if (all_methods) or ('modified entropy' in benchmark_methods):
            self._mem_inf_thre('modified entropy', -self.s_tr_m_entr, -self.s_te_m_entr, -self.t_tr_m_entr, -self.t_te_m_entr)

        return self

In [ ]:


def tensor_data_create(features, labels):
    tensor_x = torch.stack([torch.FloatTensor(i) for i in features]) # transform to torch tensors
    tensor_y = torch.stack([torch.LongTensor([i]) for i in labels])[:,0]
    dataset = torch.utils.data.TensorDataset(tensor_x,tensor_y)
    return dataset


train_datax = train_classifier_data
test_datax = test_data

train_labelx = train_classifier_label
test_labelx = test_label

print('train_classifier_data: ', len(train_labelx))
print('test_data: ', len(test_labelx))

np.random.seed(100)
train_len = train_datax.shape[0]
r = np.arange(train_len)
np.random.shuffle(r)
shadow_indices = r[:train_len//2]
target_indices = r[train_len//2:]

shadow_train_data, shadow_train_label = train_datax[shadow_indices], train_labelx[shadow_indices]
target_train_data, target_train_label = train_datax[target_indices], train_labelx[target_indices]

test_len = 1*train_len
r = np.arange(test_len)
np.random.shuffle(r)
shadow_indices = r[:test_len//2]
target_indices = r[test_len//2:]

shadow_test_data, shadow_test_label = test_datax[shadow_indices], test_labelx[shadow_indices]
target_test_data, target_test_label = test_datax[target_indices], test_labelx[target_indices]

print('shadow_train_data: ', len(shadow_train_data))
print('shadow_test_data: ', len(shadow_test_data))


shadow_train = tensor_data_create(shadow_train_data, shadow_train_label)
shadow_train_loader = torch.utils.data.DataLoader(shadow_train, batch_size=batch_size, shuffle=True, num_workers=1)

shadow_test = tensor_data_create(shadow_test_data, shadow_test_label)
shadow_test_loader = torch.utils.data.DataLoader(shadow_test, batch_size=batch_size, shuffle=True, num_workers=1)

target_train = tensor_data_create(target_train_data, target_train_label)
target_train_loader = torch.utils.data.DataLoader(target_train, batch_size=batch_size, shuffle=True, num_workers=1)

target_test = tensor_data_create(target_test_data, target_test_label)
target_test_loader = torch.utils.data.DataLoader(target_test, batch_size=batch_size, shuffle=True, num_workers=1)
print('Data loading finished')

In [ ]:


# load modified entropy attack
test_loss, final_test_acc = test_loader(shadow_train_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

test_loss, final_test_acc = test_loader(shadow_test_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

test_loss, final_test_acc = test_loader(target_train_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

test_loss, final_test_acc = test_loader(target_test_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

In [ ]:


test_loss, final_test_acc, s_tr_class, s_tr_correct = testloader_by_class(shadow_train_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

test_loss, final_test_acc, t_tr_class, t_tr_correct = testloader_by_class(target_train_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

test_loss, final_test_acc, s_te_class, s_te_correct = testloader_by_class(shadow_test_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

test_loss, final_test_acc, t_te_class, t_te_correct = testloader_by_class(target_test_loader, net, criterion, epoch, use_cuda)
print ('Classification accuracy: %.2f'%(final_test_acc))

In [ ]:


def softmax_by_row(logits, T = 1.0):
    mx = np.max(logits, axis=-1, keepdims=True)
    exp = np.exp((logits - mx)/T)
    denominator = np.sum(exp, axis=-1, keepdims=True)
    return exp/denominator



def _model_predictions(model, dataloader):
    return_outputs, return_labels = [], []

    for (inputs, labels) in dataloader:
        return_labels.append(labels.numpy())
        outputs, y1, y3, y4, y5 = model.forward(inputs.cuda()) 
        return_outputs.append( softmax_by_row(outputs.data.cpu().numpy()) )
    return_outputs = np.concatenate(return_outputs)
    return_labels = np.concatenate(return_labels)
    return (return_outputs, return_labels)

shadow_train_performance = _model_predictions(net, shadow_train_loader)
shadow_test_performance = _model_predictions(net, shadow_test_loader)

target_train_performance = _model_predictions(net, target_train_loader)
target_test_performance = _model_predictions(net, target_test_loader)

In [ ]:


print('Perform membership inference attacks!!!')
MIA = black_box_benchmarks(shadow_train_performance,shadow_test_performance,
                     target_train_performance,target_test_performance,num_classes=100)
res = MIA._mem_inf_benchmarks()

In [ ]:


risk_score = calculate_risk_score(MIA.s_tr_m_entr, MIA.s_te_m_entr, MIA.s_tr_labels, MIA.s_te_labels, MIA.t_tr_m_entr, MIA.t_tr_labels)

In [ ]:


for i in range(len(risk_score)):
#     print(risk_score[i])
    if risk_score[i] is None:
        risk_score[i] = 0

In [ ]:



x = np.sort(risk_score)
y = range(risk_score.size)
plt.plot(x,y, label = 'risk_score')
plt.legend()
plt.show()

plt.hist(risk_score, bins=20, label = 'risk_score')
plt.legend()
plt.show


np.sum(risk_score<=0.5)
count = []
for i in range(10):
    tmp = np.sum(risk_score>(0.1*(i+1)))
    count.append(tmp)
    print(0.1*i+0.1, tmp)

In [ ]:


fig = plt.figure(figsize=(20,15))
# plt.ylim(0, 0.7)
plt.plot(np.sort(res.s_tr_m_entr),range(risk_score.size), label = 's_tr_m_entr')
plt.plot(np.sort(res.s_te_m_entr),range(risk_score.size),  label = 's_te_m_entr')
plt.plot(np.sort(res.t_tr_m_entr),range(risk_score.size),  label = 't_tr_m_entr')
plt.plot(np.sort(res.t_te_m_entr),range(risk_score.size),  label = 't_te_m_entr')
plt.legend()
plt.show()

In [ ]:


# load class wise 

self = res
np.array([self.s_tr_outputs[i, self.s_tr_labels[i]] for i in range(len(self.s_tr_labels))])

t_tr_mem, t_te_non_mem = 0, 0
class_mem_res = []
class_nonmem_res = []
class_attack_acc = []
class_mem_num = []
class_nonmem_num = []
conf_thre = []
s_tr_values = self.s_tr_conf
s_te_values = self.s_te_conf
t_tr_values = self.t_tr_conf
t_te_values = self.t_te_conf
v_name = 'confidence'
for num in range(self.num_classes):
    thre = self._thre_setting(s_tr_values[self.s_tr_labels==num], s_te_values[self.s_te_labels==num])
    conf_thre.append(thre)
    class_mem = np.sum(t_tr_values[self.t_tr_labels==num]>=thre)
    class_nonmem = np.sum(t_te_values[self.t_te_labels==num]<thre)
    class_mem_res.append(class_mem)
    class_nonmem_res.append(class_nonmem)
    t_tr_mem += class_mem
    t_te_non_mem += class_nonmem
    class_mem_num.append(np.sum(self.t_tr_labels==num))
    class_nonmem_num.append(np.sum(self.t_te_labels==num))
    class_attack_acc.append( 0.5*(class_mem/(np.sum(self.t_tr_labels==num)+0.0) + class_nonmem/(np.sum(self.t_te_labels==num)+0.0)) )
    
mem_inf_acc = 0.5*(t_tr_mem/(len(self.t_tr_labels)+0.0) + t_te_non_mem/(len(self.t_te_labels)+0.0))
print('For membership inference attack via {n}, the attack acc is {acc:.3f}'.format(n=v_name,acc=mem_inf_acc))

class_mem_res = np.asarray(class_mem_res)
class_nonmem_res = np.asarray(class_nonmem_res)
class_attack_acc = np.asarray(class_attack_acc)
class_mem_num = np.asarray(class_mem_num)
class_nonmem_num = np.asarray(class_nonmem_num)
conf_thre = np.asarray(conf_thre)

In [ ]:


trainset_correct = s_tr_correct + t_tr_correct
tr_correct_index = np.argsort(trainset_correct)

In [ ]:


fig = plt.figure(figsize=(20,15))
# plt.ylim(0, 0.7)
plt.plot(np.sort(res.s_tr_conf),range(risk_score.size), label = 's_tr_m_entr')
plt.plot(np.sort(res.s_te_conf),range(risk_score.size),  label = 's_te_m_entr')
plt.plot(np.sort(res.t_tr_conf),range(risk_score.size),  label = 't_tr_m_entr')
plt.plot(np.sort(res.t_te_conf),range(risk_score.size),  label = 't_te_m_entr')
plt.legend()
plt.show()

In [ ]:


t_tr_acc =  t_tr_correct/ t_tr_class
# print(t_tr_acc )
t_te_acc = t_te_correct/t_te_class
# print(t_te_acc)

In [ ]:


fig = plt.figure(figsize=(15,10))

plt.plot(np.sort(conf_thre), label = 'conf_thre')

# plt.plot(class_attack_acc, label = 'class_attack_acc')
plt.legend()
plt.show

conf_thre[0]

In [ ]:


t_acc_gap = t_tr_acc - t_te_acc
t_gap_index = np.argsort(t_acc_gap)

fig = plt.figure(figsize=(15,10))

# plt.ylim(0, 0.7)
# plt.plot(class_mem_num, label = 'class_mem')
# plt.plot(class_nonmem_num,  label = 'class_nonmem')
# plt.plot(train_acc[gap_index], label = 'train_acc')
# plt.plot(test_acc[gap_index],  label = 'test_acc')
# plt.plot(t_tr_acc[t_gap_index], label = 't_tr_acc')
# plt.plot(t_te_acc[t_gap_index], label = 't_te_acc')
plt.plot(class_attack_acc[t_gap_index], label = 'class_attack_acc')
plt.plot(t_acc_gap[t_gap_index],  label = 't_acc_gap')
plt.legend()

plt.show

In [ ]:


# load class wise  m_entr_thre

self = res
# np.array([self.s_tr_outputs[i, self.s_tr_labels[i]] for i in range(len(self.s_tr_labels))])

t_tr_mem, t_te_non_mem = 0, 0
class_mem_res = []
class_nonmem_res = []
class_attack_acc = []
class_mem_num = []
class_nonmem_num = []
m_entr_thre = []
s_tr_values = -self.s_tr_m_entr
s_te_values = -self.s_te_m_entr
t_tr_values = -self.t_tr_m_entr
t_te_values = -self.t_te_m_entr
v_name = 'modified entropy'
for num in range(self.num_classes):
    thre = self._thre_setting(s_tr_values[self.s_tr_labels==num], s_te_values[self.s_te_labels==num])
#     thre = -2
    m_entr_thre.append(thre)
    class_mem = np.sum(t_tr_values[self.t_tr_labels==num]>=thre)
    class_nonmem = np.sum(t_te_values[self.t_te_labels==num]<thre)
    class_mem_res.append(class_mem)
    class_nonmem_res.append(class_nonmem)
    t_tr_mem += class_mem
    t_te_non_mem += class_nonmem
    class_mem_num.append(np.sum(self.t_tr_labels==num))
    class_nonmem_num.append(np.sum(self.t_te_labels==num))
    class_attack_acc.append( 0.5*(class_mem/(np.sum(self.t_tr_labels==num)+0.0) + class_nonmem/(np.sum(self.t_te_labels==num)+0.0)) )
    
mem_inf_acc = 0.5*(t_tr_mem/(len(self.t_tr_labels)+0.0) + t_te_non_mem/(len(self.t_te_labels)+0.0))
print('For membership inference attack via {n}, the attack acc is {acc:.3f}'.format(n=v_name,acc=mem_inf_acc))

class_mem_res = np.asarray(class_mem_res)
class_nonmem_res = np.asarray(class_nonmem_res)
class_attack_acc = np.asarray(class_attack_acc)
class_mem_num = np.asarray(class_mem_num)
class_nonmem_num = np.asarray(class_nonmem_num)
m_entr_thre = np.asarray(m_entr_thre)

In [ ]:


fig = plt.figure(figsize=(20,15))
# plt.ylim(0, 0.7)
plt.plot(np.sort(-res.s_tr_m_entr),range(risk_score.size), label = 's_tr_m_entr')
plt.plot(np.sort(-res.s_te_m_entr),range(risk_score.size),  label = 's_te_m_entr')
plt.plot(np.sort(-res.t_tr_m_entr),range(risk_score.size),  label = 't_tr_m_entr')
plt.plot(np.sort(-res.t_te_m_entr),range(risk_score.size),  label = 't_te_m_entr')
plt.legend()
plt.show()

In [ ]:


t_tr_acc =  t_tr_correct/ t_tr_class
# print(t_tr_acc )
t_te_acc = t_te_correct/t_te_class
# print(t_te_acc)

In [ ]:


m_entr_index = np.argsort(m_entr_thre)

fig = plt.figure(figsize=(15,10))

plt.plot(m_entr_thre[m_entr_index], label = 'm_entr_thre')

# plt.plot(class_attack_acc, label = 'class_attack_acc')
plt.legend()
plt.show

m_entr_thre[0]

In [ ]:



t_acc_gap = t_tr_acc - t_te_acc
t_gap_index = np.argsort(t_acc_gap)

fig = plt.figure(figsize=(15,10))

# plt.ylim(0, 0.7)
# plt.plot(class_mem_num, label = 'class_mem')
# plt.plot(class_nonmem_num,  label = 'class_nonmem')
# plt.plot(train_acc[gap_index], label = 'train_acc')
# plt.plot(test_acc[gap_index],  label = 'test_acc')
# plt.plot(t_tr_acc[t_gap_index], label = 't_tr_acc')
# plt.plot(t_te_acc[t_gap_index], label = 't_te_acc')
plt.plot(class_attack_acc[t_gap_index], label = 'class_attack_acc')
plt.plot(t_acc_gap[t_gap_index],  label = 't_acc_gap')
# plt.plot(class_attack_acc[m_entr_index], label = 'class_attack_acc')
# plt.plot(t_acc_gap[m_entr_index],  label = 't_acc_gap')
plt.legend()

plt.show

In [ ]:



class_attack_acc_index = np.argsort(class_attack_acc)

fig = plt.figure(figsize=(15,10))

# plt.ylim(0, 0.7)
# plt.plot(class_mem_num, label = 'class_mem')
# plt.plot(class_nonmem_num,  label = 'class_nonmem')
# plt.plot(train_acc[gap_index], label = 'train_acc')
# plt.plot(test_acc[gap_index],  label = 'test_acc')
# plt.plot(t_tr_acc[t_gap_index], label = 't_tr_acc')
# plt.plot(t_te_acc[t_gap_index], label = 't_te_acc')
plt.plot(class_attack_acc[class_attack_acc_index], label = 'class_attack_acc')
plt.plot(t_acc_gap[class_attack_acc_index],  label = 't_acc_gap')
# plt.plot(class_attack_acc[m_entr_index], label = 'class_attack_acc')
# plt.plot(t_acc_gap[m_entr_index],  label = 't_acc_gap')
plt.legend()